# Demostración

In [1]:
import pandas as pd
import numpy as np
import pickle
import time
import datetime
import re

import sys

from tensorflow import keras

In [2]:
model = keras.models.load_model("modelo_pronosticador_regina.h5")

file = open("norm.params", "rb")
train_mean, train_std, wind_dic = pickle.load(file)
file.close()

In [3]:
drop = [0, 1, 3, 4, 12, 13, 15, 14, 9, 20, 21, 27, 31, 32]

def transform_date(date, date_format):
    date = pd.Timestamp.timestamp(pd.Timestamp(datetime.datetime.strptime(date, date_format)))
    
    day = 24*60*60
    year = 365.2425 * day
    
    day_sin  = np.sin(date * (2 * np.pi / day))
    day_cos  = np.cos(date * (2 * np.pi / day))
    year_sin = np.sin(date * (2 * np.pi / year))
    year_cos = np.cos(date * (2 * np.pi / year))
    
    return [day_sin, day_cos, year_sin, year_cos]

def convert(data, regex = "\s+(?!$)", date_format = "%d/%m/%y %H:%M"):
    data = data.splitlines()
    conv_data = np.zeros((0, 23))
    
    check = len(data) - 90
    if not check==0:
        print("Hay ", check, " muestras de más.") if check > 0 else print("Faltan ", abs(check), " muestras.")
        return
    
    for sample in data:
        sample = re.split(regex, sample)
        date = sample[0] + " " + sample[1]
        sample[8]  = wind_dic[sample[8]]
        sample[11] = wind_dic[sample[11]]
        sample = np.delete(sample, drop)
        sample = sample.astype(float)
        sample[0] -= 2
        
        sample = np.concatenate((sample, transform_date(date, date_format)))
        
        sample = (sample-train_mean)/train_std

        conv_data = np.vstack((conv_data, sample))
    
    conv_data = np.reshape(conv_data, (1, 90, 23))
    return conv_data

## Helada 27/10/21 2:50 AM
### Predicción a las 21:00

In [4]:
x1 = convert("""26/09/21	6:10	8.7	8.8	8.7	54	-0.1	1.6	SW	0.27	14.5	SW	8.7	8.2	8.2	---	767.0	0.00	0.0	0	0.00	0	0.067	0.000	18.3	32	1.3	16.3	0.00	234	1	100.0	10
26/09/21	6:20	8.7	8.8	8.7	53	-0.3	3.2	SW	0.54	19.3	SW	8.7	8.1	8.1	---	766.8	0.00	0.0	0	0.00	0	0.067	0.000	18.2	32	1.2	16.2	0.00	232	1	100.0	10
26/09/21	6:30	8.6	8.7	8.5	52	-0.7	0.0	SSW	0.00	8.0	SW	8.6	8.0	8.0	---	766.9	0.00	0.0	0	0.00	0	0.068	0.000	18.1	32	1.1	16.1	0.00	234	1	100.0	10
26/09/21	6:40	8.4	8.5	8.4	50	-1.4	1.6	SW	0.27	17.7	SW	8.4	7.8	7.8	---	767.2	0.00	0.0	0	0.00	0	0.069	0.000	17.9	32	1.0	16.0	0.00	233	1	100.0	10
26/09/21	6:50	8.5	8.6	8.4	49	-1.6	0.0	SSW	0.00	12.9	SSW	8.5	7.8	7.8	---	767.3	0.00	0.0	0	0.00	0	0.068	0.000	17.8	32	0.9	15.9	0.00	234	1	100.0	10
26/09/21	7:00	8.6	8.7	8.6	47	-2.1	0.0	SW	0.00	8.0	SW	8.6	7.9	7.9	---	767.7	0.00	0.0	0	0.00	0	0.068	0.000	17.8	32	0.8	15.8	0.03	235	1	100.0	10
26/09/21	7:10	8.8	9.0	8.7	44	-2.8	1.6	SW	0.27	14.5	SSW	8.8	8.1	8.1	---	768.2	0.00	0.0	0	0.00	5	0.066	0.000	17.7	32	0.8	15.7	0.00	234	1	100.0	10
26/09/21	7:20	9.1	9.2	9.0	43	-2.8	1.6	SW	0.27	20.9	WSW	9.1	8.3	8.3	---	768.2	0.00	0.0	7	0.10	11	0.064	0.000	17.6	32	0.7	15.6	0.00	233	1	100.0	10
26/09/21	7:30	9.3	9.3	9.2	42	-3.0	1.6	SSW	0.27	16.1	SW	9.3	8.5	8.5	---	768.5	0.00	0.0	18	0.26	32	0.063	0.000	17.5	32	0.6	15.6	0.00	234	1	100.0	10
26/09/21	7:40	9.4	9.5	9.3	42	-2.9	3.2	WSW	0.54	20.9	WSW	9.4	8.6	8.6	---	768.9	0.00	0.0	42	0.60	56	0.062	0.000	17.4	32	0.5	15.4	0.00	233	1	100.0	10
26/09/21	7:50	9.6	9.6	9.5	44	-2.1	0.0	SSW	0.00	9.7	SW	9.6	8.8	8.8	---	769.3	0.00	0.0	42	0.60	49	0.061	0.000	17.3	32	0.4	15.3	0.00	233	1	100.0	10
26/09/21	8:00	9.4	9.5	9.3	46	-1.7	1.6	WSW	0.27	16.1	W	9.4	8.7	8.7	---	769.4	0.00	0.0	55	0.79	60	0.062	0.000	17.2	32	0.4	15.3	0.03	234	1	100.0	10
26/09/21	8:10	9.3	9.3	9.2	45	-2.1	0.0	SSW	0.00	9.7	SSW	9.3	8.6	8.6	---	769.5	0.00	0.0	70	1.00	81	0.063	0.000	17.2	32	0.3	15.2	0.00	234	1	100.0	10
26/09/21	8:20	9.2	9.3	9.2	43	-2.7	0.0	SSW	0.00	8.0	SW	9.2	8.4	8.4	---	769.5	0.00	0.0	97	1.39	116	0.063	0.000	17.1	32	0.2	15.1	0.00	234	1	100.0	10
26/09/21	8:30	9.4	9.5	9.3	41	-3.2	1.6	SSW	0.27	17.7	WSW	9.4	8.6	8.6	---	769.6	0.00	0.0	117	1.68	125	0.062	0.000	16.9	32	0.1	15.0	0.00	232	1	100.0	10
26/09/21	8:40	9.5	9.6	9.4	39	-3.8	4.8	SW	0.80	17.7	SSW	9.2	8.7	8.4	---	769.5	0.00	0.0	133	1.91	135	0.061	0.000	16.9	32	0.1	14.9	0.00	233	1	100.0	10
26/09/21	8:50	9.6	9.6	9.6	38	-4.1	1.6	SW	0.27	17.7	SW	9.6	8.7	8.7	---	769.7	0.00	0.0	142	2.04	146	0.061	0.000	16.9	32	0.1	14.9	0.00	234	1	100.0	10
26/09/21	9:00	9.6	9.7	9.6	38	-4.0	1.6	WSW	0.27	17.7	SW	9.6	8.7	8.7	---	769.7	0.00	0.0	179	2.57	246	0.061	0.000	16.8	32	-0.0	14.8	0.08	233	1	100.0	10
26/09/21	9:10	9.9	10.1	9.7	37	-4.1	1.6	SSW	0.27	16.1	SSW	9.9	9.0	9.0	---	769.8	0.00	0.0	239	3.43	320	0.059	0.000	16.8	32	-0.0	14.8	0.00	235	1	100.0	10
26/09/21	9:20	10.1	10.4	10.0	38	-3.6	0.0	S	0.00	6.4	S	10.1	9.2	9.2	---	769.6	0.00	0.0	327	4.69	394	0.057	0.000	16.7	32	-0.1	14.8	0.00	233	1	100.0	10
26/09/21	9:30	10.7	10.9	10.4	35	-4.1	3.2	S	0.54	20.9	SW	10.7	9.6	9.6	---	769.8	0.00	0.0	433	6.21	478	0.053	0.000	16.7	32	-0.1	14.8	0.00	234	1	100.0	10
26/09/21	9:40	11.3	11.6	10.9	33	-4.4	1.6	S	0.27	8.0	S	11.3	10.0	10.0	---	769.7	0.00	0.0	504	7.22	610	0.049	0.000	16.7	32	-0.1	14.8	0.00	233	1	100.0	10
26/09/21	9:50	11.8	12.0	11.6	31	-4.8	3.2	S	0.54	19.3	SW	11.8	10.4	10.4	---	769.7	0.00	0.0	543	7.78	622	0.045	0.000	16.6	32	-0.2	14.7	0.00	231	1	100.0	10
26/09/21	10:00	12.1	12.3	11.9	30	-5.0	3.2	S	0.54	19.3	S	12.1	10.6	10.6	---	769.7	0.00	0.0	558	8.00	608	0.044	0.000	16.6	32	-0.2	14.7	0.23	234	1	100.0	10
26/09/21	10:10	12.1	12.2	12.1	31	-4.5	3.2	S	0.54	17.7	S	12.1	10.6	10.6	---	769.7	0.00	0.0	575	8.24	606	0.043	0.000	16.6	32	-0.2	14.7	0.00	231	1	100.0	10
26/09/21	10:20	12.2	12.3	12.2	28	-5.8	3.2	SW	0.54	25.7	WSW	12.2	10.6	10.6	---	769.7	0.00	0.0	578	8.29	587	0.042	0.000	16.6	32	-0.2	14.7	0.00	234	1	100.0	10
26/09/21	10:30	12.4	12.7	12.3	28	-5.6	1.6	SSW	0.27	19.3	SW	12.4	10.8	10.8	---	769.7	0.00	0.0	599	8.59	622	0.041	0.000	16.6	32	-0.2	14.7	0.00	234	1	100.0	10
26/09/21	10:40	12.9	13.1	12.7	28	-5.2	1.6	S	0.27	8.0	S	12.9	11.2	11.2	---	769.7	0.00	0.0	675	9.68	698	0.038	0.000	16.7	32	-0.1	14.8	0.00	233	1	100.0	10
26/09/21	10:50	13.1	13.2	13.0	28	-5.0	1.6	SE	0.27	17.7	SE	13.1	11.3	11.3	---	769.7	0.00	0.0	696	9.98	715	0.036	0.000	16.7	32	-0.1	14.8	0.00	234	1	100.0	10
26/09/21	11:00	13.3	13.4	13.1	24	-6.9	3.2	S	0.54	16.1	SW	13.3	11.3	11.3	---	769.5	0.00	0.0	699	10.02	731	0.035	0.000	16.7	32	-0.1	14.8	0.33	234	1	100.0	10
26/09/21	11:10	13.3	13.3	13.2	26	-5.9	1.6	S	0.27	9.7	S	13.3	11.3	11.3	---	769.8	0.00	0.0	626	8.97	754	0.035	0.000	16.7	32	-0.1	14.8	0.00	234	1	100.0	10
26/09/21	11:20	13.3	13.5	13.2	26	-5.9	1.6	SSW	0.27	6.4	SW	13.3	11.3	11.3	---	769.7	0.00	0.0	715	10.25	744	0.035	0.000	16.8	32	-0.0	14.8	0.00	235	1	100.0	10
26/09/21	11:30	13.7	13.8	13.5	24	-6.6	1.6	SSE	0.27	11.3	SW	13.7	11.6	11.6	---	769.6	0.00	0.0	717	10.28	724	0.032	0.000	16.8	32	-0.0	14.8	0.00	233	1	100.0	10
26/09/21	11:40	13.4	13.7	13.3	25	-6.2	1.6	SSE	0.27	24.1	SSW	13.4	11.4	11.4	---	769.6	0.00	0.0	689	9.88	738	0.034	0.000	16.9	32	0.1	14.9	0.00	234	1	100.0	10
26/09/21	11:50	13.4	13.7	13.2	23	-7.3	1.6	S	0.27	20.9	SW	13.4	11.4	11.4	---	769.6	0.00	0.0	740	10.61	747	0.034	0.000	16.9	32	0.1	15.0	0.00	232	1	100.0	10
26/09/21	12:00	13.6	13.7	13.6	22	-7.7	3.2	S	0.54	17.7	WSW	13.6	11.5	11.5	---	769.5	0.00	0.0	754	10.81	763	0.033	0.000	17.0	32	0.2	15.1	0.38	234	1	100.0	10
26/09/21	12:10	13.9	14.1	13.7	22	-7.5	1.6	SW	0.27	12.9	ESE	13.9	11.8	11.8	---	769.6	0.00	0.0	767	11.00	780	0.030	0.000	17.1	32	0.2	15.1	0.00	234	1	100.0	10
26/09/21	12:20	14.1	14.1	13.9	24	-6.2	1.6	WSW	0.27	14.5	WSW	14.1	11.9	11.9	---	769.3	0.00	0.0	800	11.47	814	0.030	0.000	17.2	32	0.3	15.2	0.00	233	1	100.0	10
26/09/21	12:30	14.3	14.4	14.1	20	-8.4	1.6	SW	0.27	14.5	W	14.3	12.0	12.0	---	769.3	0.00	0.0	784	11.24	800	0.028	0.000	17.2	32	0.4	15.3	0.00	235	1	100.0	10
26/09/21	12:40	14.3	14.4	14.2	20	-8.4	3.2	WSW	0.54	17.7	WSW	14.3	12.0	12.0	---	769.3	0.00	0.0	801	11.48	823	0.028	0.000	17.3	32	0.4	15.3	0.00	233	1	100.0	10
26/09/21	12:50	14.6	14.7	14.4	22	-7.0	0.0	SSE	0.00	11.3	SW	14.6	12.3	12.3	---	769.2	0.00	0.0	782	11.21	802	0.026	0.000	17.4	32	0.5	15.4	0.00	234	1	100.0	10
26/09/21	13:00	14.8	15.1	14.6	18	-9.3	1.6	SSE	0.27	8.0	SE	14.8	12.3	12.3	---	769.2	0.00	0.0	800	11.47	809	0.024	0.000	17.4	32	0.5	15.4	0.43	234	1	100.0	10
26/09/21	13:10	14.8	15.1	14.8	18	-9.3	0.0	SSE	0.00	4.8	SE	14.8	12.3	12.3	---	769.1	0.00	0.0	798	11.44	823	0.024	0.000	17.5	32	0.6	15.6	0.00	234	1	100.0	10
26/09/21	13:20	14.9	15.1	14.9	19	-8.5	1.6	ESE	0.27	11.3	ESE	14.9	12.5	12.5	---	769.0	0.00	0.0	810	11.61	814	0.024	0.000	17.6	32	0.7	15.6	0.00	234	1	100.0	10
26/09/21	13:30	15.1	15.2	15.1	19	-8.4	1.6	S	0.27	9.7	S	15.1	12.6	12.6	---	768.7	0.00	0.0	815	11.68	821	0.022	0.000	17.7	32	0.8	15.7	0.00	235	1	100.0	10
26/09/21	13:40	15.4	15.6	15.2	21	-6.8	1.6	ESE	0.27	14.5	ESE	15.4	13.0	13.0	---	768.5	0.00	0.0	809	11.60	826	0.020	0.000	17.8	31	0.4	15.8	0.00	233	1	100.0	10
26/09/21	13:50	15.4	15.7	15.2	17	-9.5	1.6	W	0.27	16.1	W	15.4	12.8	12.8	---	768.4	0.00	0.0	774	11.10	805	0.020	0.000	17.8	31	0.5	15.8	0.00	233	1	100.0	10
26/09/21	14:00	15.5	15.7	15.4	16	-10.3	1.6	WSW	0.27	6.4	WSW	15.5	12.8	12.8	---	768.2	0.00	0.0	790	11.32	798	0.020	0.000	17.9	31	0.6	15.9	0.43	233	1	100.0	10
26/09/21	14:10	15.6	15.7	15.5	16	-10.2	0.0	WNW	0.00	6.4	WNW	15.6	12.9	12.9	---	768.1	0.00	0.0	772	11.07	786	0.019	0.000	18.1	30	0.2	16.0	0.00	234	1	100.0	10
26/09/21	14:20	15.6	15.7	15.6	17	-9.4	1.6	SSW	0.27	6.4	SSW	15.6	12.9	12.9	---	768.0	0.00	0.0	719	10.31	752	0.019	0.000	18.1	30	0.2	16.1	0.00	234	1	100.0	10
26/09/21	14:30	15.9	16.2	15.7	18	-8.4	1.6	W	0.27	24.1	W	15.9	13.3	13.3	---	767.9	0.00	0.0	743	10.65	779	0.017	0.000	18.2	30	0.3	16.2	0.00	232	1	100.0	10
26/09/21	14:40	16.1	16.2	16.0	16	-9.8	0.0	SSW	0.00	6.4	SSW	16.1	13.4	13.4	---	767.9	0.00	0.0	744	10.67	758	0.015	0.000	18.3	30	0.4	16.3	0.00	234	1	100.0	10
26/09/21	14:50	15.9	16.0	15.9	18	-8.4	1.6	SSE	0.27	6.4	ESE	15.9	13.3	13.3	---	767.7	0.00	0.0	733	10.51	740	0.017	0.000	18.4	29	0.1	16.3	0.00	234	1	100.0	10
26/09/21	15:00	16.1	16.2	15.9	16	-9.8	1.6	ESE	0.27	9.7	ESE	16.1	13.3	13.3	---	767.5	0.00	0.0	539	7.73	712	0.016	0.000	18.6	29	0.2	16.4	0.38	234	1	100.0	10
26/09/21	15:10	15.9	16.1	15.9	17	-9.1	0.0	E	0.00	4.8	SSW	15.9	13.3	13.3	---	767.4	0.00	0.0	80	1.15	144	0.017	0.000	18.7	29	0.3	16.6	0.00	233	1	100.0	10
26/09/21	15:20	16.0	16.2	15.9	16	-9.9	1.6	SE	0.27	19.3	SE	16.0	13.3	13.3	---	767.3	0.00	0.0	67	0.96	70	0.016	0.000	18.8	29	0.4	16.7	0.00	234	1	100.0	10
26/09/21	15:30	16.1	16.3	15.9	17	-9.0	0.0	SSE	0.00	6.4	WSW	16.1	13.4	13.4	---	767.2	0.00	0.0	70	1.00	70	0.015	0.000	18.8	29	0.4	16.7	0.00	234	1	100.0	10
26/09/21	15:40	16.3	16.4	16.1	15	-10.4	1.6	W	0.27	6.4	WSW	16.3	13.6	13.6	---	767.1	0.00	0.0	65	0.93	70	0.014	0.000	18.9	28	0.0	16.7	0.00	235	1	100.0	10
26/09/21	15:50	16.2	16.3	16.0	16	-9.7	1.6	W	0.27	11.3	WNW	16.2	13.4	13.4	---	767.1	0.00	0.0	60	0.86	63	0.015	0.000	19.0	28	0.1	16.8	0.00	233	1	100.0	10
26/09/21	16:00	16.4	16.6	16.3	16	-9.5	3.2	W	0.54	20.9	SW	16.4	13.6	13.6	---	767.1	0.00	0.0	57	0.82	58	0.014	0.000	19.1	28	0.2	16.9	0.08	234	1	100.0	10
26/09/21	16:10	16.6	16.7	16.6	14	-11.1	1.6	W	0.27	6.4	SW	16.6	13.7	13.7	---	766.9	0.00	0.0	60	0.86	69	0.012	0.000	19.2	28	0.3	17.0	0.00	234	1	100.0	10
26/09/21	16:20	16.8	16.9	16.7	16	-9.2	1.6	WNW	0.27	6.4	WNW	16.8	13.9	13.9	---	766.9	0.00	0.0	75	1.08	104	0.011	0.000	19.3	28	0.3	17.1	0.00	234	1	100.0	10
26/09/21	16:30	16.8	17.0	16.7	15	-10.0	1.6	W	0.27	11.3	W	16.8	13.9	13.9	---	766.9	0.00	0.0	218	3.13	336	0.010	0.000	19.4	28	0.4	17.2	0.00	234	1	100.0	10
26/09/21	16:40	17.0	17.1	16.9	14	-10.7	0.0	SSE	0.00	6.4	S	17.0	14.1	14.1	---	767.0	0.00	0.0	454	6.51	503	0.009	0.000	19.4	28	0.4	17.2	0.00	232	1	100.0	10
26/09/21	16:50	16.8	16.9	16.8	14	-10.9	0.0	S	0.00	4.8	SSW	16.8	13.9	13.9	---	766.9	0.00	0.0	489	7.01	508	0.010	0.000	19.5	28	0.5	17.3	0.00	234	1	100.0	10
26/09/21	17:00	16.9	16.9	16.8	13	-11.8	0.0	---	0.00	0.0	---	16.9	13.9	13.9	---	766.9	0.00	0.0	446	6.39	462	0.010	0.000	19.6	28	0.6	17.4	0.18	234	1	100.0	10
26/09/21	17:10	16.9	17.1	16.8	15	-9.9	1.6	SW	0.27	9.7	WSW	16.9	14.1	14.1	---	766.7	0.00	0.0	423	6.06	434	0.010	0.000	19.7	28	0.6	17.4	0.00	233	1	100.0	10
26/09/21	17:20	16.9	16.9	16.8	14	-10.8	1.6	W	0.27	9.7	WSW	16.9	13.9	13.9	---	766.8	0.00	0.0	346	4.96	408	0.010	0.000	19.7	28	0.7	17.6	0.00	234	1	100.0	10
26/09/21	17:30	16.7	16.8	16.6	13	-11.9	0.0	W	0.00	6.4	W	16.7	13.7	13.7	---	766.8	0.00	0.0	325	4.66	387	0.012	0.000	19.8	28	0.7	17.6	0.00	233	1	100.0	10
26/09/21	17:40	16.9	17.1	16.8	14	-10.8	0.0	SW	0.00	4.8	SW	16.9	13.9	13.9	---	766.9	0.00	0.0	325	4.66	345	0.010	0.000	19.8	28	0.8	17.7	0.00	234	1	100.0	10
26/09/21	17:50	16.9	17.0	16.8	14	-10.8	0.0	WSW	0.00	4.8	WSW	16.9	13.9	13.9	---	766.9	0.00	0.0	284	4.07	304	0.010	0.000	19.9	28	0.8	17.7	0.00	234	1	100.0	10
26/09/21	18:00	16.7	16.8	16.5	14	-11.0	0.0	WSW	0.00	6.4	WSW	16.7	13.8	13.8	---	767.0	0.00	0.0	248	3.56	265	0.012	0.000	19.9	29	1.4	17.9	0.20	233	1	100.0	10
26/09/21	18:10	16.5	16.6	16.4	15	-10.3	0.0	SSW	0.00	6.4	WSW	16.5	13.7	13.7	---	767.1	0.00	0.0	208	2.98	227	0.013	0.000	20.0	29	1.4	17.9	0.00	234	1	100.0	10
26/09/21	18:20	16.4	16.6	16.3	14	-11.2	0.0	SW	0.00	6.4	SW	16.4	13.6	13.6	---	767.1	0.00	0.0	167	2.39	185	0.014	0.000	20.0	29	1.4	17.9	0.00	234	1	100.0	10
26/09/21	18:30	16.4	16.5	16.3	15	-10.4	0.0	---	0.00	0.0	---	16.4	13.6	13.6	---	767.1	0.00	0.0	127	1.82	146	0.014	0.000	20.0	29	1.4	17.9	0.00	233	1	100.0	10
26/09/21	18:40	16.2	16.4	16.1	14	-11.4	0.0	---	0.00	0.0	---	16.2	13.4	13.4	---	767.2	0.00	0.0	84	1.20	107	0.015	0.000	20.0	29	1.4	17.9	0.00	234	1	100.0	10
26/09/21	18:50	15.9	16.1	15.7	18	-8.4	0.0	---	0.00	0.0	---	15.9	13.2	13.2	---	767.2	0.00	0.0	52	0.75	56	0.017	0.000	20.1	29	1.5	18.0	0.00	234	1	100.0	10
26/09/21	19:00	15.4	15.7	15.1	17	-9.6	0.0	---	0.00	0.0	---	15.4	12.8	12.8	---	767.2	0.00	0.0	24	0.34	39	0.020	0.000	20.1	29	1.5	18.1	0.08	235	1	100.0	10
26/09/21	19:10	14.7	15.1	14.2	20	-8.1	0.0	---	0.00	0.0	---	14.7	12.3	12.3	---	767.3	0.00	0.0	12	0.17	14	0.025	0.000	20.1	29	1.5	18.1	0.00	232	1	100.0	10
26/09/21	19:20	13.7	14.2	13.1	24	-6.5	0.0	---	0.00	0.0	---	13.7	11.6	11.6	---	767.3	0.00	0.0	8	0.11	9	0.032	0.000	20.0	28	0.9	17.9	0.00	234	1	100.0	10
26/09/21	19:30	12.5	13.0	12.0	27	-6.0	0.0	---	0.00	0.0	---	12.5	10.8	10.8	---	767.3	0.00	0.0	2	0.03	5	0.041	0.000	20.0	28	0.9	17.9	0.00	233	1	100.0	10
26/09/21	19:40	11.4	12.0	10.8	30	-5.6	0.0	---	0.00	0.0	---	11.4	10.0	10.0	---	767.4	0.00	0.0	0	0.00	0	0.048	0.000	19.9	27	0.4	17.7	0.00	234	1	100.0	10
26/09/21	19:50	10.3	10.7	9.9	35	-4.5	0.0	---	0.00	0.0	---	10.3	9.3	9.3	---	767.4	0.00	0.0	0	0.00	0	0.056	0.000	19.8	27	0.3	17.6	0.00	232	1	100.0	10
26/09/21	20:00	9.6	9.9	9.3	36	-4.8	0.0	---	0.00	0.0	---	9.6	8.6	8.6	---	767.5	0.00	0.0	0	0.00	0	0.061	0.000	19.7	27	0.2	17.4	0.03	235	1	100.0	10
26/09/21	20:10	8.9	9.3	8.6	37	-5.0	0.0	---	0.00	0.0	---	8.9	8.1	8.1	---	767.5	0.00	0.0	0	0.00	0	0.065	0.000	19.7	27	0.1	17.4	0.00	234	1	100.0	10
26/09/21	20:20	8.3	8.6	8.1	41	-4.2	0.0	---	0.00	0.0	---	8.3	7.4	7.4	---	767.6	0.00	0.0	0	0.00	0	0.070	0.000	19.6	27	0.0	17.2	0.00	233	1	100.0	10
26/09/21	20:30	7.8	8.1	7.7	50	-2.0	0.0	---	0.00	0.0	---	7.8	7.2	7.2	---	767.6	0.00	0.0	0	0.00	0	0.073	0.000	19.4	27	-0.1	17.1	0.00	234	1	100.0	10
26/09/21	20:40	7.7	7.7	7.6	51	-1.8	0.0	---	0.00	0.0	---	7.7	7.1	7.1	---	767.7	0.00	0.0	0	0.00	0	0.074	0.000	19.3	27	-0.2	16.9	0.00	232	1	100.0	10
26/09/21	20:50	7.6	7.7	7.3	51	-1.9	0.0	---	0.00	0.0	---	7.6	6.9	6.9	---	767.7	0.00	0.0	0	0.00	0	0.075	0.000	19.2	27	-0.3	16.8	0.00	232	1	100.0	10
26/09/21	21:00	7.1	7.3	6.9	54	-1.6	0.0	---	0.00	0.0	---	7.1	6.5	6.5	---	767.8	0.00	0.0	0	0.00	0	0.078	0.000	19.1	27	-0.3	16.7	0.03	233	1	100.0	10""")

pred = model.predict(x1)
print(pred)

[[0.5018197]]


### Predicción a las 22:00

In [5]:
x2 = convert("""26/09/21	7:10	8.8	9.0	8.7	44	-2.8	1.6	SW	0.27	14.5	SSW	8.8	8.1	8.1	---	768.2	0.00	0.0	0	0.00	5	0.066	0.000	17.7	32	0.8	15.7	0.00	234	1	100.0	10
26/09/21	7:20	9.1	9.2	9.0	43	-2.8	1.6	SW	0.27	20.9	WSW	9.1	8.3	8.3	---	768.2	0.00	0.0	7	0.10	11	0.064	0.000	17.6	32	0.7	15.6	0.00	233	1	100.0	10
26/09/21	7:30	9.3	9.3	9.2	42	-3.0	1.6	SSW	0.27	16.1	SW	9.3	8.5	8.5	---	768.5	0.00	0.0	18	0.26	32	0.063	0.000	17.5	32	0.6	15.6	0.00	234	1	100.0	10
26/09/21	7:40	9.4	9.5	9.3	42	-2.9	3.2	WSW	0.54	20.9	WSW	9.4	8.6	8.6	---	768.9	0.00	0.0	42	0.60	56	0.062	0.000	17.4	32	0.5	15.4	0.00	233	1	100.0	10
26/09/21	7:50	9.6	9.6	9.5	44	-2.1	0.0	SSW	0.00	9.7	SW	9.6	8.8	8.8	---	769.3	0.00	0.0	42	0.60	49	0.061	0.000	17.3	32	0.4	15.3	0.00	233	1	100.0	10
26/09/21	8:00	9.4	9.5	9.3	46	-1.7	1.6	WSW	0.27	16.1	W	9.4	8.7	8.7	---	769.4	0.00	0.0	55	0.79	60	0.062	0.000	17.2	32	0.4	15.3	0.03	234	1	100.0	10
26/09/21	8:10	9.3	9.3	9.2	45	-2.1	0.0	SSW	0.00	9.7	SSW	9.3	8.6	8.6	---	769.5	0.00	0.0	70	1.00	81	0.063	0.000	17.2	32	0.3	15.2	0.00	234	1	100.0	10
26/09/21	8:20	9.2	9.3	9.2	43	-2.7	0.0	SSW	0.00	8.0	SW	9.2	8.4	8.4	---	769.5	0.00	0.0	97	1.39	116	0.063	0.000	17.1	32	0.2	15.1	0.00	234	1	100.0	10
26/09/21	8:30	9.4	9.5	9.3	41	-3.2	1.6	SSW	0.27	17.7	WSW	9.4	8.6	8.6	---	769.6	0.00	0.0	117	1.68	125	0.062	0.000	16.9	32	0.1	15.0	0.00	232	1	100.0	10
26/09/21	8:40	9.5	9.6	9.4	39	-3.8	4.8	SW	0.80	17.7	SSW	9.2	8.7	8.4	---	769.5	0.00	0.0	133	1.91	135	0.061	0.000	16.9	32	0.1	14.9	0.00	233	1	100.0	10
26/09/21	8:50	9.6	9.6	9.6	38	-4.1	1.6	SW	0.27	17.7	SW	9.6	8.7	8.7	---	769.7	0.00	0.0	142	2.04	146	0.061	0.000	16.9	32	0.1	14.9	0.00	234	1	100.0	10
26/09/21	9:00	9.6	9.7	9.6	38	-4.0	1.6	WSW	0.27	17.7	SW	9.6	8.7	8.7	---	769.7	0.00	0.0	179	2.57	246	0.061	0.000	16.8	32	-0.0	14.8	0.08	233	1	100.0	10
26/09/21	9:10	9.9	10.1	9.7	37	-4.1	1.6	SSW	0.27	16.1	SSW	9.9	9.0	9.0	---	769.8	0.00	0.0	239	3.43	320	0.059	0.000	16.8	32	-0.0	14.8	0.00	235	1	100.0	10
26/09/21	9:20	10.1	10.4	10.0	38	-3.6	0.0	S	0.00	6.4	S	10.1	9.2	9.2	---	769.6	0.00	0.0	327	4.69	394	0.057	0.000	16.7	32	-0.1	14.8	0.00	233	1	100.0	10
26/09/21	9:30	10.7	10.9	10.4	35	-4.1	3.2	S	0.54	20.9	SW	10.7	9.6	9.6	---	769.8	0.00	0.0	433	6.21	478	0.053	0.000	16.7	32	-0.1	14.8	0.00	234	1	100.0	10
26/09/21	9:40	11.3	11.6	10.9	33	-4.4	1.6	S	0.27	8.0	S	11.3	10.0	10.0	---	769.7	0.00	0.0	504	7.22	610	0.049	0.000	16.7	32	-0.1	14.8	0.00	233	1	100.0	10
26/09/21	9:50	11.8	12.0	11.6	31	-4.8	3.2	S	0.54	19.3	SW	11.8	10.4	10.4	---	769.7	0.00	0.0	543	7.78	622	0.045	0.000	16.6	32	-0.2	14.7	0.00	231	1	100.0	10
26/09/21	10:00	12.1	12.3	11.9	30	-5.0	3.2	S	0.54	19.3	S	12.1	10.6	10.6	---	769.7	0.00	0.0	558	8.00	608	0.044	0.000	16.6	32	-0.2	14.7	0.23	234	1	100.0	10
26/09/21	10:10	12.1	12.2	12.1	31	-4.5	3.2	S	0.54	17.7	S	12.1	10.6	10.6	---	769.7	0.00	0.0	575	8.24	606	0.043	0.000	16.6	32	-0.2	14.7	0.00	231	1	100.0	10
26/09/21	10:20	12.2	12.3	12.2	28	-5.8	3.2	SW	0.54	25.7	WSW	12.2	10.6	10.6	---	769.7	0.00	0.0	578	8.29	587	0.042	0.000	16.6	32	-0.2	14.7	0.00	234	1	100.0	10
26/09/21	10:30	12.4	12.7	12.3	28	-5.6	1.6	SSW	0.27	19.3	SW	12.4	10.8	10.8	---	769.7	0.00	0.0	599	8.59	622	0.041	0.000	16.6	32	-0.2	14.7	0.00	234	1	100.0	10
26/09/21	10:40	12.9	13.1	12.7	28	-5.2	1.6	S	0.27	8.0	S	12.9	11.2	11.2	---	769.7	0.00	0.0	675	9.68	698	0.038	0.000	16.7	32	-0.1	14.8	0.00	233	1	100.0	10
26/09/21	10:50	13.1	13.2	13.0	28	-5.0	1.6	SE	0.27	17.7	SE	13.1	11.3	11.3	---	769.7	0.00	0.0	696	9.98	715	0.036	0.000	16.7	32	-0.1	14.8	0.00	234	1	100.0	10
26/09/21	11:00	13.3	13.4	13.1	24	-6.9	3.2	S	0.54	16.1	SW	13.3	11.3	11.3	---	769.5	0.00	0.0	699	10.02	731	0.035	0.000	16.7	32	-0.1	14.8	0.33	234	1	100.0	10
26/09/21	11:10	13.3	13.3	13.2	26	-5.9	1.6	S	0.27	9.7	S	13.3	11.3	11.3	---	769.8	0.00	0.0	626	8.97	754	0.035	0.000	16.7	32	-0.1	14.8	0.00	234	1	100.0	10
26/09/21	11:20	13.3	13.5	13.2	26	-5.9	1.6	SSW	0.27	6.4	SW	13.3	11.3	11.3	---	769.7	0.00	0.0	715	10.25	744	0.035	0.000	16.8	32	-0.0	14.8	0.00	235	1	100.0	10
26/09/21	11:30	13.7	13.8	13.5	24	-6.6	1.6	SSE	0.27	11.3	SW	13.7	11.6	11.6	---	769.6	0.00	0.0	717	10.28	724	0.032	0.000	16.8	32	-0.0	14.8	0.00	233	1	100.0	10
26/09/21	11:40	13.4	13.7	13.3	25	-6.2	1.6	SSE	0.27	24.1	SSW	13.4	11.4	11.4	---	769.6	0.00	0.0	689	9.88	738	0.034	0.000	16.9	32	0.1	14.9	0.00	234	1	100.0	10
26/09/21	11:50	13.4	13.7	13.2	23	-7.3	1.6	S	0.27	20.9	SW	13.4	11.4	11.4	---	769.6	0.00	0.0	740	10.61	747	0.034	0.000	16.9	32	0.1	15.0	0.00	232	1	100.0	10
26/09/21	12:00	13.6	13.7	13.6	22	-7.7	3.2	S	0.54	17.7	WSW	13.6	11.5	11.5	---	769.5	0.00	0.0	754	10.81	763	0.033	0.000	17.0	32	0.2	15.1	0.38	234	1	100.0	10
26/09/21	12:10	13.9	14.1	13.7	22	-7.5	1.6	SW	0.27	12.9	ESE	13.9	11.8	11.8	---	769.6	0.00	0.0	767	11.00	780	0.030	0.000	17.1	32	0.2	15.1	0.00	234	1	100.0	10
26/09/21	12:20	14.1	14.1	13.9	24	-6.2	1.6	WSW	0.27	14.5	WSW	14.1	11.9	11.9	---	769.3	0.00	0.0	800	11.47	814	0.030	0.000	17.2	32	0.3	15.2	0.00	233	1	100.0	10
26/09/21	12:30	14.3	14.4	14.1	20	-8.4	1.6	SW	0.27	14.5	W	14.3	12.0	12.0	---	769.3	0.00	0.0	784	11.24	800	0.028	0.000	17.2	32	0.4	15.3	0.00	235	1	100.0	10
26/09/21	12:40	14.3	14.4	14.2	20	-8.4	3.2	WSW	0.54	17.7	WSW	14.3	12.0	12.0	---	769.3	0.00	0.0	801	11.48	823	0.028	0.000	17.3	32	0.4	15.3	0.00	233	1	100.0	10
26/09/21	12:50	14.6	14.7	14.4	22	-7.0	0.0	SSE	0.00	11.3	SW	14.6	12.3	12.3	---	769.2	0.00	0.0	782	11.21	802	0.026	0.000	17.4	32	0.5	15.4	0.00	234	1	100.0	10
26/09/21	13:00	14.8	15.1	14.6	18	-9.3	1.6	SSE	0.27	8.0	SE	14.8	12.3	12.3	---	769.2	0.00	0.0	800	11.47	809	0.024	0.000	17.4	32	0.5	15.4	0.43	234	1	100.0	10
26/09/21	13:10	14.8	15.1	14.8	18	-9.3	0.0	SSE	0.00	4.8	SE	14.8	12.3	12.3	---	769.1	0.00	0.0	798	11.44	823	0.024	0.000	17.5	32	0.6	15.6	0.00	234	1	100.0	10
26/09/21	13:20	14.9	15.1	14.9	19	-8.5	1.6	ESE	0.27	11.3	ESE	14.9	12.5	12.5	---	769.0	0.00	0.0	810	11.61	814	0.024	0.000	17.6	32	0.7	15.6	0.00	234	1	100.0	10
26/09/21	13:30	15.1	15.2	15.1	19	-8.4	1.6	S	0.27	9.7	S	15.1	12.6	12.6	---	768.7	0.00	0.0	815	11.68	821	0.022	0.000	17.7	32	0.8	15.7	0.00	235	1	100.0	10
26/09/21	13:40	15.4	15.6	15.2	21	-6.8	1.6	ESE	0.27	14.5	ESE	15.4	13.0	13.0	---	768.5	0.00	0.0	809	11.60	826	0.020	0.000	17.8	31	0.4	15.8	0.00	233	1	100.0	10
26/09/21	13:50	15.4	15.7	15.2	17	-9.5	1.6	W	0.27	16.1	W	15.4	12.8	12.8	---	768.4	0.00	0.0	774	11.10	805	0.020	0.000	17.8	31	0.5	15.8	0.00	233	1	100.0	10
26/09/21	14:00	15.5	15.7	15.4	16	-10.3	1.6	WSW	0.27	6.4	WSW	15.5	12.8	12.8	---	768.2	0.00	0.0	790	11.32	798	0.020	0.000	17.9	31	0.6	15.9	0.43	233	1	100.0	10
26/09/21	14:10	15.6	15.7	15.5	16	-10.2	0.0	WNW	0.00	6.4	WNW	15.6	12.9	12.9	---	768.1	0.00	0.0	772	11.07	786	0.019	0.000	18.1	30	0.2	16.0	0.00	234	1	100.0	10
26/09/21	14:20	15.6	15.7	15.6	17	-9.4	1.6	SSW	0.27	6.4	SSW	15.6	12.9	12.9	---	768.0	0.00	0.0	719	10.31	752	0.019	0.000	18.1	30	0.2	16.1	0.00	234	1	100.0	10
26/09/21	14:30	15.9	16.2	15.7	18	-8.4	1.6	W	0.27	24.1	W	15.9	13.3	13.3	---	767.9	0.00	0.0	743	10.65	779	0.017	0.000	18.2	30	0.3	16.2	0.00	232	1	100.0	10
26/09/21	14:40	16.1	16.2	16.0	16	-9.8	0.0	SSW	0.00	6.4	SSW	16.1	13.4	13.4	---	767.9	0.00	0.0	744	10.67	758	0.015	0.000	18.3	30	0.4	16.3	0.00	234	1	100.0	10
26/09/21	14:50	15.9	16.0	15.9	18	-8.4	1.6	SSE	0.27	6.4	ESE	15.9	13.3	13.3	---	767.7	0.00	0.0	733	10.51	740	0.017	0.000	18.4	29	0.1	16.3	0.00	234	1	100.0	10
26/09/21	15:00	16.1	16.2	15.9	16	-9.8	1.6	ESE	0.27	9.7	ESE	16.1	13.3	13.3	---	767.5	0.00	0.0	539	7.73	712	0.016	0.000	18.6	29	0.2	16.4	0.38	234	1	100.0	10
26/09/21	15:10	15.9	16.1	15.9	17	-9.1	0.0	E	0.00	4.8	SSW	15.9	13.3	13.3	---	767.4	0.00	0.0	80	1.15	144	0.017	0.000	18.7	29	0.3	16.6	0.00	233	1	100.0	10
26/09/21	15:20	16.0	16.2	15.9	16	-9.9	1.6	SE	0.27	19.3	SE	16.0	13.3	13.3	---	767.3	0.00	0.0	67	0.96	70	0.016	0.000	18.8	29	0.4	16.7	0.00	234	1	100.0	10
26/09/21	15:30	16.1	16.3	15.9	17	-9.0	0.0	SSE	0.00	6.4	WSW	16.1	13.4	13.4	---	767.2	0.00	0.0	70	1.00	70	0.015	0.000	18.8	29	0.4	16.7	0.00	234	1	100.0	10
26/09/21	15:40	16.3	16.4	16.1	15	-10.4	1.6	W	0.27	6.4	WSW	16.3	13.6	13.6	---	767.1	0.00	0.0	65	0.93	70	0.014	0.000	18.9	28	0.0	16.7	0.00	235	1	100.0	10
26/09/21	15:50	16.2	16.3	16.0	16	-9.7	1.6	W	0.27	11.3	WNW	16.2	13.4	13.4	---	767.1	0.00	0.0	60	0.86	63	0.015	0.000	19.0	28	0.1	16.8	0.00	233	1	100.0	10
26/09/21	16:00	16.4	16.6	16.3	16	-9.5	3.2	W	0.54	20.9	SW	16.4	13.6	13.6	---	767.1	0.00	0.0	57	0.82	58	0.014	0.000	19.1	28	0.2	16.9	0.08	234	1	100.0	10
26/09/21	16:10	16.6	16.7	16.6	14	-11.1	1.6	W	0.27	6.4	SW	16.6	13.7	13.7	---	766.9	0.00	0.0	60	0.86	69	0.012	0.000	19.2	28	0.3	17.0	0.00	234	1	100.0	10
26/09/21	16:20	16.8	16.9	16.7	16	-9.2	1.6	WNW	0.27	6.4	WNW	16.8	13.9	13.9	---	766.9	0.00	0.0	75	1.08	104	0.011	0.000	19.3	28	0.3	17.1	0.00	234	1	100.0	10
26/09/21	16:30	16.8	17.0	16.7	15	-10.0	1.6	W	0.27	11.3	W	16.8	13.9	13.9	---	766.9	0.00	0.0	218	3.13	336	0.010	0.000	19.4	28	0.4	17.2	0.00	234	1	100.0	10
26/09/21	16:40	17.0	17.1	16.9	14	-10.7	0.0	SSE	0.00	6.4	S	17.0	14.1	14.1	---	767.0	0.00	0.0	454	6.51	503	0.009	0.000	19.4	28	0.4	17.2	0.00	232	1	100.0	10
26/09/21	16:50	16.8	16.9	16.8	14	-10.9	0.0	S	0.00	4.8	SSW	16.8	13.9	13.9	---	766.9	0.00	0.0	489	7.01	508	0.010	0.000	19.5	28	0.5	17.3	0.00	234	1	100.0	10
26/09/21	17:00	16.9	16.9	16.8	13	-11.8	0.0	---	0.00	0.0	---	16.9	13.9	13.9	---	766.9	0.00	0.0	446	6.39	462	0.010	0.000	19.6	28	0.6	17.4	0.18	234	1	100.0	10
26/09/21	17:10	16.9	17.1	16.8	15	-9.9	1.6	SW	0.27	9.7	WSW	16.9	14.1	14.1	---	766.7	0.00	0.0	423	6.06	434	0.010	0.000	19.7	28	0.6	17.4	0.00	233	1	100.0	10
26/09/21	17:20	16.9	16.9	16.8	14	-10.8	1.6	W	0.27	9.7	WSW	16.9	13.9	13.9	---	766.8	0.00	0.0	346	4.96	408	0.010	0.000	19.7	28	0.7	17.6	0.00	234	1	100.0	10
26/09/21	17:30	16.7	16.8	16.6	13	-11.9	0.0	W	0.00	6.4	W	16.7	13.7	13.7	---	766.8	0.00	0.0	325	4.66	387	0.012	0.000	19.8	28	0.7	17.6	0.00	233	1	100.0	10
26/09/21	17:40	16.9	17.1	16.8	14	-10.8	0.0	SW	0.00	4.8	SW	16.9	13.9	13.9	---	766.9	0.00	0.0	325	4.66	345	0.010	0.000	19.8	28	0.8	17.7	0.00	234	1	100.0	10
26/09/21	17:50	16.9	17.0	16.8	14	-10.8	0.0	WSW	0.00	4.8	WSW	16.9	13.9	13.9	---	766.9	0.00	0.0	284	4.07	304	0.010	0.000	19.9	28	0.8	17.7	0.00	234	1	100.0	10
26/09/21	18:00	16.7	16.8	16.5	14	-11.0	0.0	WSW	0.00	6.4	WSW	16.7	13.8	13.8	---	767.0	0.00	0.0	248	3.56	265	0.012	0.000	19.9	29	1.4	17.9	0.20	233	1	100.0	10
26/09/21	18:10	16.5	16.6	16.4	15	-10.3	0.0	SSW	0.00	6.4	WSW	16.5	13.7	13.7	---	767.1	0.00	0.0	208	2.98	227	0.013	0.000	20.0	29	1.4	17.9	0.00	234	1	100.0	10
26/09/21	18:20	16.4	16.6	16.3	14	-11.2	0.0	SW	0.00	6.4	SW	16.4	13.6	13.6	---	767.1	0.00	0.0	167	2.39	185	0.014	0.000	20.0	29	1.4	17.9	0.00	234	1	100.0	10
26/09/21	18:30	16.4	16.5	16.3	15	-10.4	0.0	---	0.00	0.0	---	16.4	13.6	13.6	---	767.1	0.00	0.0	127	1.82	146	0.014	0.000	20.0	29	1.4	17.9	0.00	233	1	100.0	10
26/09/21	18:40	16.2	16.4	16.1	14	-11.4	0.0	---	0.00	0.0	---	16.2	13.4	13.4	---	767.2	0.00	0.0	84	1.20	107	0.015	0.000	20.0	29	1.4	17.9	0.00	234	1	100.0	10
26/09/21	18:50	15.9	16.1	15.7	18	-8.4	0.0	---	0.00	0.0	---	15.9	13.2	13.2	---	767.2	0.00	0.0	52	0.75	56	0.017	0.000	20.1	29	1.5	18.0	0.00	234	1	100.0	10
26/09/21	19:00	15.4	15.7	15.1	17	-9.6	0.0	---	0.00	0.0	---	15.4	12.8	12.8	---	767.2	0.00	0.0	24	0.34	39	0.020	0.000	20.1	29	1.5	18.1	0.08	235	1	100.0	10
26/09/21	19:10	14.7	15.1	14.2	20	-8.1	0.0	---	0.00	0.0	---	14.7	12.3	12.3	---	767.3	0.00	0.0	12	0.17	14	0.025	0.000	20.1	29	1.5	18.1	0.00	232	1	100.0	10
26/09/21	19:20	13.7	14.2	13.1	24	-6.5	0.0	---	0.00	0.0	---	13.7	11.6	11.6	---	767.3	0.00	0.0	8	0.11	9	0.032	0.000	20.0	28	0.9	17.9	0.00	234	1	100.0	10
26/09/21	19:30	12.5	13.0	12.0	27	-6.0	0.0	---	0.00	0.0	---	12.5	10.8	10.8	---	767.3	0.00	0.0	2	0.03	5	0.041	0.000	20.0	28	0.9	17.9	0.00	233	1	100.0	10
26/09/21	19:40	11.4	12.0	10.8	30	-5.6	0.0	---	0.00	0.0	---	11.4	10.0	10.0	---	767.4	0.00	0.0	0	0.00	0	0.048	0.000	19.9	27	0.4	17.7	0.00	234	1	100.0	10
26/09/21	19:50	10.3	10.7	9.9	35	-4.5	0.0	---	0.00	0.0	---	10.3	9.3	9.3	---	767.4	0.00	0.0	0	0.00	0	0.056	0.000	19.8	27	0.3	17.6	0.00	232	1	100.0	10
26/09/21	20:00	9.6	9.9	9.3	36	-4.8	0.0	---	0.00	0.0	---	9.6	8.6	8.6	---	767.5	0.00	0.0	0	0.00	0	0.061	0.000	19.7	27	0.2	17.4	0.03	235	1	100.0	10
26/09/21	20:10	8.9	9.3	8.6	37	-5.0	0.0	---	0.00	0.0	---	8.9	8.1	8.1	---	767.5	0.00	0.0	0	0.00	0	0.065	0.000	19.7	27	0.1	17.4	0.00	234	1	100.0	10
26/09/21	20:20	8.3	8.6	8.1	41	-4.2	0.0	---	0.00	0.0	---	8.3	7.4	7.4	---	767.6	0.00	0.0	0	0.00	0	0.070	0.000	19.6	27	0.0	17.2	0.00	233	1	100.0	10
26/09/21	20:30	7.8	8.1	7.7	50	-2.0	0.0	---	0.00	0.0	---	7.8	7.2	7.2	---	767.6	0.00	0.0	0	0.00	0	0.073	0.000	19.4	27	-0.1	17.1	0.00	234	1	100.0	10
26/09/21	20:40	7.7	7.7	7.6	51	-1.8	0.0	---	0.00	0.0	---	7.7	7.1	7.1	---	767.7	0.00	0.0	0	0.00	0	0.074	0.000	19.3	27	-0.2	16.9	0.00	232	1	100.0	10
26/09/21	20:50	7.6	7.7	7.3	51	-1.9	0.0	---	0.00	0.0	---	7.6	6.9	6.9	---	767.7	0.00	0.0	0	0.00	0	0.075	0.000	19.2	27	-0.3	16.8	0.00	232	1	100.0	10
26/09/21	21:00	7.1	7.3	6.9	54	-1.6	0.0	---	0.00	0.0	---	7.1	6.5	6.5	---	767.8	0.00	0.0	0	0.00	0	0.078	0.000	19.1	27	-0.3	16.7	0.03	233	1	100.0	10
26/09/21	21:10	6.8	6.9	6.6	52	-2.4	0.0	---	0.00	0.0	---	6.8	6.2	6.2	---	767.9	0.00	0.0	0	0.00	0	0.080	0.000	19.0	27	-0.4	16.6	0.00	234	1	100.0	10
26/09/21	21:20	6.4	6.6	6.3	55	-2.0	0.0	---	0.00	0.0	---	6.4	5.9	5.9	---	767.9	0.00	0.0	0	0.00	0	0.083	0.000	18.9	27	-0.5	16.5	0.00	233	1	100.0	10
26/09/21	21:30	6.1	6.2	5.9	51	-3.3	0.0	---	0.00	0.0	---	6.1	5.4	5.4	---	767.9	0.00	0.0	0	0.00	0	0.085	0.000	18.8	28	-0.1	16.6	0.00	234	1	100.0	10
26/09/21	21:40	6.0	6.1	5.9	54	-2.6	0.0	---	0.00	0.0	---	6.0	5.4	5.4	---	768.1	0.00	0.0	0	0.00	0	0.086	0.000	18.7	28	-0.2	16.5	0.00	233	1	100.0	10
26/09/21	21:50	6.3	6.6	6.1	60	-0.9	0.0	---	0.00	0.0	---	6.3	5.9	5.9	---	768.3	0.00	0.0	0	0.00	0	0.083	0.000	18.6	28	-0.3	16.3	0.00	233	1	100.0	10
26/09/21	22:00	6.6	6.7	6.3	56	-1.6	0.0	---	0.00	0.0	---	6.6	6.0	6.0	---	768.2	0.00	0.0	0	0.00	0	0.082	0.000	18.5	28	-0.4	16.2	0.03	234	1	100.0	10""")

pred = model.predict(x2)

print(pred)

[[0.5214874]]


### Predicción a las 23:00

In [6]:
x3 = convert("""26/09/21	8:10	9.3	9.3	9.2	45	-2.1	0.0	SSW	0.00	9.7	SSW	9.3	8.6	8.6	---	769.5	0.00	0.0	70	1.00	81	0.063	0.000	17.2	32	0.3	15.2	0.00	234	1	100.0	10
26/09/21	8:20	9.2	9.3	9.2	43	-2.7	0.0	SSW	0.00	8.0	SW	9.2	8.4	8.4	---	769.5	0.00	0.0	97	1.39	116	0.063	0.000	17.1	32	0.2	15.1	0.00	234	1	100.0	10
26/09/21	8:30	9.4	9.5	9.3	41	-3.2	1.6	SSW	0.27	17.7	WSW	9.4	8.6	8.6	---	769.6	0.00	0.0	117	1.68	125	0.062	0.000	16.9	32	0.1	15.0	0.00	232	1	100.0	10
26/09/21	8:40	9.5	9.6	9.4	39	-3.8	4.8	SW	0.80	17.7	SSW	9.2	8.7	8.4	---	769.5	0.00	0.0	133	1.91	135	0.061	0.000	16.9	32	0.1	14.9	0.00	233	1	100.0	10
26/09/21	8:50	9.6	9.6	9.6	38	-4.1	1.6	SW	0.27	17.7	SW	9.6	8.7	8.7	---	769.7	0.00	0.0	142	2.04	146	0.061	0.000	16.9	32	0.1	14.9	0.00	234	1	100.0	10
26/09/21	9:00	9.6	9.7	9.6	38	-4.0	1.6	WSW	0.27	17.7	SW	9.6	8.7	8.7	---	769.7	0.00	0.0	179	2.57	246	0.061	0.000	16.8	32	-0.0	14.8	0.08	233	1	100.0	10
26/09/21	9:10	9.9	10.1	9.7	37	-4.1	1.6	SSW	0.27	16.1	SSW	9.9	9.0	9.0	---	769.8	0.00	0.0	239	3.43	320	0.059	0.000	16.8	32	-0.0	14.8	0.00	235	1	100.0	10
26/09/21	9:20	10.1	10.4	10.0	38	-3.6	0.0	S	0.00	6.4	S	10.1	9.2	9.2	---	769.6	0.00	0.0	327	4.69	394	0.057	0.000	16.7	32	-0.1	14.8	0.00	233	1	100.0	10
26/09/21	9:30	10.7	10.9	10.4	35	-4.1	3.2	S	0.54	20.9	SW	10.7	9.6	9.6	---	769.8	0.00	0.0	433	6.21	478	0.053	0.000	16.7	32	-0.1	14.8	0.00	234	1	100.0	10
26/09/21	9:40	11.3	11.6	10.9	33	-4.4	1.6	S	0.27	8.0	S	11.3	10.0	10.0	---	769.7	0.00	0.0	504	7.22	610	0.049	0.000	16.7	32	-0.1	14.8	0.00	233	1	100.0	10
26/09/21	9:50	11.8	12.0	11.6	31	-4.8	3.2	S	0.54	19.3	SW	11.8	10.4	10.4	---	769.7	0.00	0.0	543	7.78	622	0.045	0.000	16.6	32	-0.2	14.7	0.00	231	1	100.0	10
26/09/21	10:00	12.1	12.3	11.9	30	-5.0	3.2	S	0.54	19.3	S	12.1	10.6	10.6	---	769.7	0.00	0.0	558	8.00	608	0.044	0.000	16.6	32	-0.2	14.7	0.23	234	1	100.0	10
26/09/21	10:10	12.1	12.2	12.1	31	-4.5	3.2	S	0.54	17.7	S	12.1	10.6	10.6	---	769.7	0.00	0.0	575	8.24	606	0.043	0.000	16.6	32	-0.2	14.7	0.00	231	1	100.0	10
26/09/21	10:20	12.2	12.3	12.2	28	-5.8	3.2	SW	0.54	25.7	WSW	12.2	10.6	10.6	---	769.7	0.00	0.0	578	8.29	587	0.042	0.000	16.6	32	-0.2	14.7	0.00	234	1	100.0	10
26/09/21	10:30	12.4	12.7	12.3	28	-5.6	1.6	SSW	0.27	19.3	SW	12.4	10.8	10.8	---	769.7	0.00	0.0	599	8.59	622	0.041	0.000	16.6	32	-0.2	14.7	0.00	234	1	100.0	10
26/09/21	10:40	12.9	13.1	12.7	28	-5.2	1.6	S	0.27	8.0	S	12.9	11.2	11.2	---	769.7	0.00	0.0	675	9.68	698	0.038	0.000	16.7	32	-0.1	14.8	0.00	233	1	100.0	10
26/09/21	10:50	13.1	13.2	13.0	28	-5.0	1.6	SE	0.27	17.7	SE	13.1	11.3	11.3	---	769.7	0.00	0.0	696	9.98	715	0.036	0.000	16.7	32	-0.1	14.8	0.00	234	1	100.0	10
26/09/21	11:00	13.3	13.4	13.1	24	-6.9	3.2	S	0.54	16.1	SW	13.3	11.3	11.3	---	769.5	0.00	0.0	699	10.02	731	0.035	0.000	16.7	32	-0.1	14.8	0.33	234	1	100.0	10
26/09/21	11:10	13.3	13.3	13.2	26	-5.9	1.6	S	0.27	9.7	S	13.3	11.3	11.3	---	769.8	0.00	0.0	626	8.97	754	0.035	0.000	16.7	32	-0.1	14.8	0.00	234	1	100.0	10
26/09/21	11:20	13.3	13.5	13.2	26	-5.9	1.6	SSW	0.27	6.4	SW	13.3	11.3	11.3	---	769.7	0.00	0.0	715	10.25	744	0.035	0.000	16.8	32	-0.0	14.8	0.00	235	1	100.0	10
26/09/21	11:30	13.7	13.8	13.5	24	-6.6	1.6	SSE	0.27	11.3	SW	13.7	11.6	11.6	---	769.6	0.00	0.0	717	10.28	724	0.032	0.000	16.8	32	-0.0	14.8	0.00	233	1	100.0	10
26/09/21	11:40	13.4	13.7	13.3	25	-6.2	1.6	SSE	0.27	24.1	SSW	13.4	11.4	11.4	---	769.6	0.00	0.0	689	9.88	738	0.034	0.000	16.9	32	0.1	14.9	0.00	234	1	100.0	10
26/09/21	11:50	13.4	13.7	13.2	23	-7.3	1.6	S	0.27	20.9	SW	13.4	11.4	11.4	---	769.6	0.00	0.0	740	10.61	747	0.034	0.000	16.9	32	0.1	15.0	0.00	232	1	100.0	10
26/09/21	12:00	13.6	13.7	13.6	22	-7.7	3.2	S	0.54	17.7	WSW	13.6	11.5	11.5	---	769.5	0.00	0.0	754	10.81	763	0.033	0.000	17.0	32	0.2	15.1	0.38	234	1	100.0	10
26/09/21	12:10	13.9	14.1	13.7	22	-7.5	1.6	SW	0.27	12.9	ESE	13.9	11.8	11.8	---	769.6	0.00	0.0	767	11.00	780	0.030	0.000	17.1	32	0.2	15.1	0.00	234	1	100.0	10
26/09/21	12:20	14.1	14.1	13.9	24	-6.2	1.6	WSW	0.27	14.5	WSW	14.1	11.9	11.9	---	769.3	0.00	0.0	800	11.47	814	0.030	0.000	17.2	32	0.3	15.2	0.00	233	1	100.0	10
26/09/21	12:30	14.3	14.4	14.1	20	-8.4	1.6	SW	0.27	14.5	W	14.3	12.0	12.0	---	769.3	0.00	0.0	784	11.24	800	0.028	0.000	17.2	32	0.4	15.3	0.00	235	1	100.0	10
26/09/21	12:40	14.3	14.4	14.2	20	-8.4	3.2	WSW	0.54	17.7	WSW	14.3	12.0	12.0	---	769.3	0.00	0.0	801	11.48	823	0.028	0.000	17.3	32	0.4	15.3	0.00	233	1	100.0	10
26/09/21	12:50	14.6	14.7	14.4	22	-7.0	0.0	SSE	0.00	11.3	SW	14.6	12.3	12.3	---	769.2	0.00	0.0	782	11.21	802	0.026	0.000	17.4	32	0.5	15.4	0.00	234	1	100.0	10
26/09/21	13:00	14.8	15.1	14.6	18	-9.3	1.6	SSE	0.27	8.0	SE	14.8	12.3	12.3	---	769.2	0.00	0.0	800	11.47	809	0.024	0.000	17.4	32	0.5	15.4	0.43	234	1	100.0	10
26/09/21	13:10	14.8	15.1	14.8	18	-9.3	0.0	SSE	0.00	4.8	SE	14.8	12.3	12.3	---	769.1	0.00	0.0	798	11.44	823	0.024	0.000	17.5	32	0.6	15.6	0.00	234	1	100.0	10
26/09/21	13:20	14.9	15.1	14.9	19	-8.5	1.6	ESE	0.27	11.3	ESE	14.9	12.5	12.5	---	769.0	0.00	0.0	810	11.61	814	0.024	0.000	17.6	32	0.7	15.6	0.00	234	1	100.0	10
26/09/21	13:30	15.1	15.2	15.1	19	-8.4	1.6	S	0.27	9.7	S	15.1	12.6	12.6	---	768.7	0.00	0.0	815	11.68	821	0.022	0.000	17.7	32	0.8	15.7	0.00	235	1	100.0	10
26/09/21	13:40	15.4	15.6	15.2	21	-6.8	1.6	ESE	0.27	14.5	ESE	15.4	13.0	13.0	---	768.5	0.00	0.0	809	11.60	826	0.020	0.000	17.8	31	0.4	15.8	0.00	233	1	100.0	10
26/09/21	13:50	15.4	15.7	15.2	17	-9.5	1.6	W	0.27	16.1	W	15.4	12.8	12.8	---	768.4	0.00	0.0	774	11.10	805	0.020	0.000	17.8	31	0.5	15.8	0.00	233	1	100.0	10
26/09/21	14:00	15.5	15.7	15.4	16	-10.3	1.6	WSW	0.27	6.4	WSW	15.5	12.8	12.8	---	768.2	0.00	0.0	790	11.32	798	0.020	0.000	17.9	31	0.6	15.9	0.43	233	1	100.0	10
26/09/21	14:10	15.6	15.7	15.5	16	-10.2	0.0	WNW	0.00	6.4	WNW	15.6	12.9	12.9	---	768.1	0.00	0.0	772	11.07	786	0.019	0.000	18.1	30	0.2	16.0	0.00	234	1	100.0	10
26/09/21	14:20	15.6	15.7	15.6	17	-9.4	1.6	SSW	0.27	6.4	SSW	15.6	12.9	12.9	---	768.0	0.00	0.0	719	10.31	752	0.019	0.000	18.1	30	0.2	16.1	0.00	234	1	100.0	10
26/09/21	14:30	15.9	16.2	15.7	18	-8.4	1.6	W	0.27	24.1	W	15.9	13.3	13.3	---	767.9	0.00	0.0	743	10.65	779	0.017	0.000	18.2	30	0.3	16.2	0.00	232	1	100.0	10
26/09/21	14:40	16.1	16.2	16.0	16	-9.8	0.0	SSW	0.00	6.4	SSW	16.1	13.4	13.4	---	767.9	0.00	0.0	744	10.67	758	0.015	0.000	18.3	30	0.4	16.3	0.00	234	1	100.0	10
26/09/21	14:50	15.9	16.0	15.9	18	-8.4	1.6	SSE	0.27	6.4	ESE	15.9	13.3	13.3	---	767.7	0.00	0.0	733	10.51	740	0.017	0.000	18.4	29	0.1	16.3	0.00	234	1	100.0	10
26/09/21	15:00	16.1	16.2	15.9	16	-9.8	1.6	ESE	0.27	9.7	ESE	16.1	13.3	13.3	---	767.5	0.00	0.0	539	7.73	712	0.016	0.000	18.6	29	0.2	16.4	0.38	234	1	100.0	10
26/09/21	15:10	15.9	16.1	15.9	17	-9.1	0.0	E	0.00	4.8	SSW	15.9	13.3	13.3	---	767.4	0.00	0.0	80	1.15	144	0.017	0.000	18.7	29	0.3	16.6	0.00	233	1	100.0	10
26/09/21	15:20	16.0	16.2	15.9	16	-9.9	1.6	SE	0.27	19.3	SE	16.0	13.3	13.3	---	767.3	0.00	0.0	67	0.96	70	0.016	0.000	18.8	29	0.4	16.7	0.00	234	1	100.0	10
26/09/21	15:30	16.1	16.3	15.9	17	-9.0	0.0	SSE	0.00	6.4	WSW	16.1	13.4	13.4	---	767.2	0.00	0.0	70	1.00	70	0.015	0.000	18.8	29	0.4	16.7	0.00	234	1	100.0	10
26/09/21	15:40	16.3	16.4	16.1	15	-10.4	1.6	W	0.27	6.4	WSW	16.3	13.6	13.6	---	767.1	0.00	0.0	65	0.93	70	0.014	0.000	18.9	28	0.0	16.7	0.00	235	1	100.0	10
26/09/21	15:50	16.2	16.3	16.0	16	-9.7	1.6	W	0.27	11.3	WNW	16.2	13.4	13.4	---	767.1	0.00	0.0	60	0.86	63	0.015	0.000	19.0	28	0.1	16.8	0.00	233	1	100.0	10
26/09/21	16:00	16.4	16.6	16.3	16	-9.5	3.2	W	0.54	20.9	SW	16.4	13.6	13.6	---	767.1	0.00	0.0	57	0.82	58	0.014	0.000	19.1	28	0.2	16.9	0.08	234	1	100.0	10
26/09/21	16:10	16.6	16.7	16.6	14	-11.1	1.6	W	0.27	6.4	SW	16.6	13.7	13.7	---	766.9	0.00	0.0	60	0.86	69	0.012	0.000	19.2	28	0.3	17.0	0.00	234	1	100.0	10
26/09/21	16:20	16.8	16.9	16.7	16	-9.2	1.6	WNW	0.27	6.4	WNW	16.8	13.9	13.9	---	766.9	0.00	0.0	75	1.08	104	0.011	0.000	19.3	28	0.3	17.1	0.00	234	1	100.0	10
26/09/21	16:30	16.8	17.0	16.7	15	-10.0	1.6	W	0.27	11.3	W	16.8	13.9	13.9	---	766.9	0.00	0.0	218	3.13	336	0.010	0.000	19.4	28	0.4	17.2	0.00	234	1	100.0	10
26/09/21	16:40	17.0	17.1	16.9	14	-10.7	0.0	SSE	0.00	6.4	S	17.0	14.1	14.1	---	767.0	0.00	0.0	454	6.51	503	0.009	0.000	19.4	28	0.4	17.2	0.00	232	1	100.0	10
26/09/21	16:50	16.8	16.9	16.8	14	-10.9	0.0	S	0.00	4.8	SSW	16.8	13.9	13.9	---	766.9	0.00	0.0	489	7.01	508	0.010	0.000	19.5	28	0.5	17.3	0.00	234	1	100.0	10
26/09/21	17:00	16.9	16.9	16.8	13	-11.8	0.0	---	0.00	0.0	---	16.9	13.9	13.9	---	766.9	0.00	0.0	446	6.39	462	0.010	0.000	19.6	28	0.6	17.4	0.18	234	1	100.0	10
26/09/21	17:10	16.9	17.1	16.8	15	-9.9	1.6	SW	0.27	9.7	WSW	16.9	14.1	14.1	---	766.7	0.00	0.0	423	6.06	434	0.010	0.000	19.7	28	0.6	17.4	0.00	233	1	100.0	10
26/09/21	17:20	16.9	16.9	16.8	14	-10.8	1.6	W	0.27	9.7	WSW	16.9	13.9	13.9	---	766.8	0.00	0.0	346	4.96	408	0.010	0.000	19.7	28	0.7	17.6	0.00	234	1	100.0	10
26/09/21	17:30	16.7	16.8	16.6	13	-11.9	0.0	W	0.00	6.4	W	16.7	13.7	13.7	---	766.8	0.00	0.0	325	4.66	387	0.012	0.000	19.8	28	0.7	17.6	0.00	233	1	100.0	10
26/09/21	17:40	16.9	17.1	16.8	14	-10.8	0.0	SW	0.00	4.8	SW	16.9	13.9	13.9	---	766.9	0.00	0.0	325	4.66	345	0.010	0.000	19.8	28	0.8	17.7	0.00	234	1	100.0	10
26/09/21	17:50	16.9	17.0	16.8	14	-10.8	0.0	WSW	0.00	4.8	WSW	16.9	13.9	13.9	---	766.9	0.00	0.0	284	4.07	304	0.010	0.000	19.9	28	0.8	17.7	0.00	234	1	100.0	10
26/09/21	18:00	16.7	16.8	16.5	14	-11.0	0.0	WSW	0.00	6.4	WSW	16.7	13.8	13.8	---	767.0	0.00	0.0	248	3.56	265	0.012	0.000	19.9	29	1.4	17.9	0.20	233	1	100.0	10
26/09/21	18:10	16.5	16.6	16.4	15	-10.3	0.0	SSW	0.00	6.4	WSW	16.5	13.7	13.7	---	767.1	0.00	0.0	208	2.98	227	0.013	0.000	20.0	29	1.4	17.9	0.00	234	1	100.0	10
26/09/21	18:20	16.4	16.6	16.3	14	-11.2	0.0	SW	0.00	6.4	SW	16.4	13.6	13.6	---	767.1	0.00	0.0	167	2.39	185	0.014	0.000	20.0	29	1.4	17.9	0.00	234	1	100.0	10
26/09/21	18:30	16.4	16.5	16.3	15	-10.4	0.0	---	0.00	0.0	---	16.4	13.6	13.6	---	767.1	0.00	0.0	127	1.82	146	0.014	0.000	20.0	29	1.4	17.9	0.00	233	1	100.0	10
26/09/21	18:40	16.2	16.4	16.1	14	-11.4	0.0	---	0.00	0.0	---	16.2	13.4	13.4	---	767.2	0.00	0.0	84	1.20	107	0.015	0.000	20.0	29	1.4	17.9	0.00	234	1	100.0	10
26/09/21	18:50	15.9	16.1	15.7	18	-8.4	0.0	---	0.00	0.0	---	15.9	13.2	13.2	---	767.2	0.00	0.0	52	0.75	56	0.017	0.000	20.1	29	1.5	18.0	0.00	234	1	100.0	10
26/09/21	19:00	15.4	15.7	15.1	17	-9.6	0.0	---	0.00	0.0	---	15.4	12.8	12.8	---	767.2	0.00	0.0	24	0.34	39	0.020	0.000	20.1	29	1.5	18.1	0.08	235	1	100.0	10
26/09/21	19:10	14.7	15.1	14.2	20	-8.1	0.0	---	0.00	0.0	---	14.7	12.3	12.3	---	767.3	0.00	0.0	12	0.17	14	0.025	0.000	20.1	29	1.5	18.1	0.00	232	1	100.0	10
26/09/21	19:20	13.7	14.2	13.1	24	-6.5	0.0	---	0.00	0.0	---	13.7	11.6	11.6	---	767.3	0.00	0.0	8	0.11	9	0.032	0.000	20.0	28	0.9	17.9	0.00	234	1	100.0	10
26/09/21	19:30	12.5	13.0	12.0	27	-6.0	0.0	---	0.00	0.0	---	12.5	10.8	10.8	---	767.3	0.00	0.0	2	0.03	5	0.041	0.000	20.0	28	0.9	17.9	0.00	233	1	100.0	10
26/09/21	19:40	11.4	12.0	10.8	30	-5.6	0.0	---	0.00	0.0	---	11.4	10.0	10.0	---	767.4	0.00	0.0	0	0.00	0	0.048	0.000	19.9	27	0.4	17.7	0.00	234	1	100.0	10
26/09/21	19:50	10.3	10.7	9.9	35	-4.5	0.0	---	0.00	0.0	---	10.3	9.3	9.3	---	767.4	0.00	0.0	0	0.00	0	0.056	0.000	19.8	27	0.3	17.6	0.00	232	1	100.0	10
26/09/21	20:00	9.6	9.9	9.3	36	-4.8	0.0	---	0.00	0.0	---	9.6	8.6	8.6	---	767.5	0.00	0.0	0	0.00	0	0.061	0.000	19.7	27	0.2	17.4	0.03	235	1	100.0	10
26/09/21	20:10	8.9	9.3	8.6	37	-5.0	0.0	---	0.00	0.0	---	8.9	8.1	8.1	---	767.5	0.00	0.0	0	0.00	0	0.065	0.000	19.7	27	0.1	17.4	0.00	234	1	100.0	10
26/09/21	20:20	8.3	8.6	8.1	41	-4.2	0.0	---	0.00	0.0	---	8.3	7.4	7.4	---	767.6	0.00	0.0	0	0.00	0	0.070	0.000	19.6	27	0.0	17.2	0.00	233	1	100.0	10
26/09/21	20:30	7.8	8.1	7.7	50	-2.0	0.0	---	0.00	0.0	---	7.8	7.2	7.2	---	767.6	0.00	0.0	0	0.00	0	0.073	0.000	19.4	27	-0.1	17.1	0.00	234	1	100.0	10
26/09/21	20:40	7.7	7.7	7.6	51	-1.8	0.0	---	0.00	0.0	---	7.7	7.1	7.1	---	767.7	0.00	0.0	0	0.00	0	0.074	0.000	19.3	27	-0.2	16.9	0.00	232	1	100.0	10
26/09/21	20:50	7.6	7.7	7.3	51	-1.9	0.0	---	0.00	0.0	---	7.6	6.9	6.9	---	767.7	0.00	0.0	0	0.00	0	0.075	0.000	19.2	27	-0.3	16.8	0.00	232	1	100.0	10
26/09/21	21:00	7.1	7.3	6.9	54	-1.6	0.0	---	0.00	0.0	---	7.1	6.5	6.5	---	767.8	0.00	0.0	0	0.00	0	0.078	0.000	19.1	27	-0.3	16.7	0.03	233	1	100.0	10
26/09/21	21:10	6.8	6.9	6.6	52	-2.4	0.0	---	0.00	0.0	---	6.8	6.2	6.2	---	767.9	0.00	0.0	0	0.00	0	0.080	0.000	19.0	27	-0.4	16.6	0.00	234	1	100.0	10
26/09/21	21:20	6.4	6.6	6.3	55	-2.0	0.0	---	0.00	0.0	---	6.4	5.9	5.9	---	767.9	0.00	0.0	0	0.00	0	0.083	0.000	18.9	27	-0.5	16.5	0.00	233	1	100.0	10
26/09/21	21:30	6.1	6.2	5.9	51	-3.3	0.0	---	0.00	0.0	---	6.1	5.4	5.4	---	767.9	0.00	0.0	0	0.00	0	0.085	0.000	18.8	28	-0.1	16.6	0.00	234	1	100.0	10
26/09/21	21:40	6.0	6.1	5.9	54	-2.6	0.0	---	0.00	0.0	---	6.0	5.4	5.4	---	768.1	0.00	0.0	0	0.00	0	0.086	0.000	18.7	28	-0.2	16.5	0.00	233	1	100.0	10
26/09/21	21:50	6.3	6.6	6.1	60	-0.9	0.0	---	0.00	0.0	---	6.3	5.9	5.9	---	768.3	0.00	0.0	0	0.00	0	0.083	0.000	18.6	28	-0.3	16.3	0.00	233	1	100.0	10
26/09/21	22:00	6.6	6.7	6.3	56	-1.6	0.0	---	0.00	0.0	---	6.6	6.0	6.0	---	768.2	0.00	0.0	0	0.00	0	0.082	0.000	18.5	28	-0.4	16.2	0.03	234	1	100.0	10
26/09/21	22:10	6.2	6.3	6.2	58	-1.4	0.0	---	0.00	0.0	---	6.2	5.7	5.7	---	768.3	0.00	0.0	0	0.00	0	0.084	0.000	18.4	28	-0.5	16.1	0.00	233	1	100.0	10
26/09/21	22:20	6.1	6.2	6.0	54	-2.5	0.0	---	0.00	0.0	---	6.1	5.6	5.6	---	768.3	0.00	0.0	0	0.00	0	0.085	0.000	18.3	28	-0.6	16.0	0.00	233	1	100.0	10
26/09/21	22:30	5.9	6.0	5.8	58	-1.7	0.0	---	0.00	0.0	---	5.9	5.4	5.4	---	768.4	0.00	0.0	0	0.00	0	0.086	0.000	18.2	29	-0.1	16.1	0.00	233	1	100.0	10
26/09/21	22:40	5.7	5.8	5.5	59	-1.7	0.0	---	0.00	0.0	---	5.7	5.2	5.2	---	768.4	0.00	0.0	0	0.00	0	0.088	0.000	18.1	29	-0.2	15.9	0.00	230	1	100.0	10
26/09/21	22:50	5.4	5.5	5.2	60	-1.8	0.0	---	0.00	0.0	---	5.4	4.9	4.9	---	768.4	0.00	0.0	0	0.00	0	0.090	0.000	18.1	29	-0.3	15.9	0.00	233	1	100.0	10
26/09/21	23:00	4.8	5.2	4.6	60	-2.3	0.0	---	0.00	0.0	---	4.8	4.4	4.4	---	768.5	0.00	0.0	0	0.00	0	0.094	0.000	17.9	29	-0.4	15.8	0.03	233	1	100.0	10""")

pred = model.predict(x3)

print(pred)

[[0.5900776]]


## Helada 5/10/21 2:40 AM
### Predicción a las 21:00

In [7]:
x4 = convert("""05/10/21	6:10	5.1	5.3	4.9	78	1.5	0.0	---	0.00	0.0	---	5.1	4.9	4.9	---	761.3	0.00	0.0	0	0.00	0	0.092	0.000	14.8	37	0.3	13.2	0.00	233	1	100.0	10
05/10/21	6:20	4.8	4.9	4.8	79	1.5	0.0	---	0.00	0.0	---	4.8	4.7	4.7	---	761.4	0.00	0.0	0	0.00	0	0.094	0.000	14.7	37	0.2	13.1	0.00	234	1	100.0	10
05/10/21	6:30	4.9	5.2	4.8	83	2.3	0.0	---	0.00	0.0	---	4.9	4.8	4.8	---	761.5	0.00	0.0	0	0.00	0	0.093	0.000	14.7	37	0.1	13.1	0.00	234	1	100.0	10
05/10/21	6:40	5.2	5.3	5.1	81	2.2	0.0	---	0.00	0.0	---	5.2	5.1	5.1	---	761.5	0.00	0.0	0	0.00	0	0.091	0.000	14.6	37	0.1	13.0	0.00	232	1	100.0	10
05/10/21	6:50	5.0	5.1	4.9	82	2.2	0.0	---	0.00	0.0	---	5.0	4.8	4.8	---	761.4	0.00	0.0	0	0.00	0	0.093	0.000	14.6	37	0.0	12.9	0.00	234	1	100.0	10
05/10/21	7:00	4.9	4.9	4.9	82	2.1	0.0	---	0.00	0.0	---	4.9	4.8	4.8	---	761.6	0.00	0.0	1	0.01	5	0.093	0.000	14.5	37	-0.0	12.9	0.00	233	1	100.0	10
05/10/21	7:10	5.1	5.3	4.9	81	2.1	0.0	---	0.00	0.0	---	5.1	4.9	4.9	---	761.7	0.00	0.0	9	0.13	11	0.092	0.000	14.4	37	-0.1	12.9	0.00	234	1	100.0	10
05/10/21	7:20	5.4	5.6	5.3	78	1.9	0.0	---	0.00	0.0	---	5.4	5.3	5.3	---	761.8	0.00	0.0	18	0.26	25	0.090	0.000	14.4	38	0.2	12.9	0.00	234	1	100.0	10
05/10/21	7:30	5.7	5.9	5.6	77	2.0	0.0	---	0.00	0.0	---	5.7	5.5	5.5	---	761.9	0.00	0.0	33	0.47	40	0.088	0.000	14.3	38	0.1	12.8	0.00	232	1	100.0	10
05/10/21	7:40	6.1	6.2	5.9	75	2.0	0.0	---	0.00	0.0	---	6.1	5.8	5.8	---	761.9	0.00	0.0	54	0.77	58	0.085	0.000	14.3	38	0.1	12.8	0.00	234	1	100.0	10
05/10/21	7:50	6.4	6.8	6.2	68	1.0	0.0	---	0.00	0.0	---	6.4	6.1	6.1	---	762.0	0.00	0.0	57	0.82	70	0.083	0.000	14.2	38	0.1	12.7	0.00	233	1	100.0	10
05/10/21	8:00	7.1	7.4	6.8	63	0.5	0.0	---	0.00	0.0	---	7.1	6.7	6.7	---	762.0	0.00	0.0	70	1.00	83	0.078	0.000	14.1	38	-0.0	12.6	0.03	234	1	100.0	10
05/10/21	8:10	7.7	7.9	7.4	61	0.7	0.0	---	0.00	0.0	---	7.7	7.3	7.3	---	762.1	0.00	0.0	117	1.68	200	0.074	0.000	14.1	38	-0.0	12.6	0.00	233	1	100.0	10
05/10/21	8:20	8.3	8.6	8.0	57	0.2	0.0	---	0.00	0.0	---	8.3	7.8	7.8	---	762.3	0.00	0.0	122	1.75	169	0.070	0.000	14.1	38	-0.1	12.6	0.00	234	1	100.0	10
05/10/21	8:30	8.8	8.9	8.6	57	0.7	0.0	---	0.00	0.0	---	8.8	8.3	8.3	---	762.3	0.00	0.0	94	1.35	137	0.066	0.000	14.1	38	-0.1	12.6	0.00	234	1	100.0	10
05/10/21	8:40	9.1	9.2	8.9	57	1.0	0.0	---	0.00	0.0	---	9.1	8.6	8.6	---	762.3	0.00	0.0	100	1.43	153	0.064	0.000	14.0	38	-0.1	12.5	0.00	234	1	100.0	10
05/10/21	8:50	9.3	9.4	9.2	60	1.9	0.0	---	0.00	0.0	---	9.3	8.8	8.8	---	762.2	0.00	0.0	117	1.68	186	0.063	0.000	13.9	38	-0.2	12.4	0.00	234	1	100.0	10
05/10/21	9:00	9.7	9.8	9.4	55	1.1	0.0	---	0.00	0.0	---	9.7	9.1	9.1	---	762.3	0.00	0.0	205	2.94	267	0.060	0.000	13.9	38	-0.2	12.4	0.05	234	1	100.0	10
05/10/21	9:10	10.1	10.4	9.8	56	1.7	0.0	---	0.00	0.0	---	10.1	9.5	9.5	---	762.3	0.00	0.0	409	5.86	461	0.057	0.000	13.9	38	-0.2	12.4	0.00	233	1	100.0	10
05/10/21	9:20	10.9	11.3	10.4	52	1.4	0.0	---	0.00	0.0	---	10.9	10.2	10.2	---	762.2	0.00	0.0	408	5.85	462	0.052	0.000	13.9	38	-0.2	12.4	0.00	231	1	100.0	10
05/10/21	9:30	11.9	12.4	11.4	50	1.8	0.0	---	0.00	0.0	---	11.9	11.0	11.0	---	762.3	0.00	0.0	439	6.29	464	0.045	0.000	13.9	38	-0.2	12.4	0.00	233	1	100.0	10
05/10/21	9:40	12.9	13.6	12.4	48	2.2	0.0	---	0.00	0.0	---	12.9	11.9	11.9	---	762.2	0.00	0.0	470	6.74	496	0.037	0.000	13.9	38	-0.2	12.4	0.00	234	1	100.0	10
05/10/21	9:50	14.4	15.4	13.6	41	1.3	1.6	SW	0.27	6.4	SW	14.4	13.0	13.0	---	762.2	0.00	0.0	524	7.51	566	0.027	0.000	13.9	38	-0.2	12.4	0.00	233	1	100.0	10
05/10/21	10:00	15.9	16.3	15.4	37	1.3	1.6	SSW	0.27	9.7	SW	15.9	14.3	14.3	---	762.3	0.00	0.0	478	6.85	533	0.017	0.000	13.9	38	-0.2	12.4	0.23	234	1	100.0	10
05/10/21	10:10	16.4	16.5	16.3	38	2.0	1.6	SSE	0.27	11.3	SSE	16.4	14.8	14.8	---	762.4	0.00	0.0	419	6.01	552	0.014	0.000	14.0	38	-0.1	12.5	0.00	234	1	100.0	10
05/10/21	10:20	16.8	16.9	16.5	36	1.7	3.2	SSE	0.54	11.3	SSW	16.8	15.1	15.1	---	762.4	0.00	0.0	507	7.27	592	0.010	0.000	14.1	38	-0.0	12.6	0.00	234	1	100.0	10
05/10/21	10:30	17.1	17.2	16.9	35	1.5	3.2	S	0.54	30.6	WSW	17.1	15.3	15.3	---	762.4	0.00	0.0	571	8.19	636	0.009	0.000	14.2	38	0.0	12.7	0.00	235	1	100.0	10
05/10/21	10:40	17.1	17.1	17.0	37	2.2	1.6	S	0.27	17.7	SSW	17.1	15.4	15.4	---	762.4	0.00	0.0	540	7.74	677	0.009	0.000	14.2	38	0.1	12.7	0.00	234	1	100.0	10
05/10/21	10:50	17.2	17.3	17.1	36	2.0	1.6	SSW	0.27	17.7	SE	17.2	15.5	15.5	---	762.4	0.00	0.0	635	9.10	714	0.008	0.000	14.3	37	-0.2	12.8	0.00	234	1	100.0	10
05/10/21	11:00	17.5	17.7	17.3	36	2.2	3.2	S	0.54	17.7	S	17.5	15.8	15.8	---	762.5	0.00	0.0	686	9.83	733	0.006	0.000	14.5	37	-0.0	12.9	0.33	233	1	100.0	10
05/10/21	11:10	17.7	17.8	17.6	34	1.6	1.6	S	0.27	8.0	S	17.7	15.9	15.9	---	762.4	0.00	0.0	605	8.67	694	0.004	0.000	14.6	37	0.1	13.0	0.00	232	1	100.0	10
05/10/21	11:20	17.8	17.9	17.8	36	2.5	3.2	WSW	0.54	16.1	SW	17.8	16.1	16.1	---	762.4	0.00	0.0	584	8.37	677	0.003	0.000	14.7	37	0.2	13.1	0.00	234	1	100.0	10
05/10/21	11:30	18.0	18.1	17.9	33	1.5	1.6	SSW	0.27	17.7	SSW	18.0	16.1	16.1	---	762.4	0.00	0.0	658	9.43	735	0.002	0.000	14.8	37	0.3	13.2	0.00	235	1	100.0	10
05/10/21	11:40	18.1	18.2	17.9	34	1.9	1.6	S	0.27	9.7	SE	18.1	16.2	16.2	---	762.4	0.00	0.0	711	10.19	833	0.002	0.000	14.9	37	0.4	13.3	0.00	234	1	100.0	10
05/10/21	11:50	18.3	18.4	18.2	34	2.2	3.2	S	0.54	14.5	SW	18.3	16.5	16.5	---	762.3	0.00	0.0	774	11.10	814	0.000	0.000	15.1	37	0.5	13.5	0.00	233	1	100.0	10
05/10/21	12:00	18.5	18.6	18.3	34	2.3	1.6	S	0.27	9.7	S	18.5	16.7	16.7	---	762.3	0.00	0.0	749	10.74	821	0.000	0.001	15.2	37	0.6	13.6	0.41	234	1	100.0	10
05/10/21	12:10	18.7	18.8	18.6	35	2.9	3.2	S	0.54	14.5	SSW	18.7	17.0	17.0	---	762.3	0.00	0.0	675	9.68	823	0.000	0.003	15.3	36	0.3	13.7	0.00	233	1	100.0	10
05/10/21	12:20	18.7	18.8	18.6	34	2.5	1.6	SSE	0.27	9.7	SE	18.7	16.9	16.9	---	762.2	0.00	0.0	765	10.97	854	0.000	0.002	15.5	36	0.5	13.8	0.00	234	1	100.0	10
05/10/21	12:30	18.8	18.8	18.7	32	1.8	1.6	SSE	0.27	11.3	SSE	18.8	16.9	16.9	---	762.2	0.00	0.0	908	13.02	1018	0.000	0.003	15.7	36	0.6	14.0	0.00	233	1	100.0	10
05/10/21	12:40	18.6	18.7	18.5	34	2.4	1.6	SSE	0.27	11.3	SE	18.6	16.8	16.8	---	762.1	0.00	0.0	673	9.65	722	0.000	0.002	15.8	36	0.7	14.1	0.00	234	1	100.0	10
05/10/21	12:50	18.5	18.6	18.3	33	1.9	3.2	SSE	0.54	11.3	SW	18.5	16.6	16.6	---	762.1	0.00	0.0	725	10.39	777	0.000	0.001	15.9	36	0.9	14.2	0.00	233	1	100.0	10
05/10/21	13:00	18.8	18.9	18.6	32	1.7	1.6	S	0.27	9.7	SSE	18.8	16.9	16.9	---	762.1	0.00	0.0	829	11.88	856	0.000	0.003	16.1	36	1.0	14.3	0.43	232	1	100.0	10
05/10/21	13:10	18.9	19.2	18.7	31	1.4	3.2	SW	0.54	17.7	SSE	18.9	17.0	17.0	---	762.1	0.00	0.0	814	11.67	868	0.000	0.004	16.2	36	1.1	14.5	0.00	234	1	100.0	10
05/10/21	13:20	18.9	19.1	18.8	30	0.9	3.2	WSW	0.54	20.9	WSW	18.9	16.9	16.9	---	762.0	0.00	0.0	611	8.76	895	0.000	0.004	16.3	35	0.8	14.6	0.00	233	1	100.0	10
05/10/21	13:30	18.9	19.1	18.8	31	1.4	1.6	ESE	0.27	8.0	ESE	18.9	17.0	17.0	---	762.0	0.00	0.0	814	11.67	875	0.000	0.004	16.4	35	0.9	14.7	0.00	234	1	100.0	10
05/10/21	13:40	19.3	19.6	19.1	32	2.2	1.6	SSE	0.27	16.1	SE	19.3	17.4	17.4	---	762.0	0.00	0.0	831	11.91	844	0.000	0.007	16.6	35	1.1	14.8	0.00	233	1	100.0	10
05/10/21	13:50	19.5	19.7	19.3	29	1.0	1.6	SSE	0.27	11.3	S	19.5	17.4	17.4	---	761.9	0.00	0.0	823	11.80	826	0.000	0.008	16.7	35	1.2	14.9	0.00	234	1	100.0	10
05/10/21	14:00	19.3	19.6	19.2	30	1.3	1.6	S	0.27	17.7	ESE	19.3	17.3	17.3	---	761.7	0.00	0.0	822	11.78	831	0.000	0.007	16.8	35	1.3	15.1	0.46	234	1	100.0	10
05/10/21	14:10	19.5	19.6	19.4	30	1.4	1.6	E	0.27	9.7	SW	19.5	17.5	17.5	---	761.6	0.00	0.0	843	12.08	854	0.000	0.008	17.0	34	1.0	15.2	0.00	234	1	100.0	10
05/10/21	14:20	19.7	19.9	19.5	28	0.7	1.6	SW	0.27	14.5	ESE	19.7	17.6	17.6	---	761.6	0.00	0.0	849	12.17	865	0.000	0.010	17.1	34	1.1	15.3	0.00	234	1	100.0	10
05/10/21	14:30	19.8	20.1	19.6	29	1.2	3.2	S	0.54	29.0	S	19.8	17.7	17.7	---	761.5	0.00	0.0	816	11.70	977	0.000	0.010	17.3	34	1.3	15.4	0.00	233	1	100.0	10
05/10/21	14:40	20.1	20.2	19.9	30	1.9	1.6	ESE	0.27	9.7	E	20.1	18.1	18.1	---	761.5	0.00	0.0	555	7.96	991	0.000	0.012	17.4	34	1.4	15.6	0.00	234	1	100.0	10
05/10/21	14:50	19.6	20.0	19.1	31	2.0	3.2	ESE	0.54	14.5	ESE	19.6	17.6	17.6	---	761.5	0.00	0.0	277	3.97	587	0.000	0.008	17.6	34	1.5	15.7	0.00	234	1	100.0	10
05/10/21	15:00	19.1	19.3	19.0	30	1.1	0.0	ESE	0.00	3.2	ESE	19.1	17.1	17.1	---	761.4	0.00	0.0	278	3.99	336	0.000	0.005	17.7	34	1.6	15.9	0.36	234	1	100.0	10
05/10/21	15:10	19.4	19.5	19.3	30	1.4	1.6	S	0.27	12.9	S	19.4	17.4	17.4	---	761.4	0.00	0.0	300	4.30	318	0.000	0.008	17.8	33	1.3	15.9	0.00	232	1	100.0	10
05/10/21	15:20	19.5	19.6	19.3	29	1.0	1.6	S	0.27	19.3	SE	19.5	17.4	17.4	---	761.6	0.00	0.0	306	4.39	320	0.000	0.008	17.9	33	1.4	16.0	0.00	232	1	100.0	10
05/10/21	15:30	19.4	19.6	19.3	28	0.4	1.6	W	0.27	8.0	W	19.4	17.2	17.2	---	761.6	0.00	0.0	292	4.19	309	0.000	0.008	18.0	33	1.5	16.1	0.00	234	1	100.0	10
05/10/21	15:40	19.1	19.3	18.9	30	1.1	1.6	SSW	0.27	17.7	W	19.1	17.1	17.1	---	761.7	0.00	0.0	260	3.73	272	0.000	0.005	18.1	33	1.6	16.2	0.00	234	1	100.0	10
05/10/21	15:50	19.2	19.6	18.9	28	0.3	3.2	S	0.54	12.9	S	19.2	17.0	17.0	---	761.7	0.00	0.0	275	3.94	320	0.000	0.006	18.2	33	1.7	16.3	0.00	235	1	100.0	10
05/10/21	16:00	19.4	19.7	19.2	30	1.4	1.6	S	0.27	11.3	SE	19.4	17.4	17.4	---	761.8	0.00	0.0	220	3.15	244	0.000	0.007	18.3	33	1.7	16.4	0.18	234	1	100.0	10
05/10/21	16:10	20.2	20.3	19.7	28	1.1	3.2	SSW	0.54	11.3	SW	20.2	18.1	18.1	---	761.9	0.00	0.0	299	4.29	352	0.000	0.013	18.4	33	1.8	16.5	0.00	233	1	100.0	10
05/10/21	16:20	19.8	20.3	19.6	29	1.3	3.2	SSW	0.54	9.7	SSW	19.8	17.7	17.7	---	762.0	0.00	0.0	431	6.18	687	0.000	0.010	18.5	33	1.9	16.6	0.00	234	1	100.0	10
05/10/21	16:30	19.6	19.8	19.4	33	2.9	3.2	SE	0.54	16.1	ESE	19.6	17.8	17.8	---	762.3	0.00	0.0	551	7.90	721	0.000	0.009	18.6	33	2.0	16.7	0.00	234	1	100.0	10
05/10/21	16:40	19.5	19.8	18.7	32	2.4	4.8	SE	0.80	17.7	SE	19.5	17.6	17.6	---	762.4	0.00	0.0	332	4.76	728	0.000	0.008	18.6	33	2.0	16.8	0.00	234	1	100.0	10
05/10/21	16:50	17.7	18.6	16.9	34	1.6	6.4	SSE	1.07	20.9	SSE	17.7	15.8	15.8	---	762.8	0.00	0.0	189	2.71	230	0.005	0.000	18.7	33	2.1	16.9	0.00	232	1	100.0	10
05/10/21	17:00	16.7	16.9	16.4	34	0.7	3.2	SSE	0.54	22.5	SSE	16.7	14.8	14.8	---	762.9	0.00	0.0	181	2.59	230	0.012	0.000	18.8	33	2.2	16.9	0.23	233	1	100.0	10
05/10/21	17:10	16.2	16.4	15.8	36	1.1	6.4	SSE	1.07	19.3	SSE	16.2	14.4	14.4	---	762.8	0.00	0.0	171	2.45	211	0.015	0.000	18.8	33	2.2	17.0	0.00	234	1	100.0	10
05/10/21	17:20	15.6	15.8	15.4	39	1.7	3.2	SSE	0.54	24.1	SSE	15.6	14.1	14.1	---	763.4	0.00	0.0	86	1.23	109	0.019	0.000	18.8	33	2.2	17.0	0.00	234	1	100.0	10
05/10/21	17:30	14.8	15.3	14.2	43	2.4	4.8	SSE	0.80	22.5	SSE	14.8	13.5	13.5	---	763.8	0.00	0.0	76	1.09	84	0.024	0.000	18.8	33	2.2	17.0	0.00	234	1	100.0	10
05/10/21	17:40	14.0	14.2	13.8	47	2.9	1.6	SE	0.27	14.5	SE	14.0	12.8	12.8	---	764.0	0.00	0.0	100	1.43	111	0.030	0.000	18.8	33	2.2	17.0	0.00	234	1	100.0	10
05/10/21	17:50	13.7	13.8	13.6	48	2.9	1.6	S	0.27	19.3	SSW	13.7	12.6	12.6	---	764.2	0.00	0.0	75	1.08	111	0.032	0.000	18.8	33	2.2	17.0	0.00	233	1	100.0	10
05/10/21	18:00	13.4	13.6	13.1	51	3.4	1.6	S	0.27	16.1	SSW	13.4	12.4	12.4	---	764.7	0.00	0.0	39	0.56	44	0.034	0.000	18.8	33	2.2	17.0	0.08	233	1	100.0	10
05/10/21	18:10	12.7	13.1	11.9	57	4.4	3.2	SSE	0.54	19.3	SE	12.7	11.9	11.9	---	765.1	0.00	0.0	24	0.34	32	0.039	0.000	18.8	34	2.6	17.1	0.00	234	1	100.0	10
05/10/21	18:20	11.4	11.9	10.9	61	4.2	3.2	SSE	0.54	20.9	SSE	11.4	10.9	10.9	---	765.7	0.00	0.0	21	0.30	26	0.048	0.000	18.8	34	2.6	17.1	0.00	234	1	100.0	10
05/10/21	18:30	10.3	10.8	9.8	62	3.3	6.4	SE	1.07	30.6	SE	9.7	9.8	9.3	---	766.1	0.00	0.0	22	0.32	25	0.056	0.000	18.8	34	2.6	17.1	0.00	234	1	100.0	10
05/10/21	18:40	9.7	9.8	9.7	62	2.8	4.8	ESE	0.80	20.9	ESE	9.5	9.3	9.1	---	766.2	0.00	0.0	16	0.23	23	0.060	0.000	18.8	34	2.6	17.0	0.00	233	1	100.0	10
05/10/21	18:50	9.8	9.8	9.7	60	2.4	3.2	ESE	0.54	14.5	ESE	9.8	9.3	9.3	---	766.6	0.00	0.0	11	0.16	11	0.059	0.000	18.7	34	2.5	16.9	0.00	234	1	100.0	10
05/10/21	19:00	9.9	10.0	9.8	58	2.0	3.2	ESE	0.54	17.7	SSE	9.9	9.4	9.4	---	766.8	0.00	0.0	9	0.13	9	0.059	0.000	18.7	35	2.9	16.9	0.03	234	1	100.0	10
05/10/21	19:10	10.1	10.2	10.0	58	2.2	1.6	ESE	0.27	9.7	ESE	10.1	9.6	9.6	---	767.0	0.00	0.0	7	0.10	7	0.057	0.000	18.6	35	2.8	16.8	0.00	235	1	100.0	10
05/10/21	19:20	10.2	10.3	10.2	57	2.1	1.6	ESE	0.27	9.7	ESE	10.2	9.7	9.7	---	767.2	0.00	0.0	6	0.09	7	0.056	0.000	18.5	35	2.7	16.7	0.00	233	1	100.0	10
05/10/21	19:30	10.3	10.3	10.2	56	1.9	1.6	ESE	0.27	8.0	ESE	10.3	9.7	9.7	---	767.5	0.00	0.0	1	0.01	5	0.056	0.000	18.4	35	2.7	16.7	0.00	233	1	100.0	10
05/10/21	19:40	10.3	10.4	10.2	54	1.4	1.6	ESE	0.27	16.1	ESE	10.3	9.7	9.7	---	767.8	0.00	0.0	0	0.00	0	0.056	0.000	18.3	35	2.6	16.6	0.00	233	1	100.0	10
05/10/21	19:50	10.6	10.6	10.4	51	0.8	3.2	ESE	0.54	16.1	ESE	10.6	9.8	9.8	---	768.0	0.00	0.0	0	0.00	0	0.054	0.000	18.3	35	2.5	16.5	0.00	234	1	100.0	10
05/10/21	20:00	10.6	10.7	10.6	48	0.0	1.6	SSE	0.27	17.7	ESE	10.6	9.8	9.8	---	768.0	0.00	0.0	0	0.00	0	0.054	0.000	18.2	35	2.5	16.4	0.00	234	1	100.0	10
05/10/21	20:10	10.7	10.7	10.6	47	-0.2	1.6	SSE	0.27	17.7	S	10.7	9.8	9.8	---	768.2	0.00	0.0	0	0.00	0	0.053	0.000	18.2	34	2.1	16.4	0.03	233	1	100.0	10
05/10/21	20:20	10.7	10.7	10.6	45	-0.8	1.6	SSE	0.27	16.1	SSE	10.7	9.8	9.8	---	768.5	0.00	0.0	0	0.00	0	0.053	0.000	18.1	34	2.0	16.3	0.00	233	1	100.0	10
05/10/21	20:30	10.7	10.8	10.7	44	-1.1	1.6	SSE	0.27	8.0	S	10.7	9.8	9.8	---	768.9	0.00	0.0	0	0.00	0	0.053	0.000	18.1	34	1.9	16.2	0.00	233	1	100.0	10
05/10/21	20:40	10.8	10.8	10.7	44	-1.0	1.6	SSE	0.27	8.0	SSE	10.8	9.9	9.9	---	769.1	0.00	0.0	0	0.00	0	0.052	0.000	18.0	34	1.9	16.2	0.00	233	1	100.0	10
05/10/21	20:50	10.7	10.7	10.6	43	-1.4	0.0	ESE	0.00	8.0	S	10.7	9.8	9.8	---	769.3	0.00	0.0	0	0.00	0	0.053	0.000	17.9	34	1.8	16.1	0.00	233	1	100.0	10
05/10/21	21:00	10.6	10.6	10.6	43	-1.5	1.6	SSE	0.27	6.4	WSW	10.6	9.7	9.7	---	769.5	0.00	0.0	0	0.00	0	0.054	0.000	17.8	34	1.7	16.0	0.05	233	1	100.0	10""")

pred = model.predict(x4)

print(pred)

[[0.11009812]]


### Predicción a las 22:00

In [9]:
x5 = convert("""05/10/21	7:10	5.1	5.3	4.9	81	2.1	0.0	---	0.00	0.0	---	5.1	4.9	4.9	---	761.7	0.00	0.0	9	0.13	11	0.092	0.000	14.4	37	-0.1	12.9	0.00	234	1	100.0	10
05/10/21	7:20	5.4	5.6	5.3	78	1.9	0.0	---	0.00	0.0	---	5.4	5.3	5.3	---	761.8	0.00	0.0	18	0.26	25	0.090	0.000	14.4	38	0.2	12.9	0.00	234	1	100.0	10
05/10/21	7:30	5.7	5.9	5.6	77	2.0	0.0	---	0.00	0.0	---	5.7	5.5	5.5	---	761.9	0.00	0.0	33	0.47	40	0.088	0.000	14.3	38	0.1	12.8	0.00	232	1	100.0	10
05/10/21	7:40	6.1	6.2	5.9	75	2.0	0.0	---	0.00	0.0	---	6.1	5.8	5.8	---	761.9	0.00	0.0	54	0.77	58	0.085	0.000	14.3	38	0.1	12.8	0.00	234	1	100.0	10
05/10/21	7:50	6.4	6.8	6.2	68	1.0	0.0	---	0.00	0.0	---	6.4	6.1	6.1	---	762.0	0.00	0.0	57	0.82	70	0.083	0.000	14.2	38	0.1	12.7	0.00	233	1	100.0	10
05/10/21	8:00	7.1	7.4	6.8	63	0.5	0.0	---	0.00	0.0	---	7.1	6.7	6.7	---	762.0	0.00	0.0	70	1.00	83	0.078	0.000	14.1	38	-0.0	12.6	0.03	234	1	100.0	10
05/10/21	8:10	7.7	7.9	7.4	61	0.7	0.0	---	0.00	0.0	---	7.7	7.3	7.3	---	762.1	0.00	0.0	117	1.68	200	0.074	0.000	14.1	38	-0.0	12.6	0.00	233	1	100.0	10
05/10/21	8:20	8.3	8.6	8.0	57	0.2	0.0	---	0.00	0.0	---	8.3	7.8	7.8	---	762.3	0.00	0.0	122	1.75	169	0.070	0.000	14.1	38	-0.1	12.6	0.00	234	1	100.0	10
05/10/21	8:30	8.8	8.9	8.6	57	0.7	0.0	---	0.00	0.0	---	8.8	8.3	8.3	---	762.3	0.00	0.0	94	1.35	137	0.066	0.000	14.1	38	-0.1	12.6	0.00	234	1	100.0	10
05/10/21	8:40	9.1	9.2	8.9	57	1.0	0.0	---	0.00	0.0	---	9.1	8.6	8.6	---	762.3	0.00	0.0	100	1.43	153	0.064	0.000	14.0	38	-0.1	12.5	0.00	234	1	100.0	10
05/10/21	8:50	9.3	9.4	9.2	60	1.9	0.0	---	0.00	0.0	---	9.3	8.8	8.8	---	762.2	0.00	0.0	117	1.68	186	0.063	0.000	13.9	38	-0.2	12.4	0.00	234	1	100.0	10
05/10/21	9:00	9.7	9.8	9.4	55	1.1	0.0	---	0.00	0.0	---	9.7	9.1	9.1	---	762.3	0.00	0.0	205	2.94	267	0.060	0.000	13.9	38	-0.2	12.4	0.05	234	1	100.0	10
05/10/21	9:10	10.1	10.4	9.8	56	1.7	0.0	---	0.00	0.0	---	10.1	9.5	9.5	---	762.3	0.00	0.0	409	5.86	461	0.057	0.000	13.9	38	-0.2	12.4	0.00	233	1	100.0	10
05/10/21	9:20	10.9	11.3	10.4	52	1.4	0.0	---	0.00	0.0	---	10.9	10.2	10.2	---	762.2	0.00	0.0	408	5.85	462	0.052	0.000	13.9	38	-0.2	12.4	0.00	231	1	100.0	10
05/10/21	9:30	11.9	12.4	11.4	50	1.8	0.0	---	0.00	0.0	---	11.9	11.0	11.0	---	762.3	0.00	0.0	439	6.29	464	0.045	0.000	13.9	38	-0.2	12.4	0.00	233	1	100.0	10
05/10/21	9:40	12.9	13.6	12.4	48	2.2	0.0	---	0.00	0.0	---	12.9	11.9	11.9	---	762.2	0.00	0.0	470	6.74	496	0.037	0.000	13.9	38	-0.2	12.4	0.00	234	1	100.0	10
05/10/21	9:50	14.4	15.4	13.6	41	1.3	1.6	SW	0.27	6.4	SW	14.4	13.0	13.0	---	762.2	0.00	0.0	524	7.51	566	0.027	0.000	13.9	38	-0.2	12.4	0.00	233	1	100.0	10
05/10/21	10:00	15.9	16.3	15.4	37	1.3	1.6	SSW	0.27	9.7	SW	15.9	14.3	14.3	---	762.3	0.00	0.0	478	6.85	533	0.017	0.000	13.9	38	-0.2	12.4	0.23	234	1	100.0	10
05/10/21	10:10	16.4	16.5	16.3	38	2.0	1.6	SSE	0.27	11.3	SSE	16.4	14.8	14.8	---	762.4	0.00	0.0	419	6.01	552	0.014	0.000	14.0	38	-0.1	12.5	0.00	234	1	100.0	10
05/10/21	10:20	16.8	16.9	16.5	36	1.7	3.2	SSE	0.54	11.3	SSW	16.8	15.1	15.1	---	762.4	0.00	0.0	507	7.27	592	0.010	0.000	14.1	38	-0.0	12.6	0.00	234	1	100.0	10
05/10/21	10:30	17.1	17.2	16.9	35	1.5	3.2	S	0.54	30.6	WSW	17.1	15.3	15.3	---	762.4	0.00	0.0	571	8.19	636	0.009	0.000	14.2	38	0.0	12.7	0.00	235	1	100.0	10
05/10/21	10:40	17.1	17.1	17.0	37	2.2	1.6	S	0.27	17.7	SSW	17.1	15.4	15.4	---	762.4	0.00	0.0	540	7.74	677	0.009	0.000	14.2	38	0.1	12.7	0.00	234	1	100.0	10
05/10/21	10:50	17.2	17.3	17.1	36	2.0	1.6	SSW	0.27	17.7	SE	17.2	15.5	15.5	---	762.4	0.00	0.0	635	9.10	714	0.008	0.000	14.3	37	-0.2	12.8	0.00	234	1	100.0	10
05/10/21	11:00	17.5	17.7	17.3	36	2.2	3.2	S	0.54	17.7	S	17.5	15.8	15.8	---	762.5	0.00	0.0	686	9.83	733	0.006	0.000	14.5	37	-0.0	12.9	0.33	233	1	100.0	10
05/10/21	11:10	17.7	17.8	17.6	34	1.6	1.6	S	0.27	8.0	S	17.7	15.9	15.9	---	762.4	0.00	0.0	605	8.67	694	0.004	0.000	14.6	37	0.1	13.0	0.00	232	1	100.0	10
05/10/21	11:20	17.8	17.9	17.8	36	2.5	3.2	WSW	0.54	16.1	SW	17.8	16.1	16.1	---	762.4	0.00	0.0	584	8.37	677	0.003	0.000	14.7	37	0.2	13.1	0.00	234	1	100.0	10
05/10/21	11:30	18.0	18.1	17.9	33	1.5	1.6	SSW	0.27	17.7	SSW	18.0	16.1	16.1	---	762.4	0.00	0.0	658	9.43	735	0.002	0.000	14.8	37	0.3	13.2	0.00	235	1	100.0	10
05/10/21	11:40	18.1	18.2	17.9	34	1.9	1.6	S	0.27	9.7	SE	18.1	16.2	16.2	---	762.4	0.00	0.0	711	10.19	833	0.002	0.000	14.9	37	0.4	13.3	0.00	234	1	100.0	10
05/10/21	11:50	18.3	18.4	18.2	34	2.2	3.2	S	0.54	14.5	SW	18.3	16.5	16.5	---	762.3	0.00	0.0	774	11.10	814	0.000	0.000	15.1	37	0.5	13.5	0.00	233	1	100.0	10
05/10/21	12:00	18.5	18.6	18.3	34	2.3	1.6	S	0.27	9.7	S	18.5	16.7	16.7	---	762.3	0.00	0.0	749	10.74	821	0.000	0.001	15.2	37	0.6	13.6	0.41	234	1	100.0	10
05/10/21	12:10	18.7	18.8	18.6	35	2.9	3.2	S	0.54	14.5	SSW	18.7	17.0	17.0	---	762.3	0.00	0.0	675	9.68	823	0.000	0.003	15.3	36	0.3	13.7	0.00	233	1	100.0	10
05/10/21	12:20	18.7	18.8	18.6	34	2.5	1.6	SSE	0.27	9.7	SE	18.7	16.9	16.9	---	762.2	0.00	0.0	765	10.97	854	0.000	0.002	15.5	36	0.5	13.8	0.00	234	1	100.0	10
05/10/21	12:30	18.8	18.8	18.7	32	1.8	1.6	SSE	0.27	11.3	SSE	18.8	16.9	16.9	---	762.2	0.00	0.0	908	13.02	1018	0.000	0.003	15.7	36	0.6	14.0	0.00	233	1	100.0	10
05/10/21	12:40	18.6	18.7	18.5	34	2.4	1.6	SSE	0.27	11.3	SE	18.6	16.8	16.8	---	762.1	0.00	0.0	673	9.65	722	0.000	0.002	15.8	36	0.7	14.1	0.00	234	1	100.0	10
05/10/21	12:50	18.5	18.6	18.3	33	1.9	3.2	SSE	0.54	11.3	SW	18.5	16.6	16.6	---	762.1	0.00	0.0	725	10.39	777	0.000	0.001	15.9	36	0.9	14.2	0.00	233	1	100.0	10
05/10/21	13:00	18.8	18.9	18.6	32	1.7	1.6	S	0.27	9.7	SSE	18.8	16.9	16.9	---	762.1	0.00	0.0	829	11.88	856	0.000	0.003	16.1	36	1.0	14.3	0.43	232	1	100.0	10
05/10/21	13:10	18.9	19.2	18.7	31	1.4	3.2	SW	0.54	17.7	SSE	18.9	17.0	17.0	---	762.1	0.00	0.0	814	11.67	868	0.000	0.004	16.2	36	1.1	14.5	0.00	234	1	100.0	10
05/10/21	13:20	18.9	19.1	18.8	30	0.9	3.2	WSW	0.54	20.9	WSW	18.9	16.9	16.9	---	762.0	0.00	0.0	611	8.76	895	0.000	0.004	16.3	35	0.8	14.6	0.00	233	1	100.0	10
05/10/21	13:30	18.9	19.1	18.8	31	1.4	1.6	ESE	0.27	8.0	ESE	18.9	17.0	17.0	---	762.0	0.00	0.0	814	11.67	875	0.000	0.004	16.4	35	0.9	14.7	0.00	234	1	100.0	10
05/10/21	13:40	19.3	19.6	19.1	32	2.2	1.6	SSE	0.27	16.1	SE	19.3	17.4	17.4	---	762.0	0.00	0.0	831	11.91	844	0.000	0.007	16.6	35	1.1	14.8	0.00	233	1	100.0	10
05/10/21	13:50	19.5	19.7	19.3	29	1.0	1.6	SSE	0.27	11.3	S	19.5	17.4	17.4	---	761.9	0.00	0.0	823	11.80	826	0.000	0.008	16.7	35	1.2	14.9	0.00	234	1	100.0	10
05/10/21	14:00	19.3	19.6	19.2	30	1.3	1.6	S	0.27	17.7	ESE	19.3	17.3	17.3	---	761.7	0.00	0.0	822	11.78	831	0.000	0.007	16.8	35	1.3	15.1	0.46	234	1	100.0	10
05/10/21	14:10	19.5	19.6	19.4	30	1.4	1.6	E	0.27	9.7	SW	19.5	17.5	17.5	---	761.6	0.00	0.0	843	12.08	854	0.000	0.008	17.0	34	1.0	15.2	0.00	234	1	100.0	10
05/10/21	14:20	19.7	19.9	19.5	28	0.7	1.6	SW	0.27	14.5	ESE	19.7	17.6	17.6	---	761.6	0.00	0.0	849	12.17	865	0.000	0.010	17.1	34	1.1	15.3	0.00	234	1	100.0	10
05/10/21	14:30	19.8	20.1	19.6	29	1.2	3.2	S	0.54	29.0	S	19.8	17.7	17.7	---	761.5	0.00	0.0	816	11.70	977	0.000	0.010	17.3	34	1.3	15.4	0.00	233	1	100.0	10
05/10/21	14:40	20.1	20.2	19.9	30	1.9	1.6	ESE	0.27	9.7	E	20.1	18.1	18.1	---	761.5	0.00	0.0	555	7.96	991	0.000	0.012	17.4	34	1.4	15.6	0.00	234	1	100.0	10
05/10/21	14:50	19.6	20.0	19.1	31	2.0	3.2	ESE	0.54	14.5	ESE	19.6	17.6	17.6	---	761.5	0.00	0.0	277	3.97	587	0.000	0.008	17.6	34	1.5	15.7	0.00	234	1	100.0	10
05/10/21	15:00	19.1	19.3	19.0	30	1.1	0.0	ESE	0.00	3.2	ESE	19.1	17.1	17.1	---	761.4	0.00	0.0	278	3.99	336	0.000	0.005	17.7	34	1.6	15.9	0.36	234	1	100.0	10
05/10/21	15:10	19.4	19.5	19.3	30	1.4	1.6	S	0.27	12.9	S	19.4	17.4	17.4	---	761.4	0.00	0.0	300	4.30	318	0.000	0.008	17.8	33	1.3	15.9	0.00	232	1	100.0	10
05/10/21	15:20	19.5	19.6	19.3	29	1.0	1.6	S	0.27	19.3	SE	19.5	17.4	17.4	---	761.6	0.00	0.0	306	4.39	320	0.000	0.008	17.9	33	1.4	16.0	0.00	232	1	100.0	10
05/10/21	15:30	19.4	19.6	19.3	28	0.4	1.6	W	0.27	8.0	W	19.4	17.2	17.2	---	761.6	0.00	0.0	292	4.19	309	0.000	0.008	18.0	33	1.5	16.1	0.00	234	1	100.0	10
05/10/21	15:40	19.1	19.3	18.9	30	1.1	1.6	SSW	0.27	17.7	W	19.1	17.1	17.1	---	761.7	0.00	0.0	260	3.73	272	0.000	0.005	18.1	33	1.6	16.2	0.00	234	1	100.0	10
05/10/21	15:50	19.2	19.6	18.9	28	0.3	3.2	S	0.54	12.9	S	19.2	17.0	17.0	---	761.7	0.00	0.0	275	3.94	320	0.000	0.006	18.2	33	1.7	16.3	0.00	235	1	100.0	10
05/10/21	16:00	19.4	19.7	19.2	30	1.4	1.6	S	0.27	11.3	SE	19.4	17.4	17.4	---	761.8	0.00	0.0	220	3.15	244	0.000	0.007	18.3	33	1.7	16.4	0.18	234	1	100.0	10
05/10/21	16:10	20.2	20.3	19.7	28	1.1	3.2	SSW	0.54	11.3	SW	20.2	18.1	18.1	---	761.9	0.00	0.0	299	4.29	352	0.000	0.013	18.4	33	1.8	16.5	0.00	233	1	100.0	10
05/10/21	16:20	19.8	20.3	19.6	29	1.3	3.2	SSW	0.54	9.7	SSW	19.8	17.7	17.7	---	762.0	0.00	0.0	431	6.18	687	0.000	0.010	18.5	33	1.9	16.6	0.00	234	1	100.0	10
05/10/21	16:30	19.6	19.8	19.4	33	2.9	3.2	SE	0.54	16.1	ESE	19.6	17.8	17.8	---	762.3	0.00	0.0	551	7.90	721	0.000	0.009	18.6	33	2.0	16.7	0.00	234	1	100.0	10
05/10/21	16:40	19.5	19.8	18.7	32	2.4	4.8	SE	0.80	17.7	SE	19.5	17.6	17.6	---	762.4	0.00	0.0	332	4.76	728	0.000	0.008	18.6	33	2.0	16.8	0.00	234	1	100.0	10
05/10/21	16:50	17.7	18.6	16.9	34	1.6	6.4	SSE	1.07	20.9	SSE	17.7	15.8	15.8	---	762.8	0.00	0.0	189	2.71	230	0.005	0.000	18.7	33	2.1	16.9	0.00	232	1	100.0	10
05/10/21	17:00	16.7	16.9	16.4	34	0.7	3.2	SSE	0.54	22.5	SSE	16.7	14.8	14.8	---	762.9	0.00	0.0	181	2.59	230	0.012	0.000	18.8	33	2.2	16.9	0.23	233	1	100.0	10
05/10/21	17:10	16.2	16.4	15.8	36	1.1	6.4	SSE	1.07	19.3	SSE	16.2	14.4	14.4	---	762.8	0.00	0.0	171	2.45	211	0.015	0.000	18.8	33	2.2	17.0	0.00	234	1	100.0	10
05/10/21	17:20	15.6	15.8	15.4	39	1.7	3.2	SSE	0.54	24.1	SSE	15.6	14.1	14.1	---	763.4	0.00	0.0	86	1.23	109	0.019	0.000	18.8	33	2.2	17.0	0.00	234	1	100.0	10
05/10/21	17:30	14.8	15.3	14.2	43	2.4	4.8	SSE	0.80	22.5	SSE	14.8	13.5	13.5	---	763.8	0.00	0.0	76	1.09	84	0.024	0.000	18.8	33	2.2	17.0	0.00	234	1	100.0	10
05/10/21	17:40	14.0	14.2	13.8	47	2.9	1.6	SE	0.27	14.5	SE	14.0	12.8	12.8	---	764.0	0.00	0.0	100	1.43	111	0.030	0.000	18.8	33	2.2	17.0	0.00	234	1	100.0	10
05/10/21	17:50	13.7	13.8	13.6	48	2.9	1.6	S	0.27	19.3	SSW	13.7	12.6	12.6	---	764.2	0.00	0.0	75	1.08	111	0.032	0.000	18.8	33	2.2	17.0	0.00	233	1	100.0	10
05/10/21	18:00	13.4	13.6	13.1	51	3.4	1.6	S	0.27	16.1	SSW	13.4	12.4	12.4	---	764.7	0.00	0.0	39	0.56	44	0.034	0.000	18.8	33	2.2	17.0	0.08	233	1	100.0	10
05/10/21	18:10	12.7	13.1	11.9	57	4.4	3.2	SSE	0.54	19.3	SE	12.7	11.9	11.9	---	765.1	0.00	0.0	24	0.34	32	0.039	0.000	18.8	34	2.6	17.1	0.00	234	1	100.0	10
05/10/21	18:20	11.4	11.9	10.9	61	4.2	3.2	SSE	0.54	20.9	SSE	11.4	10.9	10.9	---	765.7	0.00	0.0	21	0.30	26	0.048	0.000	18.8	34	2.6	17.1	0.00	234	1	100.0	10
05/10/21	18:30	10.3	10.8	9.8	62	3.3	6.4	SE	1.07	30.6	SE	9.7	9.8	9.3	---	766.1	0.00	0.0	22	0.32	25	0.056	0.000	18.8	34	2.6	17.1	0.00	234	1	100.0	10
05/10/21	18:40	9.7	9.8	9.7	62	2.8	4.8	ESE	0.80	20.9	ESE	9.5	9.3	9.1	---	766.2	0.00	0.0	16	0.23	23	0.060	0.000	18.8	34	2.6	17.0	0.00	233	1	100.0	10
05/10/21	18:50	9.8	9.8	9.7	60	2.4	3.2	ESE	0.54	14.5	ESE	9.8	9.3	9.3	---	766.6	0.00	0.0	11	0.16	11	0.059	0.000	18.7	34	2.5	16.9	0.00	234	1	100.0	10
05/10/21	19:00	9.9	10.0	9.8	58	2.0	3.2	ESE	0.54	17.7	SSE	9.9	9.4	9.4	---	766.8	0.00	0.0	9	0.13	9	0.059	0.000	18.7	35	2.9	16.9	0.03	234	1	100.0	10
05/10/21	19:10	10.1	10.2	10.0	58	2.2	1.6	ESE	0.27	9.7	ESE	10.1	9.6	9.6	---	767.0	0.00	0.0	7	0.10	7	0.057	0.000	18.6	35	2.8	16.8	0.00	235	1	100.0	10
05/10/21	19:20	10.2	10.3	10.2	57	2.1	1.6	ESE	0.27	9.7	ESE	10.2	9.7	9.7	---	767.2	0.00	0.0	6	0.09	7	0.056	0.000	18.5	35	2.7	16.7	0.00	233	1	100.0	10
05/10/21	19:30	10.3	10.3	10.2	56	1.9	1.6	ESE	0.27	8.0	ESE	10.3	9.7	9.7	---	767.5	0.00	0.0	1	0.01	5	0.056	0.000	18.4	35	2.7	16.7	0.00	233	1	100.0	10
05/10/21	19:40	10.3	10.4	10.2	54	1.4	1.6	ESE	0.27	16.1	ESE	10.3	9.7	9.7	---	767.8	0.00	0.0	0	0.00	0	0.056	0.000	18.3	35	2.6	16.6	0.00	233	1	100.0	10
05/10/21	19:50	10.6	10.6	10.4	51	0.8	3.2	ESE	0.54	16.1	ESE	10.6	9.8	9.8	---	768.0	0.00	0.0	0	0.00	0	0.054	0.000	18.3	35	2.5	16.5	0.00	234	1	100.0	10
05/10/21	20:00	10.6	10.7	10.6	48	0.0	1.6	SSE	0.27	17.7	ESE	10.6	9.8	9.8	---	768.0	0.00	0.0	0	0.00	0	0.054	0.000	18.2	35	2.5	16.4	0.00	234	1	100.0	10
05/10/21	20:10	10.7	10.7	10.6	47	-0.2	1.6	SSE	0.27	17.7	S	10.7	9.8	9.8	---	768.2	0.00	0.0	0	0.00	0	0.053	0.000	18.2	34	2.1	16.4	0.03	233	1	100.0	10
05/10/21	20:20	10.7	10.7	10.6	45	-0.8	1.6	SSE	0.27	16.1	SSE	10.7	9.8	9.8	---	768.5	0.00	0.0	0	0.00	0	0.053	0.000	18.1	34	2.0	16.3	0.00	233	1	100.0	10
05/10/21	20:30	10.7	10.8	10.7	44	-1.1	1.6	SSE	0.27	8.0	S	10.7	9.8	9.8	---	768.9	0.00	0.0	0	0.00	0	0.053	0.000	18.1	34	1.9	16.2	0.00	233	1	100.0	10
05/10/21	20:40	10.8	10.8	10.7	44	-1.0	1.6	SSE	0.27	8.0	SSE	10.8	9.9	9.9	---	769.1	0.00	0.0	0	0.00	0	0.052	0.000	18.0	34	1.9	16.2	0.00	233	1	100.0	10
05/10/21	20:50	10.7	10.7	10.6	43	-1.4	0.0	ESE	0.00	8.0	S	10.7	9.8	9.8	---	769.3	0.00	0.0	0	0.00	0	0.053	0.000	17.9	34	1.8	16.1	0.00	233	1	100.0	10
05/10/21	21:00	10.6	10.6	10.6	43	-1.5	1.6	SSE	0.27	6.4	WSW	10.6	9.7	9.7	---	769.5	0.00	0.0	0	0.00	0	0.054	0.000	17.8	34	1.7	16.0	0.05	233	1	100.0	10
05/10/21	21:10	10.6	10.6	10.6	43	-1.5	1.6	SSE	0.27	9.7	S	10.6	9.7	9.7	---	769.7	0.00	0.0	0	0.00	0	0.054	0.000	17.8	33	1.3	15.9	0.00	233	1	100.0	10
05/10/21	21:20	10.6	10.6	10.5	43	-1.5	0.0	SSE	0.00	6.4	ESE	10.6	9.7	9.7	---	770.0	0.00	0.0	0	0.00	0	0.054	0.000	17.7	33	1.2	15.8	0.00	233	1	100.0	10
05/10/21	21:30	10.5	10.5	10.4	44	-1.3	1.6	SE	0.27	16.1	SSW	10.5	9.7	9.7	---	770.2	0.00	0.0	0	0.00	0	0.054	0.000	17.7	33	1.2	15.8	0.00	234	1	100.0	10
05/10/21	21:40	10.4	10.4	10.3	45	-1.0	0.0	SE	0.00	6.4	SE	10.4	9.6	9.6	---	770.4	0.00	0.0	0	0.00	0	0.055	0.000	17.6	33	1.1	15.7	0.00	234	1	100.0	10
05/10/21	21:50	10.2	10.3	10.2	45	-1.2	0.0	ESE	0.00	9.7	SE	10.2	9.4	9.4	---	770.5	0.00	0.0	0	0.00	0	0.056	0.000	17.5	33	1.0	15.6	0.00	234	1	100.0	10
05/10/21	22:00	10.1	10.2	10.0	45	-1.3	0.0	SSE	0.00	6.4	SSE	10.1	9.4	9.4	---	770.6	0.00	0.0	0	0.00	0	0.057	0.000	17.4	33	1.0	15.6	0.05	233	1	100.0	10""")

pred = model.predict(x5)

print(pred)

[[0.14366195]]


### Predicción a las 23:00

In [10]:
x6 = convert("""05/10/21	8:10	7.7	7.9	7.4	61	0.7	0.0	---	0.00	0.0	---	7.7	7.3	7.3	---	762.1	0.00	0.0	117	1.68	200	0.074	0.000	14.1	38	-0.0	12.6	0.00	233	1	100.0	10
05/10/21	8:20	8.3	8.6	8.0	57	0.2	0.0	---	0.00	0.0	---	8.3	7.8	7.8	---	762.3	0.00	0.0	122	1.75	169	0.070	0.000	14.1	38	-0.1	12.6	0.00	234	1	100.0	10
05/10/21	8:30	8.8	8.9	8.6	57	0.7	0.0	---	0.00	0.0	---	8.8	8.3	8.3	---	762.3	0.00	0.0	94	1.35	137	0.066	0.000	14.1	38	-0.1	12.6	0.00	234	1	100.0	10
05/10/21	8:40	9.1	9.2	8.9	57	1.0	0.0	---	0.00	0.0	---	9.1	8.6	8.6	---	762.3	0.00	0.0	100	1.43	153	0.064	0.000	14.0	38	-0.1	12.5	0.00	234	1	100.0	10
05/10/21	8:50	9.3	9.4	9.2	60	1.9	0.0	---	0.00	0.0	---	9.3	8.8	8.8	---	762.2	0.00	0.0	117	1.68	186	0.063	0.000	13.9	38	-0.2	12.4	0.00	234	1	100.0	10
05/10/21	9:00	9.7	9.8	9.4	55	1.1	0.0	---	0.00	0.0	---	9.7	9.1	9.1	---	762.3	0.00	0.0	205	2.94	267	0.060	0.000	13.9	38	-0.2	12.4	0.05	234	1	100.0	10
05/10/21	9:10	10.1	10.4	9.8	56	1.7	0.0	---	0.00	0.0	---	10.1	9.5	9.5	---	762.3	0.00	0.0	409	5.86	461	0.057	0.000	13.9	38	-0.2	12.4	0.00	233	1	100.0	10
05/10/21	9:20	10.9	11.3	10.4	52	1.4	0.0	---	0.00	0.0	---	10.9	10.2	10.2	---	762.2	0.00	0.0	408	5.85	462	0.052	0.000	13.9	38	-0.2	12.4	0.00	231	1	100.0	10
05/10/21	9:30	11.9	12.4	11.4	50	1.8	0.0	---	0.00	0.0	---	11.9	11.0	11.0	---	762.3	0.00	0.0	439	6.29	464	0.045	0.000	13.9	38	-0.2	12.4	0.00	233	1	100.0	10
05/10/21	9:40	12.9	13.6	12.4	48	2.2	0.0	---	0.00	0.0	---	12.9	11.9	11.9	---	762.2	0.00	0.0	470	6.74	496	0.037	0.000	13.9	38	-0.2	12.4	0.00	234	1	100.0	10
05/10/21	9:50	14.4	15.4	13.6	41	1.3	1.6	SW	0.27	6.4	SW	14.4	13.0	13.0	---	762.2	0.00	0.0	524	7.51	566	0.027	0.000	13.9	38	-0.2	12.4	0.00	233	1	100.0	10
05/10/21	10:00	15.9	16.3	15.4	37	1.3	1.6	SSW	0.27	9.7	SW	15.9	14.3	14.3	---	762.3	0.00	0.0	478	6.85	533	0.017	0.000	13.9	38	-0.2	12.4	0.23	234	1	100.0	10
05/10/21	10:10	16.4	16.5	16.3	38	2.0	1.6	SSE	0.27	11.3	SSE	16.4	14.8	14.8	---	762.4	0.00	0.0	419	6.01	552	0.014	0.000	14.0	38	-0.1	12.5	0.00	234	1	100.0	10
05/10/21	10:20	16.8	16.9	16.5	36	1.7	3.2	SSE	0.54	11.3	SSW	16.8	15.1	15.1	---	762.4	0.00	0.0	507	7.27	592	0.010	0.000	14.1	38	-0.0	12.6	0.00	234	1	100.0	10
05/10/21	10:30	17.1	17.2	16.9	35	1.5	3.2	S	0.54	30.6	WSW	17.1	15.3	15.3	---	762.4	0.00	0.0	571	8.19	636	0.009	0.000	14.2	38	0.0	12.7	0.00	235	1	100.0	10
05/10/21	10:40	17.1	17.1	17.0	37	2.2	1.6	S	0.27	17.7	SSW	17.1	15.4	15.4	---	762.4	0.00	0.0	540	7.74	677	0.009	0.000	14.2	38	0.1	12.7	0.00	234	1	100.0	10
05/10/21	10:50	17.2	17.3	17.1	36	2.0	1.6	SSW	0.27	17.7	SE	17.2	15.5	15.5	---	762.4	0.00	0.0	635	9.10	714	0.008	0.000	14.3	37	-0.2	12.8	0.00	234	1	100.0	10
05/10/21	11:00	17.5	17.7	17.3	36	2.2	3.2	S	0.54	17.7	S	17.5	15.8	15.8	---	762.5	0.00	0.0	686	9.83	733	0.006	0.000	14.5	37	-0.0	12.9	0.33	233	1	100.0	10
05/10/21	11:10	17.7	17.8	17.6	34	1.6	1.6	S	0.27	8.0	S	17.7	15.9	15.9	---	762.4	0.00	0.0	605	8.67	694	0.004	0.000	14.6	37	0.1	13.0	0.00	232	1	100.0	10
05/10/21	11:20	17.8	17.9	17.8	36	2.5	3.2	WSW	0.54	16.1	SW	17.8	16.1	16.1	---	762.4	0.00	0.0	584	8.37	677	0.003	0.000	14.7	37	0.2	13.1	0.00	234	1	100.0	10
05/10/21	11:30	18.0	18.1	17.9	33	1.5	1.6	SSW	0.27	17.7	SSW	18.0	16.1	16.1	---	762.4	0.00	0.0	658	9.43	735	0.002	0.000	14.8	37	0.3	13.2	0.00	235	1	100.0	10
05/10/21	11:40	18.1	18.2	17.9	34	1.9	1.6	S	0.27	9.7	SE	18.1	16.2	16.2	---	762.4	0.00	0.0	711	10.19	833	0.002	0.000	14.9	37	0.4	13.3	0.00	234	1	100.0	10
05/10/21	11:50	18.3	18.4	18.2	34	2.2	3.2	S	0.54	14.5	SW	18.3	16.5	16.5	---	762.3	0.00	0.0	774	11.10	814	0.000	0.000	15.1	37	0.5	13.5	0.00	233	1	100.0	10
05/10/21	12:00	18.5	18.6	18.3	34	2.3	1.6	S	0.27	9.7	S	18.5	16.7	16.7	---	762.3	0.00	0.0	749	10.74	821	0.000	0.001	15.2	37	0.6	13.6	0.41	234	1	100.0	10
05/10/21	12:10	18.7	18.8	18.6	35	2.9	3.2	S	0.54	14.5	SSW	18.7	17.0	17.0	---	762.3	0.00	0.0	675	9.68	823	0.000	0.003	15.3	36	0.3	13.7	0.00	233	1	100.0	10
05/10/21	12:20	18.7	18.8	18.6	34	2.5	1.6	SSE	0.27	9.7	SE	18.7	16.9	16.9	---	762.2	0.00	0.0	765	10.97	854	0.000	0.002	15.5	36	0.5	13.8	0.00	234	1	100.0	10
05/10/21	12:30	18.8	18.8	18.7	32	1.8	1.6	SSE	0.27	11.3	SSE	18.8	16.9	16.9	---	762.2	0.00	0.0	908	13.02	1018	0.000	0.003	15.7	36	0.6	14.0	0.00	233	1	100.0	10
05/10/21	12:40	18.6	18.7	18.5	34	2.4	1.6	SSE	0.27	11.3	SE	18.6	16.8	16.8	---	762.1	0.00	0.0	673	9.65	722	0.000	0.002	15.8	36	0.7	14.1	0.00	234	1	100.0	10
05/10/21	12:50	18.5	18.6	18.3	33	1.9	3.2	SSE	0.54	11.3	SW	18.5	16.6	16.6	---	762.1	0.00	0.0	725	10.39	777	0.000	0.001	15.9	36	0.9	14.2	0.00	233	1	100.0	10
05/10/21	13:00	18.8	18.9	18.6	32	1.7	1.6	S	0.27	9.7	SSE	18.8	16.9	16.9	---	762.1	0.00	0.0	829	11.88	856	0.000	0.003	16.1	36	1.0	14.3	0.43	232	1	100.0	10
05/10/21	13:10	18.9	19.2	18.7	31	1.4	3.2	SW	0.54	17.7	SSE	18.9	17.0	17.0	---	762.1	0.00	0.0	814	11.67	868	0.000	0.004	16.2	36	1.1	14.5	0.00	234	1	100.0	10
05/10/21	13:20	18.9	19.1	18.8	30	0.9	3.2	WSW	0.54	20.9	WSW	18.9	16.9	16.9	---	762.0	0.00	0.0	611	8.76	895	0.000	0.004	16.3	35	0.8	14.6	0.00	233	1	100.0	10
05/10/21	13:30	18.9	19.1	18.8	31	1.4	1.6	ESE	0.27	8.0	ESE	18.9	17.0	17.0	---	762.0	0.00	0.0	814	11.67	875	0.000	0.004	16.4	35	0.9	14.7	0.00	234	1	100.0	10
05/10/21	13:40	19.3	19.6	19.1	32	2.2	1.6	SSE	0.27	16.1	SE	19.3	17.4	17.4	---	762.0	0.00	0.0	831	11.91	844	0.000	0.007	16.6	35	1.1	14.8	0.00	233	1	100.0	10
05/10/21	13:50	19.5	19.7	19.3	29	1.0	1.6	SSE	0.27	11.3	S	19.5	17.4	17.4	---	761.9	0.00	0.0	823	11.80	826	0.000	0.008	16.7	35	1.2	14.9	0.00	234	1	100.0	10
05/10/21	14:00	19.3	19.6	19.2	30	1.3	1.6	S	0.27	17.7	ESE	19.3	17.3	17.3	---	761.7	0.00	0.0	822	11.78	831	0.000	0.007	16.8	35	1.3	15.1	0.46	234	1	100.0	10
05/10/21	14:10	19.5	19.6	19.4	30	1.4	1.6	E	0.27	9.7	SW	19.5	17.5	17.5	---	761.6	0.00	0.0	843	12.08	854	0.000	0.008	17.0	34	1.0	15.2	0.00	234	1	100.0	10
05/10/21	14:20	19.7	19.9	19.5	28	0.7	1.6	SW	0.27	14.5	ESE	19.7	17.6	17.6	---	761.6	0.00	0.0	849	12.17	865	0.000	0.010	17.1	34	1.1	15.3	0.00	234	1	100.0	10
05/10/21	14:30	19.8	20.1	19.6	29	1.2	3.2	S	0.54	29.0	S	19.8	17.7	17.7	---	761.5	0.00	0.0	816	11.70	977	0.000	0.010	17.3	34	1.3	15.4	0.00	233	1	100.0	10
05/10/21	14:40	20.1	20.2	19.9	30	1.9	1.6	ESE	0.27	9.7	E	20.1	18.1	18.1	---	761.5	0.00	0.0	555	7.96	991	0.000	0.012	17.4	34	1.4	15.6	0.00	234	1	100.0	10
05/10/21	14:50	19.6	20.0	19.1	31	2.0	3.2	ESE	0.54	14.5	ESE	19.6	17.6	17.6	---	761.5	0.00	0.0	277	3.97	587	0.000	0.008	17.6	34	1.5	15.7	0.00	234	1	100.0	10
05/10/21	15:00	19.1	19.3	19.0	30	1.1	0.0	ESE	0.00	3.2	ESE	19.1	17.1	17.1	---	761.4	0.00	0.0	278	3.99	336	0.000	0.005	17.7	34	1.6	15.9	0.36	234	1	100.0	10
05/10/21	15:10	19.4	19.5	19.3	30	1.4	1.6	S	0.27	12.9	S	19.4	17.4	17.4	---	761.4	0.00	0.0	300	4.30	318	0.000	0.008	17.8	33	1.3	15.9	0.00	232	1	100.0	10
05/10/21	15:20	19.5	19.6	19.3	29	1.0	1.6	S	0.27	19.3	SE	19.5	17.4	17.4	---	761.6	0.00	0.0	306	4.39	320	0.000	0.008	17.9	33	1.4	16.0	0.00	232	1	100.0	10
05/10/21	15:30	19.4	19.6	19.3	28	0.4	1.6	W	0.27	8.0	W	19.4	17.2	17.2	---	761.6	0.00	0.0	292	4.19	309	0.000	0.008	18.0	33	1.5	16.1	0.00	234	1	100.0	10
05/10/21	15:40	19.1	19.3	18.9	30	1.1	1.6	SSW	0.27	17.7	W	19.1	17.1	17.1	---	761.7	0.00	0.0	260	3.73	272	0.000	0.005	18.1	33	1.6	16.2	0.00	234	1	100.0	10
05/10/21	15:50	19.2	19.6	18.9	28	0.3	3.2	S	0.54	12.9	S	19.2	17.0	17.0	---	761.7	0.00	0.0	275	3.94	320	0.000	0.006	18.2	33	1.7	16.3	0.00	235	1	100.0	10
05/10/21	16:00	19.4	19.7	19.2	30	1.4	1.6	S	0.27	11.3	SE	19.4	17.4	17.4	---	761.8	0.00	0.0	220	3.15	244	0.000	0.007	18.3	33	1.7	16.4	0.18	234	1	100.0	10
05/10/21	16:10	20.2	20.3	19.7	28	1.1	3.2	SSW	0.54	11.3	SW	20.2	18.1	18.1	---	761.9	0.00	0.0	299	4.29	352	0.000	0.013	18.4	33	1.8	16.5	0.00	233	1	100.0	10
05/10/21	16:20	19.8	20.3	19.6	29	1.3	3.2	SSW	0.54	9.7	SSW	19.8	17.7	17.7	---	762.0	0.00	0.0	431	6.18	687	0.000	0.010	18.5	33	1.9	16.6	0.00	234	1	100.0	10
05/10/21	16:30	19.6	19.8	19.4	33	2.9	3.2	SE	0.54	16.1	ESE	19.6	17.8	17.8	---	762.3	0.00	0.0	551	7.90	721	0.000	0.009	18.6	33	2.0	16.7	0.00	234	1	100.0	10
05/10/21	16:40	19.5	19.8	18.7	32	2.4	4.8	SE	0.80	17.7	SE	19.5	17.6	17.6	---	762.4	0.00	0.0	332	4.76	728	0.000	0.008	18.6	33	2.0	16.8	0.00	234	1	100.0	10
05/10/21	16:50	17.7	18.6	16.9	34	1.6	6.4	SSE	1.07	20.9	SSE	17.7	15.8	15.8	---	762.8	0.00	0.0	189	2.71	230	0.005	0.000	18.7	33	2.1	16.9	0.00	232	1	100.0	10
05/10/21	17:00	16.7	16.9	16.4	34	0.7	3.2	SSE	0.54	22.5	SSE	16.7	14.8	14.8	---	762.9	0.00	0.0	181	2.59	230	0.012	0.000	18.8	33	2.2	16.9	0.23	233	1	100.0	10
05/10/21	17:10	16.2	16.4	15.8	36	1.1	6.4	SSE	1.07	19.3	SSE	16.2	14.4	14.4	---	762.8	0.00	0.0	171	2.45	211	0.015	0.000	18.8	33	2.2	17.0	0.00	234	1	100.0	10
05/10/21	17:20	15.6	15.8	15.4	39	1.7	3.2	SSE	0.54	24.1	SSE	15.6	14.1	14.1	---	763.4	0.00	0.0	86	1.23	109	0.019	0.000	18.8	33	2.2	17.0	0.00	234	1	100.0	10
05/10/21	17:30	14.8	15.3	14.2	43	2.4	4.8	SSE	0.80	22.5	SSE	14.8	13.5	13.5	---	763.8	0.00	0.0	76	1.09	84	0.024	0.000	18.8	33	2.2	17.0	0.00	234	1	100.0	10
05/10/21	17:40	14.0	14.2	13.8	47	2.9	1.6	SE	0.27	14.5	SE	14.0	12.8	12.8	---	764.0	0.00	0.0	100	1.43	111	0.030	0.000	18.8	33	2.2	17.0	0.00	234	1	100.0	10
05/10/21	17:50	13.7	13.8	13.6	48	2.9	1.6	S	0.27	19.3	SSW	13.7	12.6	12.6	---	764.2	0.00	0.0	75	1.08	111	0.032	0.000	18.8	33	2.2	17.0	0.00	233	1	100.0	10
05/10/21	18:00	13.4	13.6	13.1	51	3.4	1.6	S	0.27	16.1	SSW	13.4	12.4	12.4	---	764.7	0.00	0.0	39	0.56	44	0.034	0.000	18.8	33	2.2	17.0	0.08	233	1	100.0	10
05/10/21	18:10	12.7	13.1	11.9	57	4.4	3.2	SSE	0.54	19.3	SE	12.7	11.9	11.9	---	765.1	0.00	0.0	24	0.34	32	0.039	0.000	18.8	34	2.6	17.1	0.00	234	1	100.0	10
05/10/21	18:20	11.4	11.9	10.9	61	4.2	3.2	SSE	0.54	20.9	SSE	11.4	10.9	10.9	---	765.7	0.00	0.0	21	0.30	26	0.048	0.000	18.8	34	2.6	17.1	0.00	234	1	100.0	10
05/10/21	18:30	10.3	10.8	9.8	62	3.3	6.4	SE	1.07	30.6	SE	9.7	9.8	9.3	---	766.1	0.00	0.0	22	0.32	25	0.056	0.000	18.8	34	2.6	17.1	0.00	234	1	100.0	10
05/10/21	18:40	9.7	9.8	9.7	62	2.8	4.8	ESE	0.80	20.9	ESE	9.5	9.3	9.1	---	766.2	0.00	0.0	16	0.23	23	0.060	0.000	18.8	34	2.6	17.0	0.00	233	1	100.0	10
05/10/21	18:50	9.8	9.8	9.7	60	2.4	3.2	ESE	0.54	14.5	ESE	9.8	9.3	9.3	---	766.6	0.00	0.0	11	0.16	11	0.059	0.000	18.7	34	2.5	16.9	0.00	234	1	100.0	10
05/10/21	19:00	9.9	10.0	9.8	58	2.0	3.2	ESE	0.54	17.7	SSE	9.9	9.4	9.4	---	766.8	0.00	0.0	9	0.13	9	0.059	0.000	18.7	35	2.9	16.9	0.03	234	1	100.0	10
05/10/21	19:10	10.1	10.2	10.0	58	2.2	1.6	ESE	0.27	9.7	ESE	10.1	9.6	9.6	---	767.0	0.00	0.0	7	0.10	7	0.057	0.000	18.6	35	2.8	16.8	0.00	235	1	100.0	10
05/10/21	19:20	10.2	10.3	10.2	57	2.1	1.6	ESE	0.27	9.7	ESE	10.2	9.7	9.7	---	767.2	0.00	0.0	6	0.09	7	0.056	0.000	18.5	35	2.7	16.7	0.00	233	1	100.0	10
05/10/21	19:30	10.3	10.3	10.2	56	1.9	1.6	ESE	0.27	8.0	ESE	10.3	9.7	9.7	---	767.5	0.00	0.0	1	0.01	5	0.056	0.000	18.4	35	2.7	16.7	0.00	233	1	100.0	10
05/10/21	19:40	10.3	10.4	10.2	54	1.4	1.6	ESE	0.27	16.1	ESE	10.3	9.7	9.7	---	767.8	0.00	0.0	0	0.00	0	0.056	0.000	18.3	35	2.6	16.6	0.00	233	1	100.0	10
05/10/21	19:50	10.6	10.6	10.4	51	0.8	3.2	ESE	0.54	16.1	ESE	10.6	9.8	9.8	---	768.0	0.00	0.0	0	0.00	0	0.054	0.000	18.3	35	2.5	16.5	0.00	234	1	100.0	10
05/10/21	20:00	10.6	10.7	10.6	48	0.0	1.6	SSE	0.27	17.7	ESE	10.6	9.8	9.8	---	768.0	0.00	0.0	0	0.00	0	0.054	0.000	18.2	35	2.5	16.4	0.00	234	1	100.0	10
05/10/21	20:10	10.7	10.7	10.6	47	-0.2	1.6	SSE	0.27	17.7	S	10.7	9.8	9.8	---	768.2	0.00	0.0	0	0.00	0	0.053	0.000	18.2	34	2.1	16.4	0.03	233	1	100.0	10
05/10/21	20:20	10.7	10.7	10.6	45	-0.8	1.6	SSE	0.27	16.1	SSE	10.7	9.8	9.8	---	768.5	0.00	0.0	0	0.00	0	0.053	0.000	18.1	34	2.0	16.3	0.00	233	1	100.0	10
05/10/21	20:30	10.7	10.8	10.7	44	-1.1	1.6	SSE	0.27	8.0	S	10.7	9.8	9.8	---	768.9	0.00	0.0	0	0.00	0	0.053	0.000	18.1	34	1.9	16.2	0.00	233	1	100.0	10
05/10/21	20:40	10.8	10.8	10.7	44	-1.0	1.6	SSE	0.27	8.0	SSE	10.8	9.9	9.9	---	769.1	0.00	0.0	0	0.00	0	0.052	0.000	18.0	34	1.9	16.2	0.00	233	1	100.0	10
05/10/21	20:50	10.7	10.7	10.6	43	-1.4	0.0	ESE	0.00	8.0	S	10.7	9.8	9.8	---	769.3	0.00	0.0	0	0.00	0	0.053	0.000	17.9	34	1.8	16.1	0.00	233	1	100.0	10
05/10/21	21:00	10.6	10.6	10.6	43	-1.5	1.6	SSE	0.27	6.4	WSW	10.6	9.7	9.7	---	769.5	0.00	0.0	0	0.00	0	0.054	0.000	17.8	34	1.7	16.0	0.05	233	1	100.0	10
05/10/21	21:10	10.6	10.6	10.6	43	-1.5	1.6	SSE	0.27	9.7	S	10.6	9.7	9.7	---	769.7	0.00	0.0	0	0.00	0	0.054	0.000	17.8	33	1.3	15.9	0.00	233	1	100.0	10
05/10/21	21:20	10.6	10.6	10.5	43	-1.5	0.0	SSE	0.00	6.4	ESE	10.6	9.7	9.7	---	770.0	0.00	0.0	0	0.00	0	0.054	0.000	17.7	33	1.2	15.8	0.00	233	1	100.0	10
05/10/21	21:30	10.5	10.5	10.4	44	-1.3	1.6	SE	0.27	16.1	SSW	10.5	9.7	9.7	---	770.2	0.00	0.0	0	0.00	0	0.054	0.000	17.7	33	1.2	15.8	0.00	234	1	100.0	10
05/10/21	21:40	10.4	10.4	10.3	45	-1.0	0.0	SE	0.00	6.4	SE	10.4	9.6	9.6	---	770.4	0.00	0.0	0	0.00	0	0.055	0.000	17.6	33	1.1	15.7	0.00	234	1	100.0	10
05/10/21	21:50	10.2	10.3	10.2	45	-1.2	0.0	ESE	0.00	9.7	SE	10.2	9.4	9.4	---	770.5	0.00	0.0	0	0.00	0	0.056	0.000	17.5	33	1.0	15.6	0.00	234	1	100.0	10
05/10/21	22:00	10.1	10.2	10.0	45	-1.3	0.0	SSE	0.00	6.4	SSE	10.1	9.4	9.4	---	770.6	0.00	0.0	0	0.00	0	0.057	0.000	17.4	33	1.0	15.6	0.05	233	1	100.0	10
05/10/21	22:10	9.9	10.1	9.8	46	-1.2	0.0	---	0.00	0.0	---	9.9	9.2	9.2	---	770.8	0.00	0.0	0	0.00	0	0.058	0.000	17.4	33	0.9	15.5	0.00	234	1	100.0	10
05/10/21	22:20	9.7	9.8	9.6	48	-0.8	0.0	SSE	0.00	3.2	S	9.7	9.1	9.1	---	770.9	0.00	0.0	0	0.00	0	0.060	0.000	17.3	33	0.9	15.4	0.00	234	1	100.0	10
05/10/21	22:30	9.4	9.6	9.3	48	-1.0	0.0	ESE	0.00	6.4	ESE	9.4	8.8	8.8	---	771.2	0.00	0.0	0	0.00	0	0.062	0.000	17.2	33	0.8	15.3	0.00	234	1	100.0	10
05/10/21	22:40	9.2	9.3	9.1	49	-1.0	0.0	SE	0.00	4.8	SSW	9.2	8.5	8.5	---	771.1	0.00	0.0	0	0.00	0	0.064	0.000	17.2	33	0.8	15.3	0.00	234	1	100.0	10
05/10/21	22:50	8.9	9.1	8.8	49	-1.3	1.6	SE	0.27	9.7	ESE	8.9	8.2	8.2	---	771.6	0.00	0.0	0	0.00	0	0.066	0.000	17.2	33	0.7	15.3	0.00	232	1	100.0	10
05/10/21	23:00	8.8	8.9	8.7	48	-1.6	1.6	ESE	0.27	9.7	ESE	8.8	8.2	8.2	---	771.8	0.00	0.0	0	0.00	0	0.066	0.000	17.1	33	0.6	15.2	0.03	233	1	100.0	10""")

pred = model.predict(x6)

print(pred)

[[0.17958513]]


### Predicción a las 00:00

In [11]:
x7 = convert("""05/10/21	9:10	10.1	10.4	9.8	56	1.7	0.0	---	0.00	0.0	---	10.1	9.5	9.5	---	762.3	0.00	0.0	409	5.86	461	0.057	0.000	13.9	38	-0.2	12.4	0.00	233	1	100.0	10
05/10/21	9:20	10.9	11.3	10.4	52	1.4	0.0	---	0.00	0.0	---	10.9	10.2	10.2	---	762.2	0.00	0.0	408	5.85	462	0.052	0.000	13.9	38	-0.2	12.4	0.00	231	1	100.0	10
05/10/21	9:30	11.9	12.4	11.4	50	1.8	0.0	---	0.00	0.0	---	11.9	11.0	11.0	---	762.3	0.00	0.0	439	6.29	464	0.045	0.000	13.9	38	-0.2	12.4	0.00	233	1	100.0	10
05/10/21	9:40	12.9	13.6	12.4	48	2.2	0.0	---	0.00	0.0	---	12.9	11.9	11.9	---	762.2	0.00	0.0	470	6.74	496	0.037	0.000	13.9	38	-0.2	12.4	0.00	234	1	100.0	10
05/10/21	9:50	14.4	15.4	13.6	41	1.3	1.6	SW	0.27	6.4	SW	14.4	13.0	13.0	---	762.2	0.00	0.0	524	7.51	566	0.027	0.000	13.9	38	-0.2	12.4	0.00	233	1	100.0	10
05/10/21	10:00	15.9	16.3	15.4	37	1.3	1.6	SSW	0.27	9.7	SW	15.9	14.3	14.3	---	762.3	0.00	0.0	478	6.85	533	0.017	0.000	13.9	38	-0.2	12.4	0.23	234	1	100.0	10
05/10/21	10:10	16.4	16.5	16.3	38	2.0	1.6	SSE	0.27	11.3	SSE	16.4	14.8	14.8	---	762.4	0.00	0.0	419	6.01	552	0.014	0.000	14.0	38	-0.1	12.5	0.00	234	1	100.0	10
05/10/21	10:20	16.8	16.9	16.5	36	1.7	3.2	SSE	0.54	11.3	SSW	16.8	15.1	15.1	---	762.4	0.00	0.0	507	7.27	592	0.010	0.000	14.1	38	-0.0	12.6	0.00	234	1	100.0	10
05/10/21	10:30	17.1	17.2	16.9	35	1.5	3.2	S	0.54	30.6	WSW	17.1	15.3	15.3	---	762.4	0.00	0.0	571	8.19	636	0.009	0.000	14.2	38	0.0	12.7	0.00	235	1	100.0	10
05/10/21	10:40	17.1	17.1	17.0	37	2.2	1.6	S	0.27	17.7	SSW	17.1	15.4	15.4	---	762.4	0.00	0.0	540	7.74	677	0.009	0.000	14.2	38	0.1	12.7	0.00	234	1	100.0	10
05/10/21	10:50	17.2	17.3	17.1	36	2.0	1.6	SSW	0.27	17.7	SE	17.2	15.5	15.5	---	762.4	0.00	0.0	635	9.10	714	0.008	0.000	14.3	37	-0.2	12.8	0.00	234	1	100.0	10
05/10/21	11:00	17.5	17.7	17.3	36	2.2	3.2	S	0.54	17.7	S	17.5	15.8	15.8	---	762.5	0.00	0.0	686	9.83	733	0.006	0.000	14.5	37	-0.0	12.9	0.33	233	1	100.0	10
05/10/21	11:10	17.7	17.8	17.6	34	1.6	1.6	S	0.27	8.0	S	17.7	15.9	15.9	---	762.4	0.00	0.0	605	8.67	694	0.004	0.000	14.6	37	0.1	13.0	0.00	232	1	100.0	10
05/10/21	11:20	17.8	17.9	17.8	36	2.5	3.2	WSW	0.54	16.1	SW	17.8	16.1	16.1	---	762.4	0.00	0.0	584	8.37	677	0.003	0.000	14.7	37	0.2	13.1	0.00	234	1	100.0	10
05/10/21	11:30	18.0	18.1	17.9	33	1.5	1.6	SSW	0.27	17.7	SSW	18.0	16.1	16.1	---	762.4	0.00	0.0	658	9.43	735	0.002	0.000	14.8	37	0.3	13.2	0.00	235	1	100.0	10
05/10/21	11:40	18.1	18.2	17.9	34	1.9	1.6	S	0.27	9.7	SE	18.1	16.2	16.2	---	762.4	0.00	0.0	711	10.19	833	0.002	0.000	14.9	37	0.4	13.3	0.00	234	1	100.0	10
05/10/21	11:50	18.3	18.4	18.2	34	2.2	3.2	S	0.54	14.5	SW	18.3	16.5	16.5	---	762.3	0.00	0.0	774	11.10	814	0.000	0.000	15.1	37	0.5	13.5	0.00	233	1	100.0	10
05/10/21	12:00	18.5	18.6	18.3	34	2.3	1.6	S	0.27	9.7	S	18.5	16.7	16.7	---	762.3	0.00	0.0	749	10.74	821	0.000	0.001	15.2	37	0.6	13.6	0.41	234	1	100.0	10
05/10/21	12:10	18.7	18.8	18.6	35	2.9	3.2	S	0.54	14.5	SSW	18.7	17.0	17.0	---	762.3	0.00	0.0	675	9.68	823	0.000	0.003	15.3	36	0.3	13.7	0.00	233	1	100.0	10
05/10/21	12:20	18.7	18.8	18.6	34	2.5	1.6	SSE	0.27	9.7	SE	18.7	16.9	16.9	---	762.2	0.00	0.0	765	10.97	854	0.000	0.002	15.5	36	0.5	13.8	0.00	234	1	100.0	10
05/10/21	12:30	18.8	18.8	18.7	32	1.8	1.6	SSE	0.27	11.3	SSE	18.8	16.9	16.9	---	762.2	0.00	0.0	908	13.02	1018	0.000	0.003	15.7	36	0.6	14.0	0.00	233	1	100.0	10
05/10/21	12:40	18.6	18.7	18.5	34	2.4	1.6	SSE	0.27	11.3	SE	18.6	16.8	16.8	---	762.1	0.00	0.0	673	9.65	722	0.000	0.002	15.8	36	0.7	14.1	0.00	234	1	100.0	10
05/10/21	12:50	18.5	18.6	18.3	33	1.9	3.2	SSE	0.54	11.3	SW	18.5	16.6	16.6	---	762.1	0.00	0.0	725	10.39	777	0.000	0.001	15.9	36	0.9	14.2	0.00	233	1	100.0	10
05/10/21	13:00	18.8	18.9	18.6	32	1.7	1.6	S	0.27	9.7	SSE	18.8	16.9	16.9	---	762.1	0.00	0.0	829	11.88	856	0.000	0.003	16.1	36	1.0	14.3	0.43	232	1	100.0	10
05/10/21	13:10	18.9	19.2	18.7	31	1.4	3.2	SW	0.54	17.7	SSE	18.9	17.0	17.0	---	762.1	0.00	0.0	814	11.67	868	0.000	0.004	16.2	36	1.1	14.5	0.00	234	1	100.0	10
05/10/21	13:20	18.9	19.1	18.8	30	0.9	3.2	WSW	0.54	20.9	WSW	18.9	16.9	16.9	---	762.0	0.00	0.0	611	8.76	895	0.000	0.004	16.3	35	0.8	14.6	0.00	233	1	100.0	10
05/10/21	13:30	18.9	19.1	18.8	31	1.4	1.6	ESE	0.27	8.0	ESE	18.9	17.0	17.0	---	762.0	0.00	0.0	814	11.67	875	0.000	0.004	16.4	35	0.9	14.7	0.00	234	1	100.0	10
05/10/21	13:40	19.3	19.6	19.1	32	2.2	1.6	SSE	0.27	16.1	SE	19.3	17.4	17.4	---	762.0	0.00	0.0	831	11.91	844	0.000	0.007	16.6	35	1.1	14.8	0.00	233	1	100.0	10
05/10/21	13:50	19.5	19.7	19.3	29	1.0	1.6	SSE	0.27	11.3	S	19.5	17.4	17.4	---	761.9	0.00	0.0	823	11.80	826	0.000	0.008	16.7	35	1.2	14.9	0.00	234	1	100.0	10
05/10/21	14:00	19.3	19.6	19.2	30	1.3	1.6	S	0.27	17.7	ESE	19.3	17.3	17.3	---	761.7	0.00	0.0	822	11.78	831	0.000	0.007	16.8	35	1.3	15.1	0.46	234	1	100.0	10
05/10/21	14:10	19.5	19.6	19.4	30	1.4	1.6	E	0.27	9.7	SW	19.5	17.5	17.5	---	761.6	0.00	0.0	843	12.08	854	0.000	0.008	17.0	34	1.0	15.2	0.00	234	1	100.0	10
05/10/21	14:20	19.7	19.9	19.5	28	0.7	1.6	SW	0.27	14.5	ESE	19.7	17.6	17.6	---	761.6	0.00	0.0	849	12.17	865	0.000	0.010	17.1	34	1.1	15.3	0.00	234	1	100.0	10
05/10/21	14:30	19.8	20.1	19.6	29	1.2	3.2	S	0.54	29.0	S	19.8	17.7	17.7	---	761.5	0.00	0.0	816	11.70	977	0.000	0.010	17.3	34	1.3	15.4	0.00	233	1	100.0	10
05/10/21	14:40	20.1	20.2	19.9	30	1.9	1.6	ESE	0.27	9.7	E	20.1	18.1	18.1	---	761.5	0.00	0.0	555	7.96	991	0.000	0.012	17.4	34	1.4	15.6	0.00	234	1	100.0	10
05/10/21	14:50	19.6	20.0	19.1	31	2.0	3.2	ESE	0.54	14.5	ESE	19.6	17.6	17.6	---	761.5	0.00	0.0	277	3.97	587	0.000	0.008	17.6	34	1.5	15.7	0.00	234	1	100.0	10
05/10/21	15:00	19.1	19.3	19.0	30	1.1	0.0	ESE	0.00	3.2	ESE	19.1	17.1	17.1	---	761.4	0.00	0.0	278	3.99	336	0.000	0.005	17.7	34	1.6	15.9	0.36	234	1	100.0	10
05/10/21	15:10	19.4	19.5	19.3	30	1.4	1.6	S	0.27	12.9	S	19.4	17.4	17.4	---	761.4	0.00	0.0	300	4.30	318	0.000	0.008	17.8	33	1.3	15.9	0.00	232	1	100.0	10
05/10/21	15:20	19.5	19.6	19.3	29	1.0	1.6	S	0.27	19.3	SE	19.5	17.4	17.4	---	761.6	0.00	0.0	306	4.39	320	0.000	0.008	17.9	33	1.4	16.0	0.00	232	1	100.0	10
05/10/21	15:30	19.4	19.6	19.3	28	0.4	1.6	W	0.27	8.0	W	19.4	17.2	17.2	---	761.6	0.00	0.0	292	4.19	309	0.000	0.008	18.0	33	1.5	16.1	0.00	234	1	100.0	10
05/10/21	15:40	19.1	19.3	18.9	30	1.1	1.6	SSW	0.27	17.7	W	19.1	17.1	17.1	---	761.7	0.00	0.0	260	3.73	272	0.000	0.005	18.1	33	1.6	16.2	0.00	234	1	100.0	10
05/10/21	15:50	19.2	19.6	18.9	28	0.3	3.2	S	0.54	12.9	S	19.2	17.0	17.0	---	761.7	0.00	0.0	275	3.94	320	0.000	0.006	18.2	33	1.7	16.3	0.00	235	1	100.0	10
05/10/21	16:00	19.4	19.7	19.2	30	1.4	1.6	S	0.27	11.3	SE	19.4	17.4	17.4	---	761.8	0.00	0.0	220	3.15	244	0.000	0.007	18.3	33	1.7	16.4	0.18	234	1	100.0	10
05/10/21	16:10	20.2	20.3	19.7	28	1.1	3.2	SSW	0.54	11.3	SW	20.2	18.1	18.1	---	761.9	0.00	0.0	299	4.29	352	0.000	0.013	18.4	33	1.8	16.5	0.00	233	1	100.0	10
05/10/21	16:20	19.8	20.3	19.6	29	1.3	3.2	SSW	0.54	9.7	SSW	19.8	17.7	17.7	---	762.0	0.00	0.0	431	6.18	687	0.000	0.010	18.5	33	1.9	16.6	0.00	234	1	100.0	10
05/10/21	16:30	19.6	19.8	19.4	33	2.9	3.2	SE	0.54	16.1	ESE	19.6	17.8	17.8	---	762.3	0.00	0.0	551	7.90	721	0.000	0.009	18.6	33	2.0	16.7	0.00	234	1	100.0	10
05/10/21	16:40	19.5	19.8	18.7	32	2.4	4.8	SE	0.80	17.7	SE	19.5	17.6	17.6	---	762.4	0.00	0.0	332	4.76	728	0.000	0.008	18.6	33	2.0	16.8	0.00	234	1	100.0	10
05/10/21	16:50	17.7	18.6	16.9	34	1.6	6.4	SSE	1.07	20.9	SSE	17.7	15.8	15.8	---	762.8	0.00	0.0	189	2.71	230	0.005	0.000	18.7	33	2.1	16.9	0.00	232	1	100.0	10
05/10/21	17:00	16.7	16.9	16.4	34	0.7	3.2	SSE	0.54	22.5	SSE	16.7	14.8	14.8	---	762.9	0.00	0.0	181	2.59	230	0.012	0.000	18.8	33	2.2	16.9	0.23	233	1	100.0	10
05/10/21	17:10	16.2	16.4	15.8	36	1.1	6.4	SSE	1.07	19.3	SSE	16.2	14.4	14.4	---	762.8	0.00	0.0	171	2.45	211	0.015	0.000	18.8	33	2.2	17.0	0.00	234	1	100.0	10
05/10/21	17:20	15.6	15.8	15.4	39	1.7	3.2	SSE	0.54	24.1	SSE	15.6	14.1	14.1	---	763.4	0.00	0.0	86	1.23	109	0.019	0.000	18.8	33	2.2	17.0	0.00	234	1	100.0	10
05/10/21	17:30	14.8	15.3	14.2	43	2.4	4.8	SSE	0.80	22.5	SSE	14.8	13.5	13.5	---	763.8	0.00	0.0	76	1.09	84	0.024	0.000	18.8	33	2.2	17.0	0.00	234	1	100.0	10
05/10/21	17:40	14.0	14.2	13.8	47	2.9	1.6	SE	0.27	14.5	SE	14.0	12.8	12.8	---	764.0	0.00	0.0	100	1.43	111	0.030	0.000	18.8	33	2.2	17.0	0.00	234	1	100.0	10
05/10/21	17:50	13.7	13.8	13.6	48	2.9	1.6	S	0.27	19.3	SSW	13.7	12.6	12.6	---	764.2	0.00	0.0	75	1.08	111	0.032	0.000	18.8	33	2.2	17.0	0.00	233	1	100.0	10
05/10/21	18:00	13.4	13.6	13.1	51	3.4	1.6	S	0.27	16.1	SSW	13.4	12.4	12.4	---	764.7	0.00	0.0	39	0.56	44	0.034	0.000	18.8	33	2.2	17.0	0.08	233	1	100.0	10
05/10/21	18:10	12.7	13.1	11.9	57	4.4	3.2	SSE	0.54	19.3	SE	12.7	11.9	11.9	---	765.1	0.00	0.0	24	0.34	32	0.039	0.000	18.8	34	2.6	17.1	0.00	234	1	100.0	10
05/10/21	18:20	11.4	11.9	10.9	61	4.2	3.2	SSE	0.54	20.9	SSE	11.4	10.9	10.9	---	765.7	0.00	0.0	21	0.30	26	0.048	0.000	18.8	34	2.6	17.1	0.00	234	1	100.0	10
05/10/21	18:30	10.3	10.8	9.8	62	3.3	6.4	SE	1.07	30.6	SE	9.7	9.8	9.3	---	766.1	0.00	0.0	22	0.32	25	0.056	0.000	18.8	34	2.6	17.1	0.00	234	1	100.0	10
05/10/21	18:40	9.7	9.8	9.7	62	2.8	4.8	ESE	0.80	20.9	ESE	9.5	9.3	9.1	---	766.2	0.00	0.0	16	0.23	23	0.060	0.000	18.8	34	2.6	17.0	0.00	233	1	100.0	10
05/10/21	18:50	9.8	9.8	9.7	60	2.4	3.2	ESE	0.54	14.5	ESE	9.8	9.3	9.3	---	766.6	0.00	0.0	11	0.16	11	0.059	0.000	18.7	34	2.5	16.9	0.00	234	1	100.0	10
05/10/21	19:00	9.9	10.0	9.8	58	2.0	3.2	ESE	0.54	17.7	SSE	9.9	9.4	9.4	---	766.8	0.00	0.0	9	0.13	9	0.059	0.000	18.7	35	2.9	16.9	0.03	234	1	100.0	10
05/10/21	19:10	10.1	10.2	10.0	58	2.2	1.6	ESE	0.27	9.7	ESE	10.1	9.6	9.6	---	767.0	0.00	0.0	7	0.10	7	0.057	0.000	18.6	35	2.8	16.8	0.00	235	1	100.0	10
05/10/21	19:20	10.2	10.3	10.2	57	2.1	1.6	ESE	0.27	9.7	ESE	10.2	9.7	9.7	---	767.2	0.00	0.0	6	0.09	7	0.056	0.000	18.5	35	2.7	16.7	0.00	233	1	100.0	10
05/10/21	19:30	10.3	10.3	10.2	56	1.9	1.6	ESE	0.27	8.0	ESE	10.3	9.7	9.7	---	767.5	0.00	0.0	1	0.01	5	0.056	0.000	18.4	35	2.7	16.7	0.00	233	1	100.0	10
05/10/21	19:40	10.3	10.4	10.2	54	1.4	1.6	ESE	0.27	16.1	ESE	10.3	9.7	9.7	---	767.8	0.00	0.0	0	0.00	0	0.056	0.000	18.3	35	2.6	16.6	0.00	233	1	100.0	10
05/10/21	19:50	10.6	10.6	10.4	51	0.8	3.2	ESE	0.54	16.1	ESE	10.6	9.8	9.8	---	768.0	0.00	0.0	0	0.00	0	0.054	0.000	18.3	35	2.5	16.5	0.00	234	1	100.0	10
05/10/21	20:00	10.6	10.7	10.6	48	0.0	1.6	SSE	0.27	17.7	ESE	10.6	9.8	9.8	---	768.0	0.00	0.0	0	0.00	0	0.054	0.000	18.2	35	2.5	16.4	0.00	234	1	100.0	10
05/10/21	20:10	10.7	10.7	10.6	47	-0.2	1.6	SSE	0.27	17.7	S	10.7	9.8	9.8	---	768.2	0.00	0.0	0	0.00	0	0.053	0.000	18.2	34	2.1	16.4	0.03	233	1	100.0	10
05/10/21	20:20	10.7	10.7	10.6	45	-0.8	1.6	SSE	0.27	16.1	SSE	10.7	9.8	9.8	---	768.5	0.00	0.0	0	0.00	0	0.053	0.000	18.1	34	2.0	16.3	0.00	233	1	100.0	10
05/10/21	20:30	10.7	10.8	10.7	44	-1.1	1.6	SSE	0.27	8.0	S	10.7	9.8	9.8	---	768.9	0.00	0.0	0	0.00	0	0.053	0.000	18.1	34	1.9	16.2	0.00	233	1	100.0	10
05/10/21	20:40	10.8	10.8	10.7	44	-1.0	1.6	SSE	0.27	8.0	SSE	10.8	9.9	9.9	---	769.1	0.00	0.0	0	0.00	0	0.052	0.000	18.0	34	1.9	16.2	0.00	233	1	100.0	10
05/10/21	20:50	10.7	10.7	10.6	43	-1.4	0.0	ESE	0.00	8.0	S	10.7	9.8	9.8	---	769.3	0.00	0.0	0	0.00	0	0.053	0.000	17.9	34	1.8	16.1	0.00	233	1	100.0	10
05/10/21	21:00	10.6	10.6	10.6	43	-1.5	1.6	SSE	0.27	6.4	WSW	10.6	9.7	9.7	---	769.5	0.00	0.0	0	0.00	0	0.054	0.000	17.8	34	1.7	16.0	0.05	233	1	100.0	10
05/10/21	21:10	10.6	10.6	10.6	43	-1.5	1.6	SSE	0.27	9.7	S	10.6	9.7	9.7	---	769.7	0.00	0.0	0	0.00	0	0.054	0.000	17.8	33	1.3	15.9	0.00	233	1	100.0	10
05/10/21	21:20	10.6	10.6	10.5	43	-1.5	0.0	SSE	0.00	6.4	ESE	10.6	9.7	9.7	---	770.0	0.00	0.0	0	0.00	0	0.054	0.000	17.7	33	1.2	15.8	0.00	233	1	100.0	10
05/10/21	21:30	10.5	10.5	10.4	44	-1.3	1.6	SE	0.27	16.1	SSW	10.5	9.7	9.7	---	770.2	0.00	0.0	0	0.00	0	0.054	0.000	17.7	33	1.2	15.8	0.00	234	1	100.0	10
05/10/21	21:40	10.4	10.4	10.3	45	-1.0	0.0	SE	0.00	6.4	SE	10.4	9.6	9.6	---	770.4	0.00	0.0	0	0.00	0	0.055	0.000	17.6	33	1.1	15.7	0.00	234	1	100.0	10
05/10/21	21:50	10.2	10.3	10.2	45	-1.2	0.0	ESE	0.00	9.7	SE	10.2	9.4	9.4	---	770.5	0.00	0.0	0	0.00	0	0.056	0.000	17.5	33	1.0	15.6	0.00	234	1	100.0	10
05/10/21	22:00	10.1	10.2	10.0	45	-1.3	0.0	SSE	0.00	6.4	SSE	10.1	9.4	9.4	---	770.6	0.00	0.0	0	0.00	0	0.057	0.000	17.4	33	1.0	15.6	0.05	233	1	100.0	10
05/10/21	22:10	9.9	10.1	9.8	46	-1.2	0.0	---	0.00	0.0	---	9.9	9.2	9.2	---	770.8	0.00	0.0	0	0.00	0	0.058	0.000	17.4	33	0.9	15.5	0.00	234	1	100.0	10
05/10/21	22:20	9.7	9.8	9.6	48	-0.8	0.0	SSE	0.00	3.2	S	9.7	9.1	9.1	---	770.9	0.00	0.0	0	0.00	0	0.060	0.000	17.3	33	0.9	15.4	0.00	234	1	100.0	10
05/10/21	22:30	9.4	9.6	9.3	48	-1.0	0.0	ESE	0.00	6.4	ESE	9.4	8.8	8.8	---	771.2	0.00	0.0	0	0.00	0	0.062	0.000	17.2	33	0.8	15.3	0.00	234	1	100.0	10
05/10/21	22:40	9.2	9.3	9.1	49	-1.0	0.0	SE	0.00	4.8	SSW	9.2	8.5	8.5	---	771.1	0.00	0.0	0	0.00	0	0.064	0.000	17.2	33	0.8	15.3	0.00	234	1	100.0	10
05/10/21	22:50	8.9	9.1	8.8	49	-1.3	1.6	SE	0.27	9.7	ESE	8.9	8.2	8.2	---	771.6	0.00	0.0	0	0.00	0	0.066	0.000	17.2	33	0.7	15.3	0.00	232	1	100.0	10
05/10/21	23:00	8.8	8.9	8.7	48	-1.6	1.6	ESE	0.27	9.7	ESE	8.8	8.2	8.2	---	771.8	0.00	0.0	0	0.00	0	0.066	0.000	17.1	33	0.6	15.2	0.03	233	1	100.0	10
05/10/21	23:10	8.6	8.7	8.5	48	-1.8	1.6	SE	0.27	8.0	SE	8.6	7.9	7.9	---	771.8	0.00	0.0	0	0.00	0	0.068	0.000	16.9	33	0.5	15.1	0.00	234	1	100.0	10
05/10/21	23:20	8.3	8.5	8.2	49	-1.8	0.0	SSE	0.00	3.2	SSE	8.3	7.7	7.7	---	772.0	0.00	0.0	0	0.00	0	0.069	0.000	16.9	33	0.5	15.0	0.00	234	1	100.0	10
05/10/21	23:30	7.9	8.2	7.8	50	-1.9	0.0	---	0.00	0.0	---	7.9	7.3	7.3	---	772.2	0.00	0.0	0	0.00	0	0.072	0.000	16.8	33	0.4	14.9	0.00	234	1	100.0	10
05/10/21	23:40	7.6	7.8	7.5	50	-2.2	0.0	ESE	0.00	6.4	ESE	7.6	7.0	7.0	---	772.3	0.00	0.0	0	0.00	0	0.074	0.000	16.7	33	0.3	14.8	0.00	234	1	100.0	10
05/10/21	23:50	7.4	7.5	7.2	51	-2.1	0.0	---	0.00	0.0	---	7.4	6.8	6.8	---	772.5	0.00	0.0	0	0.00	0	0.076	0.000	16.7	32	-0.1	14.8	0.00	234	1	100.0	10
06/10/21	00:00	7.1	7.2	6.9	51	-2.4	0.0	---	0.00	0.0	---	7.1	6.4	6.4	---	772.4	0.00	0.0	0	0.00	0	0.078	0.000	16.6	32	-0.2	14.7	0.03	234	1	100.0	10""")

pred = model.predict(x7)

print(pred)

[[0.39934334]]


In [8]:
x = convert("""15/03/22   1:10   12.3   12.3   12.3    86   10.0   0.0   ---   0.00   0.0   ---   12.3   12.2   12.2    ---  1017.4  0.00   0.0     0     0.00      0     0.042   0.000   23.3    45   10.7   23.0    0.00   232    1    100.0   10 
15/03/22   1:20   12.0   12.2   11.8    86    9.7   0.0   ---   0.00   0.0   ---   12.0   11.9   11.9    ---  1017.3  0.00   0.0     0     0.00      0     0.044   0.000   23.2    45   10.6   22.8    0.00   234    1    100.0   10 
15/03/22   1:30   11.7   11.8   11.6    87    9.6   0.0   ---   0.00   0.0   ---   11.7   11.7   11.7    ---  1017.3  0.00   0.0     0     0.00      0     0.046   0.000   23.1    45   10.5   22.7    0.00   234    1    100.0   10 
15/03/22   1:40   11.7   11.7   11.6    87    9.6   0.0   ---   0.00   0.0   ---   11.7   11.7   11.7    ---  1017.1  0.00   0.0     0     0.00      0     0.046   0.000   23.0    45   10.4   22.6    0.00   235    1    100.0   10 
15/03/22   1:50   11.7   11.8   11.6    87    9.6   0.0   ---   0.00   0.0   ---   11.7   11.7   11.7    ---  1017.2  0.00   0.0     0     0.00      0     0.046   0.000   22.9    45   10.3   22.4    0.00   234    1    100.0   10 
15/03/22   2:00   11.6   11.7   11.4    87    9.5   0.0   ---   0.00   0.0   ---   11.6   11.6   11.6    ---  1017.0  0.00   0.0     0     0.00      0     0.047   0.000   22.8    45   10.2   22.3    0.00   234    1    100.0   10 
15/03/22   2:10   11.5   11.6   11.4    88    9.6   0.0   ---   0.00   0.0   ---   11.5   11.6   11.6    ---  1017.0  0.00   0.0     0     0.00      0     0.047   0.000   22.7    45   10.1   22.1    0.00   231    1    100.0   10 
15/03/22   2:20   11.3   11.5   11.2    88    9.4   0.0   ---   0.00   0.0   ---   11.3   11.4   11.4    ---  1016.8  0.00   0.0     0     0.00      0     0.049   0.000   22.6    45   10.0   21.9    0.00   234    1    100.0   10 
15/03/22   2:30   11.2   11.3   11.2    88    9.3   0.0   ---   0.00   0.0   ---   11.2   11.3   11.3    ---  1016.9  0.00   0.0     0     0.00      0     0.049   0.000   22.4    45    9.9   21.8    0.00   234    1    100.0   10 
15/03/22   2:40   11.3   11.5   11.2    88    9.4   0.0   ---   0.00   0.0   ---   11.3   11.4   11.4    ---  1016.7  0.00   0.0     0     0.00      0     0.049   0.000   22.3    46   10.1   21.7    0.00   233    1    100.0   10 
15/03/22   2:50   11.6   11.6   11.4    87    9.5   0.0   ---   0.00   0.0   ---   11.6   11.6   11.6    ---  1017.0  0.00   0.0     0     0.00      0     0.047   0.000   22.2    46   10.0   21.5    0.00   233    1    100.0   10 
15/03/22   3:00   11.6   11.6   11.4    88    9.6   0.0   ---   0.00   0.0   ---   11.6   11.6   11.6    ---  1016.8  0.00   0.0     0     0.00      0     0.047   0.000   22.2    46   10.0   21.4    0.00   233    1    100.0   10 
15/03/22   3:10   11.6   11.6   11.5    87    9.5   0.0   ---   0.00   0.0   ---   11.6   11.6   11.6    ---  1016.5  0.00   0.0     0     0.00      0     0.047   0.000   22.1    46    9.9   21.2    0.00   234    1    100.0   10 
15/03/22   3:20   11.2   11.5   10.9    87    9.1   0.0   ---   0.00   0.0   ---   11.2   11.3   11.3    ---  1016.5  0.00   0.0     0     0.00      0     0.049   0.000   21.9    46    9.7   21.0    0.00   233    1    100.0   10 
15/03/22   3:30   10.9   11.1   10.8    88    9.0   0.0   ---   0.00   0.0   ---   10.9   11.0   11.0    ---  1016.5  0.00   0.0     0     0.00      0     0.051   0.000   21.8    46    9.6   20.8    0.00   234    1    100.0   10 
15/03/22   3:40   11.2   11.2   11.1    88    9.3   0.0   ---   0.00   0.0   ---   11.2   11.2   11.2    ---  1016.8  0.00   0.0     0     0.00      0     0.050   0.000   21.7    46    9.6   20.7    0.00   234    1    100.0   10 
15/03/22   3:50   11.2   11.3   11.2    88    9.3   0.0   ---   0.00   0.0   ---   11.2   11.3   11.3    ---  1016.7  0.00   0.0     0     0.00      0     0.049   0.000   21.6    46    9.5   20.6    0.00   233    1    100.0   10 
15/03/22   4:00   11.3   11.4   11.2    86    9.1   0.0   ---   0.00   0.0   ---   11.3   11.3   11.3    ---  1016.7  0.00   0.0     0     0.00      0     0.049   0.000   21.5    46    9.4   20.4    0.00   234    1    100.0   10 
15/03/22   4:10   11.2   11.2   11.1    87    9.1   0.0   ---   0.00   0.0   ---   11.2   11.2   11.2    ---  1016.7  0.00   0.0     0     0.00      0     0.050   0.000   21.4    46    9.3   20.3    0.00   234    1    100.0   10 
15/03/22   4:20   11.2   11.2   11.1    87    9.1   0.0   ---   0.00   0.0   ---   11.2   11.2   11.2    ---  1016.7  0.00   0.0     0     0.00      0     0.050   0.000   21.3    46    9.2   20.2    0.00   233    1    100.0   10 
15/03/22   4:30   11.1   11.2   10.9    87    9.0   0.0   ---   0.00   0.0   ---   11.1   11.1   11.1    ---  1016.9  0.00   0.0     0     0.00      0     0.051   0.000   21.2    46    9.1   20.1    0.00   234    1    100.0   10 
15/03/22   4:40   10.9   10.9   10.8    88    9.0   0.0   ---   0.00   0.0   ---   10.9   10.9   10.9    ---  1016.9  0.00   0.0     0     0.00      0     0.052   0.000   21.1    46    9.0   19.9    0.00   233    1    100.0   10 
15/03/22   4:50   10.8   10.9   10.7    87    8.8   0.0   ---   0.00   0.0   ---   10.8   10.9   10.9    ---  1016.8  0.00   0.0     0     0.00      0     0.052   0.000   21.0    47    9.3   19.9    0.00   234    1    100.0   10 
15/03/22   5:00   10.6   10.7   10.5    88    8.7   0.0   ---   0.00   0.0   ---   10.6   10.6   10.6    ---  1017.0  0.00   0.0     0     0.00      0     0.054   0.000   20.9    47    9.2   19.9    0.00   234    1    100.0   10 
15/03/22   5:10   10.6   10.7   10.5    88    8.7   0.0   ---   0.00   0.0   ---   10.6   10.7   10.7    ---  1017.1  0.00   0.0     0     0.00      0     0.054   0.000   20.9    47    9.2   19.8    0.00   233    1    100.0   10 
15/03/22   5:20   10.6   10.7   10.6    88    8.7   0.0   ---   0.00   0.0   ---   10.6   10.7   10.7    ---  1017.1  0.00   0.0     0     0.00      0     0.054   0.000   20.8    47    9.0   19.8    0.00   234    1    100.0   10 
15/03/22   5:30   10.6   10.7   10.6    89    8.8   0.0   ---   0.00   0.0   ---   10.6   10.7   10.7    ---  1017.3  0.00   0.0     0     0.00      0     0.054   0.000   20.7    47    8.9   19.7    0.00   232    1    100.0   10 
15/03/22   5:40   10.6   10.6   10.3    87    8.5   0.0   ---   0.00   0.0   ---   10.6   10.6   10.6    ---  1017.3  0.00   0.0     0     0.00      0     0.054   0.000   20.6    47    8.8   19.6    0.00   233    1    100.0   10 
15/03/22   5:50   10.3   10.3   10.2    89    8.5   0.0   ---   0.00   0.0   ---   10.3   10.4   10.4    ---  1017.7  0.00   0.0     0     0.00      0     0.056   0.000   20.5    47    8.8   19.6    0.00   234    1    100.0   10 
15/03/22   6:00   10.4   10.6   10.3    88    8.5   0.0   ---   0.00   0.0   ---   10.4   10.5   10.5    ---  1017.6  0.00   0.0     0     0.00      0     0.055   0.000   20.4    47    8.7   19.4    0.00   234    1    100.0   10 
15/03/22   6:10   10.6   10.7   10.3    86    8.3   0.0   ---   0.00   0.0   ---   10.6   10.6   10.6    ---  1017.5  0.00   0.0     0     0.00      0     0.054   0.000   20.3    47    8.6   19.3    0.00   235    1    100.0   10 
15/03/22   6:20   10.2   10.3   10.1    87    8.1   0.0   ---   0.00   0.0   ---   10.2   10.2   10.2    ---  1017.7  0.00   0.0     0     0.00      0     0.057   0.000   20.2    47    8.5   19.3    0.00   233    1    100.0   10 
15/03/22   6:30   10.0   10.1    9.8    87    7.9   0.0   ---   0.00   0.0   ---   10.0   10.1   10.1    ---  1018.0  0.00   0.0     0     0.00      5     0.058   0.000   20.1    47    8.4   19.2    0.00   234    1    100.0   10 
15/03/22   6:40    9.9    9.9    9.8    88    8.0   0.0   ---   0.00   0.0   ---    9.9   10.0   10.0    ---  1018.0  0.00   0.0     7     0.10      9     0.059   0.000   20.1    47    8.4   19.2    0.00   234    1    100.0   10 
15/03/22   6:50   10.1   10.4    9.9    89    8.4   0.0   ---   0.00   0.0   ---   10.1   10.2   10.2    ---  1018.2  0.00   0.0    16     0.23     26     0.057   0.000   20.0    47    8.3   19.1    0.00   234    1    100.0   10 
15/03/22   7:00   10.6   10.9   10.4    90    9.0   0.0   ---   0.00   0.0   ---   10.6   10.7   10.7    ---  1018.2  0.00   0.0    39     0.56     51     0.054   0.000   19.9    47    8.2   19.0    0.00   233    1    100.0   10 
15/03/22   7:10   11.3   11.7   10.9    88    9.4   0.0   ---   0.00   0.0   ---   11.3   11.3   11.3    ---  1018.2  0.00   0.0    65     0.93     77     0.049   0.000   19.8    48    8.5   19.0    0.00   232    1    100.0   10 
15/03/22   7:20   12.1   12.5   11.7    87   10.0   0.0   ---   0.00   0.0   ---   12.1   12.1   12.1    ---  1018.5  0.00   0.0    93     1.33    107     0.044   0.000   19.7    48    8.4   18.8    0.00   235    1    100.0   10 
15/03/22   7:30   13.1   13.6   12.6    83   10.2   0.0    SW   0.00   6.4    SW   13.1   12.9   12.9    ---  1018.3  0.00   0.0   126     1.81    142     0.037   0.000   19.7    48    8.3   18.8    0.00   233    1    100.0   10 
15/03/22   7:40   14.1   14.7   13.7    79   10.5   0.0   ---   0.00   0.0   ---   14.1   13.9   13.9    ---  1018.6  0.00   0.0   164     2.35    183     0.029   0.000   19.7    48    8.3   18.8    0.00   234    1    100.0   10 
15/03/22   7:50   15.2   15.6   14.7    76   11.0   0.0   ---   0.00   0.0   ---   15.2   14.9   14.9    ---  1018.6  0.00   0.0   202     2.90    218     0.022   0.000   19.6    48    8.2   18.7    0.00   232    1    100.0   10 
15/03/22   8:00   16.0   16.4   15.6    73   11.2   0.0   ---   0.00   0.0   ---   16.0   15.8   15.8    ---  1018.8  0.00   0.0   238     3.41    253     0.016   0.000   19.6    48    8.2   18.7    0.08   234    1    100.0   10 
15/03/22   8:10   16.8   17.1   16.4    73   11.9   0.0   WNW   0.00   3.2   WNW   16.8   16.6   16.6    ---  1018.8  0.00   0.0   271     3.88    285     0.011   0.000   19.4    48    8.1   18.6    0.00   234    1    100.0   10 
15/03/22   8:20   17.5   17.8   17.1    70   12.0   0.0   ---   0.00   0.0   ---   17.5   17.4   17.4    ---  1018.8  0.00   0.0   302     4.33    315     0.006   0.000   19.5    49    8.5   18.7    0.00   235    1    100.0   10 
15/03/22   8:30   18.2   18.6   17.8    71   12.8   0.0   ---   0.00   0.0   ---   18.2   18.2   18.2    ---  1018.8  0.00   0.0   332     4.76    346     0.001   0.000   19.6    49    8.5   18.7    0.00   233    1    100.0   10 
15/03/22   8:40   18.9   19.2   18.6    67   12.7   0.0   WNW   0.00   3.2   WNW   18.9   18.9   18.9    ---  1018.9  0.00   0.0   363     5.20    378     0.000   0.004   19.6    49    8.5   18.7    0.00   233    1    100.0   10 
15/03/22   8:50   19.5   19.8   19.2    67   13.2   0.0   WNW   0.00   1.6   WNW   19.5   19.6   19.6    ---  1018.8  0.00   0.0   395     5.66    408     0.000   0.008   19.6    49    8.5   18.7    0.00   231    1    100.0   10 
15/03/22   9:00   20.3   20.8   19.8    65   13.5   0.0   ---   0.00   0.0   ---   20.3   20.3   20.3    ---  1018.7  0.00   0.0   423     6.06    436     0.000   0.014   19.6    49    8.6   18.8    0.20   233    1    100.0   10 
15/03/22   9:10   21.2   21.6   20.8    63   13.9   0.0   ---   0.00   0.0   ---   21.2   20.9   20.9    ---  1018.6  0.00   0.0   452     6.48    466     0.000   0.020   19.7    49    8.6   18.8    0.00   234    1    100.0   10 
15/03/22   9:20   21.8   22.1   21.7    61   14.0   0.0   WNW   0.00   1.6   WNW   21.8   21.7   21.7    ---  1018.6  0.00   0.0   481     6.90    494     0.000   0.024   19.7    49    8.6   18.8    0.00   233    1    100.0   10 
15/03/22   9:30   22.4   22.7   22.2    59   14.0   0.0   WNW   0.00   3.2   WNW   22.4   22.4   22.4    ---  1018.4  0.00   0.0   508     7.28    519     0.000   0.029   19.7    50    9.0   18.9    0.00   235    1    100.0   10 
15/03/22   9:40   23.3   23.7   22.8    54   13.4   0.0   ---   0.00   0.0   ---   23.3   23.3   23.3    ---  1018.3  0.00   0.0   534     7.65    547     0.000   0.034   19.8    50    9.1   19.1    0.00   234    1    100.0   10 
15/03/22   9:50   24.4   25.1   23.7    47   12.3   0.0   WNW   0.00   1.6   WNW   24.4   24.3   24.3    ---  1018.3  0.00   0.0   562     8.06    575     0.000   0.042   20.0    51    9.6   19.3    0.00   233    1    100.0   10 
15/03/22  10:00   25.3   25.6   25.1    48   13.5   0.0   SSW   0.00   1.6   WSW   25.3   25.2   25.2    ---  1018.2  0.00   0.0   589     8.44    598     0.000   0.049   20.1    51    9.7   19.4    0.33   233    1    100.0   10 
15/03/22  10:10   25.9   26.1   25.6    44   12.7   0.0   ESE   0.00   6.4   ENE   25.9   25.7   25.7    ---  1018.2  0.00   0.0   612     8.77    624     0.000   0.052   20.2    51    9.8   19.6    0.00   234    1    100.0   10 
15/03/22  10:20   26.1   26.1   26.0    45   13.2   0.0   ESE   0.00   4.8   ESE   26.1   25.9   25.9    ---  1018.0  0.00   0.0   614     8.80    638     0.000   0.054   20.3    51    9.9   19.6    0.00   234    1    100.0   10 
15/03/22  10:30   26.2   26.3   26.1    46   13.6   1.6   ESE   0.27   8.0    SE   26.2   26.1   26.1    ---  1018.1  0.00   0.0   654     9.38    666     0.000   0.054   20.5    51   10.0   19.8    0.00   235    1    100.0   10 
15/03/22  10:40   26.3   26.4   26.3    45   13.4   1.6   ESE   0.27  11.3    SE   26.3   26.2   26.2    ---  1018.1  0.00   0.0   676     9.69    684     0.000   0.056   20.7    50    9.9   19.9    0.00   233    1    100.0   10 
15/03/22  10:50   26.4   26.6   26.3    47   14.2   1.6   ESE   0.27   9.7   ESE   26.4   26.3   26.3    ---  1018.0  0.00   0.0   694     9.95    703     0.000   0.056   20.9    50   10.1   20.0    0.00   234    1    100.0   10 
15/03/22  11:00   27.0   27.3   26.6    45   14.1   0.0     N   0.00   4.8     N   27.0   26.8   26.8    ---  1017.7  0.00   0.0   712    10.21    717     0.000   0.060   21.1    50   10.2   20.1    0.43   234    1    100.0   10 
15/03/22  11:10   27.1   27.4   26.9    42   13.1   1.6   ESE   0.27   8.0   ESE   27.1   26.7   26.7    ---  1017.6  0.00   0.0   728    10.44    735     0.000   0.061   21.3    50   10.4   20.4    0.00   233    1    100.0   10 
15/03/22  11:20   27.0   27.1   26.9    43   13.4   3.2   ESE   0.54   8.0   ESE   27.0   26.7   26.7    ---  1017.6  0.00   0.0   741    10.62    749     0.000   0.060   21.5    49   10.3   20.6    0.00   233    1    100.0   10 
15/03/22  11:30   27.2   27.6   27.0    42   13.1   0.0   ESE   0.00   8.0   ESE   27.2   26.7   26.7    ---  1017.5  0.00   0.0   754    10.81    759     0.000   0.061   21.6    49   10.4   20.8    0.00   233    1    100.0   10 
15/03/22  11:40   27.8   27.9   27.6    38   12.2   0.0    NW   0.00   9.7   NNW   27.8   27.3   27.3    ---  1017.5  0.00   0.0   767    11.00    773     0.000   0.066   21.8    49   10.6   21.1    0.00   235    1    100.0   10 
15/03/22  11:50   28.3   28.4   27.9    37   12.2   0.0    SE   0.00   3.2    NW   28.3   27.8   27.8    ---  1017.2  0.00   0.0   779    11.17    784     0.000   0.069   22.1    49   10.8   21.4    0.00   233    1    100.0   10 
15/03/22  12:00   28.2   28.5   27.6    37   12.1   4.8   ESE   0.80  11.3   ESE   28.2   27.7   27.7    ---  1017.3  0.00   0.0   786    11.27    788     0.000   0.069   22.2    48   10.7   21.6    0.51   234    1    100.0   10 
15/03/22  12:10   27.6   28.0   27.4    40   12.8   0.0   ESE   0.00   6.4   ESE   27.6   27.2   27.2    ---  1017.0  0.00   0.0   791    11.34    795     0.000   0.064   22.4    48   10.8   21.8    0.00   234    1    100.0   10 
15/03/22  12:20   28.5   28.8   28.1    36   12.0   0.0   ---   0.00   0.0   ---   28.5   28.0   28.0    ---  1016.9  0.00   0.0   795    11.40    800     0.000   0.071   22.6    47   10.7   22.1    0.00   234    1    100.0   10 
15/03/22  12:30   29.0   29.1   28.8    36   12.4   0.0   ESE   0.00   1.6     E   29.0   28.7   28.7    ---  1016.7  0.00   0.0   800    11.47    802     0.000   0.074   22.8    47   10.9   22.4    0.00   234    1    100.0   10 
15/03/22  12:40   29.1   29.2   29.1    36   12.5   0.0   ENE   0.00   1.6   ESE   29.1   28.8   28.8    ---  1016.6  0.00   0.0   802    11.50    803     0.000   0.075   22.9    46   10.7   22.6    0.00   235    1    100.0   10 
15/03/22  12:50   29.1   29.2   29.1    34   11.6   0.0     E   0.00   1.6     E   29.1   28.6   28.6    ---  1016.4  0.00   0.0   802    11.50    803     0.000   0.075   23.2    46   10.9   22.8    0.00   234    1    100.0   10 
15/03/22  13:00   29.1   29.3   29.1    37   12.9   0.0   WNW   0.00   1.6   WNW   29.1   28.9   28.9    ---  1016.4  0.00   0.0   798    11.44    800     0.000   0.075   23.4    46   11.1   23.1    0.53   234    1    100.0   10 
15/03/22  13:10   29.6   29.7   29.4    37   13.4   0.0     W   0.00   8.0     W   29.6   29.4   29.4    ---  1016.1  0.00   0.0   795    11.40    798     0.000   0.078   23.6    46   11.3   23.3    0.00   234    1    100.0   10 
15/03/22  13:20   29.6   29.7   29.4    35   12.5   0.0     W   0.00   6.4     W   29.6   29.1   29.1    ---  1015.8  0.00   0.0   790    11.32    793     0.000   0.078   23.8    45   11.1   23.6    0.00   233    1    100.0   10 
15/03/22  13:30   29.5   29.6   29.4    34   12.0   0.0    NW   0.00   4.8    NW   29.5   28.9   28.9    ---  1015.7  0.00   0.0   783    11.22    788     0.000   0.078   23.9    45   11.3   23.8    0.00   233    1    100.0   10 
15/03/22  13:40   29.6   29.6   29.5    35   12.5   1.6     W   0.27   9.7   WSW   29.6   29.1   29.1    ---  1015.5  0.00   0.0   773    11.08    779     0.000   0.078   24.2    45   11.5   23.9    0.00   234    1    100.0   10 
15/03/22  13:50   29.7   29.8   29.6    36   13.0   1.6     W   0.27   8.0     W   29.7   29.3   29.3    ---  1015.5  0.00   0.0   762    10.92    768     0.000   0.079   24.3    44   11.3   24.1    0.00   235    1    100.0   10 
15/03/22  14:00   29.7   29.8   29.6    34   12.1   1.6   SSW   0.27   6.4   SSW   29.7   29.1   29.1    ---  1015.2  0.00   0.0   750    10.75    756     0.000   0.079   24.5    44   11.4   24.2    0.53   233    1    100.0   10 
15/03/22  14:10   29.7   29.8   29.6    34   12.1   0.0    SW   0.00   6.4    SW   29.7   29.1   29.1    ---  1015.0  0.00   0.0   736    10.55    745     0.000   0.079   24.7    44   11.6   24.4    0.00   234    1    100.0   10 
15/03/22  14:20   29.8   29.9   29.6    34   12.2   0.0    SW   0.00   6.4    SW   29.8   29.3   29.3    ---  1014.8  0.00   0.0   572     8.20    724     0.000   0.079   24.8    43   11.4   24.5    0.00   234    1    100.0   10 
15/03/22  14:30   29.7   30.0   29.6    35   12.6   0.0    SW   0.00   3.2    SW   29.7   29.3   29.3    ---  1014.6  0.00   0.0   104     1.49    279     0.000   0.079   25.0    43   11.5   24.7    0.00   234    1    100.0   10 
15/03/22  14:40   30.2   30.3   30.0    33   12.1   0.0   WNW   0.00   4.8   WNW   30.2   29.6   29.6    ---  1014.4  0.00   0.0    36     0.52     37     0.000   0.082   25.2    43   11.7   24.8    0.00   234    1    100.0   10 
15/03/22  14:50   30.5   30.9   30.3    34   12.9   0.0     W   0.00   1.6     W   30.5   29.9   29.9    ---  1014.3  0.00   0.0    35     0.50     37     0.000   0.084   25.3    42   11.5   24.9    0.00   235    1    100.0   10 
15/03/22  15:00   31.0   31.2   30.8    32   12.4   0.0     W   0.00   1.6     W   31.0   30.0   30.0    ---  1014.2  0.00   0.0    37     0.53     37     0.000   0.088   25.5    42   11.6   25.1    0.18   234    1    100.0   10 
15/03/22  15:10   30.6   30.8   30.6    34   13.0   0.0     W   0.00   3.2     W   30.6   30.1   30.1    ---  1013.8  0.00   0.0    37     0.53     37     0.000   0.085   25.6    42   11.7   25.2    0.00   234    1    100.0   10 
15/03/22  15:20   30.8   30.9   30.6    31   11.7   0.0    SW   0.00   1.6   SSW   30.8   29.8   29.8    ---  1013.7  0.00   0.0    35     0.50     35     0.000   0.087   25.8    42   11.9   25.4    0.00   233    1    100.0   10 
15/03/22  15:30   30.8   30.9   30.7    34   13.1   0.0     W   0.00   4.8     W   30.8   30.1   30.1    ---  1013.5  0.00   0.0    35     0.50     35     0.000   0.086   25.9    42   12.0   25.6    0.00   234    1    100.0   10 
15/03/22  15:40   30.6   30.7   30.6    34   12.9   0.0   WNW   0.00   6.4   WNW   30.6   30.0   30.0    ---  1013.2  0.00   0.0    39     0.56     56     0.000   0.085   26.1    42   12.2   25.8    0.00   234    1    100.0   10 
15/03/22  15:50   30.8   31.1   30.7    32   12.2   0.0   ---   0.00   0.0   ---   30.8   29.9   29.9    ---  1013.1  0.00   0.0   158     2.26    271     0.000   0.087   26.3    41   12.0   25.8    0.00   233    1    100.0   10 
15/03/22  16:00   31.4   31.7   31.1    32   12.7   0.0   ---   0.00   0.0   ---   31.4   30.4   30.4    ---  1013.1  0.00   0.0   396     5.68    461     0.000   0.091   26.4    41   12.1   25.9    0.10   235    1    100.0   10 """)

print(model.predict(x))

[[0.02034476]]
